# Monetization and Overlapping Ties in Multimodal Networks

This notebook documents for posterity the code used to clean and format data and generate models for an unpublished analysis of the relationship between the density of overlapping ties in Multi-Modal Social Networks and the presence of Monetized labor exchange.

This code is publicly shared in keeping with my commitment to transparency and reproducibility in social science research. Data to accompany this code were confidential and used by permission with approval from the Institutional Review Board of my University at the time. For replication or reproduction attempts, the data must now be obtained through the UofM ICPSR's Data Sharing for Demographic Research program. More information is available here: https://www.icpsr.umich.edu/web/pages/DSDR/index.html

The code is presented un-edited with occassional notes or clarifications.


In [ ]:
********************************************************************
**     Program Name: /home/jhull/nangrong/prog_sas/p02_nets/2000/p02_00a1.sas
**     Programmer: james r. hull
**     Start Date: 2011 03 09
**     Purpose:
**        1.) Create A1 = PAID LABOR NETWORKS # WORKERS
**
**     Input Data:
**        '/home/jhull/nangrong/data_sas/2000/current/indiv00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/hh00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/comm00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/sibs00.xpt'
**
**     Output Data:
**        1.) /home/jhull/nangrong/data_paj/2000/a1/vXX00a1.net
**
**     Notes: 
**        1.) 
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=60;

%let f=00a1;  ** names file (allows portability) **;
%let y=00; 

**********************
**  Data Libraries  **
**********************;

libname in&f.1 xport '/home/jhull/nangrong/data_sas/2000/current/indiv00.xpt';
libname in&f.2 xport '/home/jhull/nangrong/data_sas/2000/current/hh00.xpt';
libname in&f.3 xport '/home/jhull/nangrong/data_sas/2000/current/comm00.xpt';
libname in&f.4 xport '/home/jhull/nangrong/data_sas/2000/current/sibs00.xpt';


******************************
**  Create Working Dataset  **
******************************;

********************************************************************
**  Length 10
**  0 In this 1984 village                2 + Village # + House #
**  2 In this village but blt             2 + Village # + 999
**  Length 7 
**  4 Another village in Nang Rong        3 + Village #
**  8 Unknown village in Nang Rong        3 + 999999
**  5 Another district in Buriram         4 + District # + 0000
**  6 Another province                    5 + District # + 0000
**  7 Another country 
**  8 Missing/Don't know                  9999999999 or 9999999
**  Length 9
**  9 Code 2 or 3 Returning HH member
*******************************************************************;

***************************************************************************
** Stack rice harvest labor data into a child file and label by location **
***************************************************************************;

data work&f.01a;
     set in&f.2.hh00 (keep=HHID00 X6_84C: X6_84W:);
     keep HHID00 X6_86L X6_86N X6_86W LOCATION;

     length X6_86L $ 10;

     array a(1:7) X6_84C1-X6_84C7;
     array b(1:7) X6_84W1-X6_84W7;

     do i=1 to 7;
          X6_86L=a(i);
          X6_86N=1;
          X6_86W=b(i);
          LOCATION=9;
          if a(i) ne " ." then output;  * Keep only those cases with data *;
     end;
run;

data work&f.01b;
     set in&f.2.hh00 (keep=HHID00 X6_85H: X6_85N: X6_85W:);
     keep HHID00 X6_86L X6_86N X6_86W LOCATION;

     length X6_86L $ 10;

     array a(1:13) X6_85H1-X6_85H13;
     array b(1:13) X6_85N1-X6_85N13;
     array c(1:13) X6_85W1-X6_85W13;

     do i=1 to 13;
 
          ** FIX ANOTHER CRAZY ERROR IN ORIGINAL DATA WITH REVERSED CODING FOR IN-VILLAGE LABOR **;
         
          if substr(a(i),5,3) in ('020','021','170','180') then a(i)=cat("2",substr(a(i),5,6),substr(a(i),2,3));

          if a(i)="9999999999" then LOCATION=8;
             else if substr(a(i),8,3)='999' then LOCATION=2;
             else LOCATION=0;

          X6_86L=a(i);
          X6_86N=b(i);
          X6_86W=c(i);

          if a(i) ne "         ." then output;  * Keep only those cases with data *;
     end;
run;

data work&f.01c;
     set in&f.2.hh00 (keep=HHID00 X6_86L: X6_86N: X6_86W:);
     keep HHID00 X6_86L X6_86N X6_86W LOCATION;

     length X6_86L $ 10;

     array a(1:10) X6_86L1-X6_86L10;
     array b(1:10) X6_86N1-X6_86N10;
     array c(1:10) X6_86W1-X6_86W10;

     do i=1 to 10;
          X6_86L=a(i);
          X6_86N=b(i);
          X6_86W=c(i);
          if a(i)='9999999' then LOCATION=8;
             else if substr(a(i),1,1)='5' then LOCATION=6;
             else if substr(a(i),1,1)='4' then LOCATION=5;
             else if substr(a(i),1,2)='39' then LOCATION=8;
             else LOCATION=4;
          if a(i) ne "      ." then output;  * Keep only those cases with data *;
     end;

run;

*********************************************************
** Take Care of Missing Data Issues - Recodes at least **
*********************************************************;

data work&f.02a;
     set work&f.01a;

     if X6_86W=9 then X6_86W=.;
     if X6_86N=99 then X6_86N=1; * Assume at least 1 person worked *;

run;

****************************************************************************
** This code collapses multiple code 2 & 3 workers from a household to    **
** a single observation and sums the values for each into summary counts. **
** For the "type of labor" variable, I use an "any paid --> paid" rule    **
** because paying a code 2 or 3 laborer is a rare behavior and distinct   **
****************************************************************************;

 data work&f.02a2;
     set work&f.02a;

     by HHID00;

     retain SUM_N SUM_TYPE SUM_LOC i;


     if first.HHID00 then do;
                            SUM_N=0;
                            SUM_TYPE=2; * Default is unpaid labor*;
                            SUM_LOC=9;
                            i=1;
                         end;

     SUM_N=SUM_N+X6_86N;
     if X6_86W=1 then SUM_TYPE=1;  * Any paid --> all paid *;
     SUM_LOC=9;
     i=i+1;

     if last.HHID00 then output;

run;

data work&f.02a3 (drop=SUM_N SUM_TYPE SUM_LOC i);
     set work&f.02a2;

     X6_86L="   ";
     X6_86N=SUM_N;
     X6_86W=SUM_TYPE;
     LOCATION=SUM_LOC;

run;

*********************************************************
** Take Care of Missing Data Issues - Recodes at least **
*********************************************************;

data work&f.02b;
     set work&f.01b;

     if X6_86W=9 then X6_86W=.;
     if X6_86N=99 then X6_86N=1; * Assume at least 1 person worked *;
run;

data work&f.02c;
     set work&f.01c;

     if X6_86W=9 then X6_86W=.;
     if X6_86N=99 then X6_86N=1; * Assume at least 1 person worked *;
run;

**************************
** Merge files together **
**************************;

data work&f.03;
     set work&f.02a3
         work&f.02b
         work&f.02c;
run;

***************************************************************************
** Add V84 identifiers to 2000 data file as per Rick's suggestion on web **
***************************************************************************;



proc sort data=work&f.03 out=work&f.04;
     by HHID00 X6_86L LOCATION;
run;

data vill_id_&f.01;
     set in&f.1.indiv00;
     keep HHID00 V84;
run;

proc sort data=vill_id_&f.01 out=vill_id_&f.02 nodupkey;
     by HHID00 v84;
run;

data vill_id_&f.03;
     merge work&f.04 (in=a)
           vill_id_&f.02 (in=b);
           if a=1 and b=1 then output;
     by HHID00;
run;

proc sort data=vill_id_&f.03 out=work&f.05;
     by V84;
run;

******************************************************************************
** This step removes all cases about which there is no information about    **
** how their laborers were compensated. This is my fix for the time being.  **
** Note: in doing so, I lose 7 cases (a case here is a helper group)        **
******************************************************************************;

data work&f.06;
     set work&f.05;

     rename X6_86L=HELPHHID;

     if X6_86W ^in (.,9) then output;

run;

************************************************************************************
** The steps below convert the ban lek ti information on the helping household    **
** into the standard HHID##, as a preparatory step to creating network datafiles. **
************************************************************************************;

data work&f.07;
     set work&f.06;

     if LOCATION=9 then do;
                           HELPHHID=HHID00;
                        end;
        else if LOCATION in(0,4) then do;
                                  HELPHHID=substr(HELPHHID,2,9);
                                end;
        else if LOCATION in (2) then do;
                                    HELPHHID=substr(HELPHHID,2,6);
                                end;
        else HELPHHID=".";
run;

*********************************
** Clean HELPHH00 of BAD CODES **
*********************************; 

data work&f.08a (drop=HHID&y.C);
     set work&f.07 (rename=(HHID&y=HHID&y.C));
     HELPHH00=input(HELPHHID,9.0);
     V84N=input(V84,2.0);
     HHID00=input(HHID&y.C,9.);
run;

data work&f.08b (drop=HHID00 V84);
     set in&f.1.indiv00 (keep=HHID00 V84);
     HELPHH00=input(HHID00,9.0);
     V84N=input(V84,2.0);
run;

proc sort data=work&f.08a out=work&f.09a;
     by V84N HELPHH00;
run;

proc sort data=work&f.08b out=work&f.09b nodupkey;
     by V84N HELPHH00;
run;

data work&f.10 aonly bonly;
     merge work&f.09a (in=a)
           work&f.09b (in=b);
     by V84N HELPHH00;
     if a=1 and b=1 then output work&f.10;
     if a=1 and b=0 then output aonly;
     if a=0 and b=1 then output bonly;
run;


** This step removes the HELPHH00 codes from a number of HHs **;
** that had non-existent or incorrect HHs listed as helping  **;

data aonly_fix;
     set aonly;
     if HELPHH00>999999 then do;     
                               HELPHH00=".";
                               HELPHHID=".";
                             end;
run;
 
data work&f.11;
     set work&f.10 (in=a)
         aonly_fix (in=b);
run;

************************************************************************
**input average village wages during high demand from community survey**
************************************************************************;

data work&f.vill_wages_01a (drop=X45MHIGH X45MTYP);  
    set in&f.3.comm&y (keep=VILL00 X45MHIGH X45MTYP);

    if X45MHIGH=9999998 then X45MHIGH=.; ** USING ONLY MALES B/C MALE AND FEMALE WAGES IDENTICAL **;
    if X45MTYP=9999998 then X45MTYP=.;

    if X45MHIGH=. then RICEWAGH=125.77;
       else RICEWAGH=X45MHIGH;
    if X45MTYP=. then RICEWAGN=105.29;
       else RICEWAGN=X45MTYP;
run;

proc sort data=work&f.vill_wages_01a out=work&f.vill_wages_01b nodupkey;
     by VILL&y;
run;

proc sort data=in&f.1.indiv&y  out=work&f.vill_wages_02a nodupkey;
     by HHID&y;
run;

data work&f.make_vill;
     set work&f.vill_wages_02a (keep=HHID&y);
     length VILL&y $ 6;

     VILL&y=substr(HHID&y,1,6);
run;

data work&f.numeric (drop=HHID&y.C);
     set work&f.make_vill (rename=(HHID&y=HHID&y.C));

     HHID&y=input(HHID&y.C,best9.);
run;

proc sort data=work&f.numeric out=work&f.vill_wages_02b;
     by VILL&y HHID&y;
run;

data work&f.vill_wages_03;
     merge work&f.vill_wages_01b (in=a)
           work&f.vill_wages_02b (in=b);
     by VILL&y;
     if b=1 then output;
run;

proc sort data=work&f.vill_wages_03 out=work&f.vill_wages_04;
     by HHID&y;
run;

proc sort data=work&f.11 out=work&f.vill_wages_05;
     by HHID&y;
run;

data work&f.vill_wages_06;
     merge work&f.vill_wages_04 (in=a)
           work&f.vill_wages_05 (in=b);
     by HHID&y;
     if b=1 then output;
run;

proc sort data=work&f.vill_wages_06 out=work&f.11B;
     by HHID&y;
run;

data work&f.11C;
     set work&f.11B;

     if X6_86W=3 then PAIDHH00=2;
        else PAIDHH00=X6_86W;

     if X6_86W=1 then ALLWAGE=round(X6_86N*RICEWAGH,.01);   ** ACTUAL DAYS WORKED*AVG WAGES IN VILL**;
        else ALLWAGE=0;                                             

     NUMWRKS=X6_86N;
  
run;

data work&f.12 (keep=HHID00 PAIDHH00 HELPHH00 LOCATION ALLWAGE NUMWRKS V84N);
     set work&f.11C;
run;

proc sort data=work&f.12 out=work&f.13;              ** MAJOR CHECKPOINT - TOTAL LABOR CHILD FILE **;
     by HHID&y HELPHH&y;
run;

*******************************************************************************************************************************;

***********************************************
** Separate Paid and Unpaid Labor into Files **
***********************************************;

** Note, all code below must be repeated for paid and then unpaid labor **;

**************************************************************************
** PAID: Create Village Aggregate Measures for Help from Beyond Village **
**************************************************************************;

%let p=paid;    *** saves keystrokes ***;

data work&f.14_&p (drop=PAIDHH&y);
     set work&f.13;
     if PAIDHH&y=1;
run;

** SPLIT DATA INTO 3 GROUPS - SAME VILL, OTH VILL, ALL OTHER **;

data work&f.15_&p._othvill (drop=LOCATION);   ** 1 **;
     set work&f.14_&p;
     if LOCATION in (4);
     if HELPHH&y ne .;
run;

data work&f.15_&p._vill (drop=LOCATION);      ** 2 **;
     set work&f.14_&p;
     if LOCATION in (0,1);
     if HELPHH&y ne .;
run;

data work&f.15_&p._other (drop=LOCATION);     ** 3 **;
     set work&f.14_&p;
     if (LOCATION in (2,3,5,6,7,8,9) OR HELPHH&y=.);
run;

****************************************************
** (1) OTHVILL: PROCESS LABOR FROM OTHER VILLAGES **
****************************************************;

** Collapse Labor from the same 84 village **;

data work&f.16_&p._othvill (keep=HHID&y V84N TOT_WRKS TOT_WAGE);
     set work&f.15_&p._othvill;

     retain TOT_WRKS TOT_WAGE;

     by HHID&y V84N;

     if first.V84N then do;
                          TOT_WRKS=0;
                          TOT_WAGE=0;
                        end;
     if NUMWRKS ne . then TOT_WRKS=TOT_WRKS+NUMWRKS;
     if ALLWAGE ne . then TOT_WAGE=TOT_WAGE+ALLWAGE;

    if last.V84N then output;
 
run;

** Merge with all rice-growing households **;

data work&f.add_all_01 (drop=HHID&y.C); 
     set vill_id_&f.02 (rename=(HHID&y=HHID&y.C));
     HHID&y=input(HHID&y.C,best12.);
run;

proc sort data=work&f.add_all_01 out=work&f.add_all_02;
     by HHID&y;
run;

data work&f.17_&p._othvill;
     merge work&f.16_&p._othvill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;

data work&f.18_&p._othvill (drop=V84N TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._othvill;

     retain PID_H01-PID_H10 PWRK_H01-PWRK_H10 PWAG_H01-PWAG_H10 i;

     length PID_H01-PID_H10 8;             * 10 for good measure *;
     length PWRK_H01-PWRK_H10 8;             * 10 for good measure *;
     length PWAG_H01-PWAG_H10 8;             * 10 for good measure *;

     array id(1:10) PID_H01-PID_H10;
     array wk(1:10) PWRK_H01-PWRK_H10;
     array wg(1:10) PWAG_H01-PWAG_H10;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 10;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=V84N;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
 
run;

***********************************************
** (2) VILL: PROCESS LABOR FROM SAME VILLAGE **
***********************************************;

** Placeholder for consistency **;

data work&f.16_&p._vill (keep=HHID&y HELPHH&y TOT_WRKS TOT_WAGE);
     set work&f.15_&p._vill;

     TOT_WRKS=NUMWRKS;
     TOT_WAGE=ALLWAGE;
RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._vill;
     merge work&f.16_&p._vill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;

data work&f.18_&p._vill (drop=HELPHH&y TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._vill;

     retain PID_H11-PID_H60 PWRK_H11-PWRK_H60 PWAG_H11-PWAG_H60 i;    ** Starts at 11 to leave room for village **; 

     length PID_H11-PID_H60 8;                 * 50 for good measure *;
     length PWRK_H11-PWRK_H60 8;             * 50 for good measure *;
     length PWAG_H11-PWAG_H60 8;             * 50 for good measure *;

     array id(1:50) PID_H11-PID_H60;
     array wk(1:50) PWRK_H11-PWRK_H60;
     array wg(1:50) PWAG_H11-PWAG_H60;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 50;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=HELPHH&y;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
run;

****************************************
** (3) OTHER: PROCESS ALL OTHER LABOR **
****************************************;

** This labor is aggregated and included in an attribute file **;

** Placeholder for consistency **;

data work&f.16_&p._other (keep=HHID&y HELPHH&y NUMWRKS ALLWAGE);
     set work&f.15_&p._other;

RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._other;
     merge work&f.16_&p._other (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;
** THIS FILE TO BE MERGED BELOW WITH OTHER ATTRIBUTES **;

data work&f.18_&p._other (drop=HELPHH&y NUMWRKS ALLWAGE);
     set work&f.17_&p._other;

     retain OTH_PWRK OTH_PWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            OTH_PWRK=0;
                            OTH_PWAG=0;
                          end;

     if NUMWRKS ne . then OTH_PWRK=OTH_PWRK+NUMWRKS;
     if ALLWAGE ne . then OTH_PWAG=OTH_PWAG+ALLWAGE;

    if last.HHID&y then output;
run;                                                             


***********************************
** Add other attribute variables ** 
***********************************;

** HH-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_hh (drop=HHID&y.C);
     set in&f.2.hh&y (keep=HHID&y RICE rename=(HHID&y=HHID&y.C));

     if RICE in (.,2,8) then RICE=0;

     IS_HH=1;

     HHID&y=input(HHID&y.C,best12.);
    
run;

** Merge with all rice-growing households **;

data work&f._&p._add_all_hh (drop=V84N LOCATION HELPHH&y);
     merge work&f.14_&p (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
                          
     if b=1 then output;
run;

data work&f._&p._add_tot_hh (drop=NUMWRKS ALLWAGE); **Generate Count Vars for ALL labor **;
     set work&f._&p._add_all_hh;

     retain ALL_PWRK ALL_PWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            ALL_PWRK=0;
                            ALL_PWAG=0;
                          end;

     if NUMWRKS ne . then ALL_PWRK=ALL_PWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_PWAG=ALL_PWAG+ALLWAGE;

    if last.HHID&y then output;
run;                           

** MERGE **;

data work&f.19_&p._all_hh;
     merge work&f._&p._add_rice_hh (in=a)
           work&f._&p._add_tot_hh (in=b)
           work&f.18_&p._other (in=c);
     by HHID&y;
     if c=1 then output;
run;

data work&f.19_&p._all_hh_2 (drop=V84);
     set work&f.19_&p._all_hh;
     V84N=input(V84,2.0);
run;

** VILLAGE-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_vill (drop=V84 HHID&y);
     set work&f.add_all_02;
     RICE=1;
     IS_HH=0;
     OTH_PWRK=0;
     OTH_PWAG=0;
     V84N=input(V84,2.0);
run;

** Prep for merge **;

proc sort data=work&f._&p._add_rice_vill out=work&f._&p._add_rice_vill_2 nodupkey;
     by V84N;
run;

proc sort data=work&f.14_&p out=work&f.14_&p._by_v84;
     by V84N;
run;

**Generate Count Vars for ALL labor **;

data work&f._&p._add_tot_vill (drop=HHID&y HELPHH&y NUMWRKS ALLWAGE); 
     set work&f.14_&p._by_v84;

     retain ALL_PWRK ALL_PWAG;
 
     by V84N;

     if first.V84N then do;
                            ALL_PWRK=0;
                            ALL_PWAG=0;
                          end;

     if NUMWRKS ne . then ALL_PWRK=ALL_PWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_PWAG=ALL_PWAG+ALLWAGE;

     if last.V84N then output;
run;                           

data work&f.19_&p._all_vill;
     merge work&f._&p._add_rice_vill_2 (in=a)
           work&f._&p._add_tot_vill (in=b);
     by V84N;

     HHID&y=V84N;
     if b=1 then output;
run;

***************************************
** Merge HH and VILL Attribute Files **
***************************************;

data work&f.19_&p._attrib;
     set work&f.19_&p._all_vill (drop=LOCATION)
         work&f.19_&p._all_hh_2;
run;

**********************************************
** Merge Same and Other Village Labor Files **
**********************************************;

data work&f.19_&p._adjlist;
     merge work&f.18_&p._othvill (in=b)
           work&f.18_&p._vill (in=a);
     by HHID&y;

     if a=1 and b=1 then output;
run;

***********************************************************
** Add Villages to adjacency matrix - all missing values **
***********************************************************;

proc sort data=vill_id_&f.02 out=add_vill_&p._&f._01 nodupkey;
     by V84;
run;

data add_vill_&p._&f._02 (drop=V84 j);
     set add_vill_&p._&f._01 (drop=HHID&y);

     length PID_H01-PID_H60 8;                 
     length PWRK_H01-PWRK_H60 8;             
     length PWAG_H01-PWAG_H60 8;             

     array id(1:60) PID_H01-PID_H60;
     array wk(1:60) PWRK_H01-PWRK_H60;
     array wg(1:60) PWAG_H01-PWAG_H60;
 
     do j= 1 to 60;
        id(j)=.;
        wk(j)=.;
        wg(j)=.;
     end;
 
     HHID&y=input(V84,best12.);    
     V84N=input(V84,2.);    

run;   

data add_vill_&p._&f._03 (drop=V84);
     set work&f.19_&p._adjlist;
     V84N=input(V84,2.0);
run;
 
data add_vill_&p._&f._04;
     set add_vill_&p._&f._02 
         add_vill_&p._&f._03;
run;


****************************************************************************
** UNPAID: Create Village Aggregate Measures for Help from Beyond Village **
****************************************************************************;

%let p=unpaid;    *** saves keystrokes ***;

data work&f.14_unpaid (drop=PAIDHH&y);
     set work&f.13;
     if PAIDHH&y=2;
run;

** SPLIT DATA INTO 3 GROUPS - SAME VILL, OTH VILL, ALL OTHER **;

data work&f.15_&p._othvill (drop=LOCATION);   ** 1 **;
     set work&f.14_&p;
     if LOCATION in (4);
     if HELPHH&y ne .;
run;

data work&f.15_&p._vill (drop=LOCATION);      ** 2 **;
     set work&f.14_&p;
     if LOCATION in (0,1);
     if HELPHH&y ne .;
run;

data work&f.15_&p._other (drop=LOCATION);     ** 3 **;
     set work&f.14_&p;
     if (LOCATION in (2,3,5,6,7,8,9) OR HELPHH&y=.);
run;

****************************************************
** (1) OTHVILL: PROCESS LABOR FROM OTHER VILLAGES **
****************************************************;

** Collapse Labor from the same 84 village **;

data work&f.16_&p._othvill (keep=HHID&y V84N TOT_WRKS TOT_WAGE);
     set work&f.15_&p._othvill;

     retain TOT_WRKS TOT_WAGE;

     by HHID&y V84N;

     if first.V84N then do;
                          TOT_WRKS=0;
                          TOT_WAGE=0;
                        end;
     if NUMWRKS ne . then TOT_WRKS=TOT_WRKS+NUMWRKS;
     if ALLWAGE ne . then TOT_WAGE=TOT_WAGE+ALLWAGE;

    if last.V84N then output;
 
run;

** Merge with all rice-growing households **;

data work&f.add_all_01 (drop=HHID&y.C); 
     set vill_id_&f.02 (rename=(HHID&y=HHID&y.C));
     HHID&y=input(HHID&y.C,best12.);
run;

proc sort data=work&f.add_all_01 out=work&f.add_all_02;
     by HHID&y;
run;

data work&f.17_&p._othvill;
     merge work&f.16_&p._othvill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;


data work&f.18_&p._othvill (drop=V84N TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._othvill;

     retain UID_H01-UID_H10 UWRK_H01-UWRK_H10 UWAG_H01-UWAG_H10 i;

     length UID_H01-UID_H10 8;             * 10 for good measure *;
     length UWRK_H01-UWRK_H10 8;             * 10 for good measure *;
     length UWAG_H01-UWAG_H10 8;             * 10 for good measure *;

     array id(1:10) UID_H01-UID_H10;
     array wk(1:10) UWRK_H01-UWRK_H10;
     array wg(1:10) UWAG_H01-UWAG_H10;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 10;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=V84N;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
 
run;

***********************************************
** (2) VILL: PROCESS LABOR FROM SAME VILLAGE **
***********************************************;

** Placeholder for consistency **;

data work&f.16_&p._vill (keep=HHID&y HELPHH&y TOT_WRKS TOT_WAGE);
     set work&f.15_&p._vill;

     TOT_WRKS=NUMWRKS;
     TOT_WAGE=ALLWAGE;
RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._vill;
     merge work&f.16_&p._vill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;

data work&f.18_&p._vill (drop=HELPHH&y TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._vill;

     retain UID_H11-UID_H60 UWRK_H11-UWRK_H60 UWAG_H11-UWAG_H60 i;    ** Starts at 11 to leave room for village **; 

     length UID_H11-UID_H60 8;                 * 50 for good measure *;
     length UWRK_H11-UWRK_H60 8;             * 50 for good measure *;
     length UWAG_H11-UWAG_H60 8;             * 50 for good measure *;

     array id(1:50) UID_H11-UID_H60;
     array wk(1:50) UWRK_H11-UWRK_H60;
     array wg(1:50) UWAG_H11-UWAG_H60;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 50;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=HELPHH&y;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
run;

****************************************
** (3) OTHER: PROCESS ALL OTHER LABOR **
****************************************;

** This labor is aggregated and included in an attribute file **;

** Placeholder for consistency **;

data work&f.16_&p._other (keep=HHID&y HELPHH&y NUMWRKS ALLWAGE);
     set work&f.15_&p._other;

RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._other;
     merge work&f.16_&p._other (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;
** THIS FILE TO BE MERGED BELOW WITH OTHER ATTRIBUTES **;

data work&f.18_&p._other (drop=HELPHH&y NUMWRKS ALLWAGE);
     set work&f.17_&p._other;

     retain OTH_UWRK OTH_UWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            OTH_UWRK=0;
                            OTH_UWAG=0;
                          end;

     if NUMWRKS ne . then OTH_UWRK=OTH_UWRK+NUMWRKS;
     if ALLWAGE ne . then OTH_UWAG=OTH_UWAG+ALLWAGE;

    if last.HHID&y then output;
run;                                                             


***********************************
** Add other attribute variables ** 
***********************************;

** HH-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_hh (drop=HHID&y.C);
     set in&f.2.hh&y (keep=HHID&y RICE rename=(HHID&y=HHID&y.C));

     if RICE in (.,2,8) then RICE=0;

     IS_HH=1;

     HHID&y=input(HHID&y.C,best12.);

run;

** Merge with all rice-growing households **;

data work&f._&p._add_all_hh (drop=V84N LOCATION HELPHH&y);
     merge work&f.14_&p (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
                          
     if b=1 then output;
run;

data work&f._&p._add_tot_hh (drop=NUMWRKS ALLWAGE); **Generate Count Vars for ALL labor **;
     set work&f._&p._add_all_hh;

     retain ALL_UWRK ALL_UWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            ALL_UWRK=0;
                            ALL_UWAG=0;
                          end;

     if NUMWRKS ne . then ALL_UWRK=ALL_UWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_UWAG=ALL_UWAG+ALLWAGE;

    if last.HHID&y then output;
run;                           

** MERGE **;

data work&f.19_&p._all_hh;
     merge work&f._&p._add_rice_hh (in=a)
           work&f._&p._add_tot_hh (in=b)
           work&f.18_&p._other (in=c);
     by HHID&y;
     if c=1 then output;
run;

data work&f.19_&p._all_hh_2 (drop=V84);
     set work&f.19_&p._all_hh;
     V84N=input(V84,2.0);
run;

** VILLAGE-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_vill (drop=V84 HHID&y);
     set work&f.add_all_02;
     RICE=1;
     IS_HH=0;
     OTH_UWRK=0;
     OTH_UWAG=0;
     V84N=input(V84,2.0);
run;

** Prep for merge **;

proc sort data=work&f._&p._add_rice_vill out=work&f._&p._add_rice_vill_2 nodupkey;
     by V84N;
run;

proc sort data=work&f.14_&p out=work&f.14_&p._by_v84;
     by V84N;
run;

**Generate Count Vars for ALL labor **;

data work&f._&p._add_tot_vill (drop=HHID&y HELPHH&y NUMWRKS ALLWAGE); 
     set work&f.14_&p._by_v84;

     retain ALL_UWRK ALL_UWAG;
 
     by V84N;

     if first.V84N then do;
                            ALL_UWRK=0;
                            ALL_UWAG=0;
                          end;

     if NUMWRKS ne . then ALL_UWRK=ALL_UWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_UWAG=ALL_UWAG+ALLWAGE;

     if last.V84N then output;
run;                           

data work&f.19_&p._all_vill;
     merge work&f._&p._add_rice_vill_2 (in=a)
           work&f._&p._add_tot_vill (in=b);
     by V84N;

     HHID&y=V84N;
     if b=1 then output;
run;

***************************************
** Merge HH and VILL Attribute Files **
***************************************;

data work&f.19_&p._attrib;
     set work&f.19_&p._all_vill (drop=LOCATION)
         work&f.19_&p._all_hh_2;
run;

**********************************************
** Merge Same and Other Village Labor Files **
**********************************************;

data work&f.19_&p._adjlist;
     merge work&f.18_&p._othvill (in=b)
           work&f.18_&p._vill (in=a);
     by HHID&y;

     if a=1 and b=1 then output;
run;

***********************************************************
** Add Villages to adjacency matrix - all missing values **
***********************************************************;

proc sort data=vill_id_&f.02 out=add_vill_&p._&f._01 nodupkey;
     by V84;
run;

data add_vill_&p._&f._02 (drop=V84 j);
     set add_vill_&p._&f._01 (drop=HHID&y);

     length UID_H01-UID_H60 8;                 
     length UWRK_H01-UWRK_H60 8;             
     length UWAG_H01-UWAG_H60 8;             

     array id(1:60) UID_H01-UID_H60;
     array wk(1:60) UWRK_H01-UWRK_H60;
     array wg(1:60) UWAG_H01-UWAG_H60;
 
     do j= 1 to 60;
        id(j)=.;
        wk(j)=.;
        wg(j)=.;
     end;
 
     HHID&y=input(V84,best12.);    
     V84N=input(V84,2.);    

run;   

data add_vill_&p._&f._03 (drop=V84);
     set work&f.19_&p._adjlist;
     V84N=input(V84,2.0);
run;
 
data add_vill_&p._&f._04;
     set add_vill_&p._&f._02 
         add_vill_&p._&f._03;
run;


*******************************************************************************************************************;

*********************************************************************
** Temporarily Merge Together All 4 Final Files to purge ID errors **
*********************************************************************;

proc sort data=add_vill_paid_&f._04 out=work&f.20_paid_adj;
     by HHID&y V84N;
run;

proc sort data=add_vill_unpaid_&f._04 out=work&f.20_unpaid_adj;
     by HHID&y V84N;
run;

proc sort data=work&f.19_paid_attrib out=work&f.20_paid_att;
     by HHID&y V84N;
run;

proc sort data=work&f.19_unpaid_attrib out=work&f.20_unpaid_att;
     by HHID&y V84N;
run;


data work&f.21_all_files;
     merge work&f.20_paid_adj (in=a)
           work&f.20_unpaid_adj (in=b)
           work&f.20_paid_att (in=c)
           work&f.20_unpaid_att (drop=RICE IS_HH in=d);
     by HHID&y V84N;
     if a=1 and b=1 and c=1 and d=1 then output;
run;  

*******************************************
** Format files to be exported to UCINET **
*******************************************;

** I have decided to revert these matrices back to hh-only - 2011 02 07 **;
** These changes are reflected only in the four data steps below **;
** The village-level information still exists in all prior data steps **;
** In the future, I'd like to add a village-level only analysis **;

** I have also re-combined all files into a single one as of 2011 02 09 **;

data r&y._all (drop=PID_H01-PID_H10 PWRK_H01-PWRK_H10 PWAG_H01-PWAG_H10
                    UID_H01-UID_H10 UWRK_H01-UWRK_H10 UWAG_H01-UWAG_H10);
     set work&f.21_all_files;

     ALL_WRKS=ALL_PWRK+ALL_UWRK;
     ALL_WAGE=ALL_PWAG+ALL_UWAG;

     if HHID&y>100;               * Remove village datalines *;
run;

**************************************************************************************************;

************************************************************************************
* 6 ** VILLAGE NETWORKS: RICE PAID ** Create VALUED adjacency matrices: # WORKERS **
************************************************************************************;

** Use transpose in final step to reverse direction of ties, indicating labor movement **;

************************************************
** Create separate village files EACH VILLAGE **
************************************************;

%macro v_split (numvill=);  %* macro splits villages *;

       %* NUMVILL=Number of Unique Villages in file *;

%do i=1 %to &numvill;

    data r00_p&i (drop=V84N);
         set r&y._all;
         if V84N=&i;
    run;

%end;

%mend v_split;

%v_split (numvill=51);

%macro v_adj1 (numvill=);
%do i=1 %to &numvill;
proc iml;
     %include '/home/jhull/public/span/adjval.mod';
     %include '/home/jhull/public/span/pajwrite.mod';
     %let p1=%quote(/home/jhull/nangrong/data_paj/2000/a1-net/v0);
     %let p2=%quote(00a1.net);
     use r00_p&i;
     read all var{
                  PID_H11 PID_H12 PID_H13 PID_H14 PID_H15
                  PID_H16 PID_H17 PID_H18 PID_H19 PID_H20
                  PID_H21 PID_H22 PID_H23 PID_H24 PID_H25
                  PID_H26 PID_H27 PID_H28 PID_H29 PID_H30
                  PID_H31 PID_H32 PID_H33 PID_H34 PID_H35
                  PID_H36 PID_H37 PID_H38 PID_H39 PID_H40
                  PID_H41 PID_H42 PID_H43 PID_H44 PID_H45
                  PID_H46 PID_H47 PID_H48 PID_H49 PID_H50
                  PID_H51 PID_H52 PID_H53 PID_H54 PID_H55
                  PID_H56 PID_H57 PID_H58 PID_H59 PID_H60} into rcv;
     read all var{
                  PWRK_H11 PWRK_H12 PWRK_H13 PWRK_H14 PWRK_H15
                  PWRK_H16 PWRK_H17 PWRK_H18 PWRK_H19 PWRK_H20
                  PWRK_H21 PWRK_H22 PWRK_H23 PWRK_H24 PWRK_H25
                  PWRK_H26 PWRK_H27 PWRK_H28 PWRK_H29 PWRK_H30
                  PWRK_H31 PWRK_H32 PWRK_H33 PWRK_H34 PWRK_H35
                  PWRK_H36 PWRK_H37 PWRK_H38 PWRK_H39 PWRK_H40
                  PWRK_H41 PWRK_H42 PWRK_H43 PWRK_H44 PWRK_H45
                  PWRK_H46 PWRK_H47 PWRK_H48 PWRK_H49 PWRK_H50
                  PWRK_H51 PWRK_H52 PWRK_H53 PWRK_H54 PWRK_H55
                  PWRK_H56 PWRK_H57 PWRK_H58 PWRK_H59 PWRK_H60} into val;
     read all var{HHID00} into snd;
     r00_p=adjval(snd,rcv,val);
     id=r00_p[,1];
     r00_p=r00_p[,2:ncol(r00_p)];
     adj=r00_p;
     file "&p1.&i.&p2";
     call pajwrite(adj`,id,2);
     adjinv=adj`;
     create adj0&i from adjinv;
            append from adjinv;
     idinv=id`;
     create id0&i from idinv;
            append from idinv;
quit;
%end;
%mend v_adj1;

%v_adj1(numvill=9);

%macro v_adj2 (numvill=);
%do i=10 %to &numvill;
proc iml;
     %include '/home/jhull/public/span/adjval.mod';
     %include '/home/jhull/public/span/pajwrite.mod';
     %let p1=%quote(/home/jhull/nangrong/data_paj/2000/a1-net/v);
     %let p2=%quote(00a1.net);
     use r00_p&i;
     read all var{
                  PID_H11 PID_H12 PID_H13 PID_H14 PID_H15
                  PID_H16 PID_H17 PID_H18 PID_H19 PID_H20
                  PID_H21 PID_H22 PID_H23 PID_H24 PID_H25
                  PID_H26 PID_H27 PID_H28 PID_H29 PID_H30
                  PID_H31 PID_H32 PID_H33 PID_H34 PID_H35
                  PID_H36 PID_H37 PID_H38 PID_H39 PID_H40
                  PID_H41 PID_H42 PID_H43 PID_H44 PID_H45
                  PID_H46 PID_H47 PID_H48 PID_H49 PID_H50
                  PID_H51 PID_H52 PID_H53 PID_H54 PID_H55
                  PID_H56 PID_H57 PID_H58 PID_H59 PID_H60} into rcv;
     read all var{
                  PWRK_H11 PWRK_H12 PWRK_H13 PWRK_H14 PWRK_H15
                  PWRK_H16 PWRK_H17 PWRK_H18 PWRK_H19 PWRK_H20
                  PWRK_H21 PWRK_H22 PWRK_H23 PWRK_H24 PWRK_H25
                  PWRK_H26 PWRK_H27 PWRK_H28 PWRK_H29 PWRK_H30
                  PWRK_H31 PWRK_H32 PWRK_H33 PWRK_H34 PWRK_H35
                  PWRK_H36 PWRK_H37 PWRK_H38 PWRK_H39 PWRK_H40
                  PWRK_H41 PWRK_H42 PWRK_H43 PWRK_H44 PWRK_H45
                  PWRK_H46 PWRK_H47 PWRK_H48 PWRK_H49 PWRK_H50
                  PWRK_H51 PWRK_H52 PWRK_H53 PWRK_H54 PWRK_H55
                  PWRK_H56 PWRK_H57 PWRK_H58 PWRK_H59 PWRK_H60} into val;
     read all var{HHID00} into snd;
     r00_p=adjval(snd,rcv,val);
     id=r00_p[,1];
     r00_p=r00_p[,2:ncol(r00_p)];
     adj=r00_p;
     file "&p1.&i.&p2";
     call pajwrite(adj`,id,2);
     adjinv=adj`;
     create adj&i from adjinv;
            append from adjinv;
     idinv=id`;
     create id&i from idinv;
            append from idinv;
quit;
%end;
%mend v_adj2;

%v_adj2(numvill=51);

%macro v_adj3(numvill=);
%do i=1 %to &numvill;
    data _null_ ;
         %let p1=%quote(/home/jhull/nangrong/data_paj/2000/a1-adj/v0);
         %let p2=%quote(00a1.adj);
         set adj0&i;
         file "&p1.&i.&p2"  lrecl=1000;
         put (_ALL_) (+0);
run;
    data _null_ ;
         %let p3=%quote(/home/jhull/nangrong/data_paj/2000/a1-id/v0);
         %let p4=%quote(00a1.id);
         set id0&i;
         file "&p3.&i.&p4"  lrecl=10000;
         put (_ALL_) (+0);
run;

%end;
%mend v_adj3;

%v_adj3(numvill=9);

%macro v_adj4(numvill=);
%do i=10 %to &numvill;  
    data _null_ ;
         %let p1=%quote(/home/jhull/nangrong/data_paj/2000/a1-adj/v);
         %let p2=%quote(00a1.adj);
         set adj&i;
         file "&p1.&i.&p2"  lrecl=1000;
         put (_ALL_) (+0);
run;
    data _null_ ;
         %let p3=%quote(/home/jhull/nangrong/data_paj/2000/a1-id/v);
         %let p4=%quote(00a1.id);
         set id&i;
         file "&p3.&i.&p4"  lrecl=10000;
         put (_ALL_) (+0);
run;
%end;
%mend v_adj4;

%v_adj4(numvill=51);

In [ ]:
********************************************************************
**     Program Name: /home/jhull/nangrong/prog_sas/p02_nets/2000/p02_00a2.sas
**     Programmer: james r. hull
**     Start Date: 2011 03 09
**     Purpose:
**        1.) Create A2 = UNPAID LABOR NETWORKS # WORKERS
**
**     Input Data:
**        '/home/jhull/nangrong/data_sas/2000/current/indiv00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/hh00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/comm00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/sibs00.xpt'
**
**     Output Data:
**        1.) /home/jhull/nangrong/data_paj/2000/a2/vXX00a2.net
**
**     Notes: 
**        1.)
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=60;

%let f=00a2;  ** names file (allows portability) **;
%let y=00; 

**********************
**  Data Libraries  **
**********************;

libname in&f.1 xport '/home/jhull/nangrong/data_sas/2000/current/indiv00.xpt';
libname in&f.2 xport '/home/jhull/nangrong/data_sas/2000/current/hh00.xpt';
libname in&f.3 xport '/home/jhull/nangrong/data_sas/2000/current/comm00.xpt';
libname in&f.4 xport '/home/jhull/nangrong/data_sas/2000/current/sibs00.xpt';

******************************
**  Create Working Dataset  **
******************************;

********************************************************************
**  Length 10
**  0 In this 1984 village                2 + Village # + House #
**  2 In this village but blt             2 + Village # + 999
**  Length 7 
**  4 Another village in Nang Rong        3 + Village #
**  8 Unknown village in Nang Rong        3 + 999999
**  5 Another district in Buriram         4 + District # + 0000
**  6 Another province                    5 + District # + 0000
**  7 Another country 
**  8 Missing/Don't know                  9999999999 or 9999999
**  Length 9
**  9 Code 2 or 3 Returning HH member
*******************************************************************;

***************************************************************************
** Stack rice harvest labor data into a child file and label by location **
***************************************************************************;

data work&f.01a;
     set in&f.2.hh00 (keep=HHID00 X6_84C: X6_84W:);
     keep HHID00 X6_86L X6_86N X6_86W LOCATION;

     length X6_86L $ 10;

     array a(1:7) X6_84C1-X6_84C7;
     array b(1:7) X6_84W1-X6_84W7;

     do i=1 to 7;
          X6_86L=a(i);
          X6_86N=1;
          X6_86W=b(i);
          LOCATION=9;
          if a(i) ne " ." then output;  * Keep only those cases with data *;
     end;
run;

data work&f.01b;
     set in&f.2.hh00 (keep=HHID00 X6_85H: X6_85N: X6_85W:);
     keep HHID00 X6_86L X6_86N X6_86W LOCATION;

     length X6_86L $ 10;

     array a(1:13) X6_85H1-X6_85H13;
     array b(1:13) X6_85N1-X6_85N13;
     array c(1:13) X6_85W1-X6_85W13;

     do i=1 to 13;
 
          ** FIX ANOTHER CRAZY ERROR IN ORIGINAL DATA WITH REVERSED CODING FOR IN-VILLAGE LABOR **;
         
          if substr(a(i),5,3) in ('020','021','170','180') then a(i)=cat("2",substr(a(i),5,6),substr(a(i),2,3));

          if a(i)="9999999999" then LOCATION=8;
             else if substr(a(i),8,3)='999' then LOCATION=2;
             else LOCATION=0;

          X6_86L=a(i);
          X6_86N=b(i);
          X6_86W=c(i);

          if a(i) ne "         ." then output;  * Keep only those cases with data *;
     end;
run;

data work&f.01c;
     set in&f.2.hh00 (keep=HHID00 X6_86L: X6_86N: X6_86W:);
     keep HHID00 X6_86L X6_86N X6_86W LOCATION;

     length X6_86L $ 10;

     array a(1:10) X6_86L1-X6_86L10;
     array b(1:10) X6_86N1-X6_86N10;
     array c(1:10) X6_86W1-X6_86W10;

     do i=1 to 10;
          X6_86L=a(i);
          X6_86N=b(i);
          X6_86W=c(i);
          if a(i)='9999999' then LOCATION=8;
             else if substr(a(i),1,1)='5' then LOCATION=6;
             else if substr(a(i),1,1)='4' then LOCATION=5;
             else if substr(a(i),1,2)='39' then LOCATION=8;
             else LOCATION=4;
          if a(i) ne "      ." then output;  * Keep only those cases with data *;
     end;

run;

*********************************************************
** Take Care of Missing Data Issues - Recodes at least **
*********************************************************;

data work&f.02a;
     set work&f.01a;

     if X6_86W=9 then X6_86W=.;
     if X6_86N=99 then X6_86N=1; * Assume at least 1 person worked *;

run;

****************************************************************************
** This code collapses multiple code 2 & 3 workers from a household to    **
** a single observation and sums the values for each into summary counts. **
** For the "type of labor" variable, I use an "any paid --> paid" rule    **
** because paying a code 2 or 3 laborer is a rare behavior and distinct   **
****************************************************************************;

 data work&f.02a2;
     set work&f.02a;

     by HHID00;

     retain SUM_N SUM_TYPE SUM_LOC i;


     if first.HHID00 then do;
                            SUM_N=0;
                            SUM_TYPE=2; * Default is unpaid labor*;
                            SUM_LOC=9;
                            i=1;
                         end;

     SUM_N=SUM_N+X6_86N;
     if X6_86W=1 then SUM_TYPE=1;  * Any paid --> all paid *;
     SUM_LOC=9;
     i=i+1;

     if last.HHID00 then output;

run;

data work&f.02a3 (drop=SUM_N SUM_TYPE SUM_LOC i);
     set work&f.02a2;

     X6_86L="   ";
     X6_86N=SUM_N;
     X6_86W=SUM_TYPE;
     LOCATION=SUM_LOC;

run;

*********************************************************
** Take Care of Missing Data Issues - Recodes at least **
*********************************************************;

data work&f.02b;
     set work&f.01b;

     if X6_86W=9 then X6_86W=.;
     if X6_86N=99 then X6_86N=1; * Assume at least 1 person worked *;
run;

data work&f.02c;
     set work&f.01c;

     if X6_86W=9 then X6_86W=.;
     if X6_86N=99 then X6_86N=1; * Assume at least 1 person worked *;
run;

**************************
** Merge files together **
**************************;

data work&f.03;
     set work&f.02a3
         work&f.02b
         work&f.02c;
run;

***************************************************************************
** Add V84 identifiers to 2000 data file as per Rick's suggestion on web **
***************************************************************************;



proc sort data=work&f.03 out=work&f.04;
     by HHID00 X6_86L LOCATION;
run;

data vill_id_&f.01;
     set in&f.1.indiv00;
     keep HHID00 V84;
run;

proc sort data=vill_id_&f.01 out=vill_id_&f.02 nodupkey;
     by HHID00 v84;
run;

data vill_id_&f.03;
     merge work&f.04 (in=a)
           vill_id_&f.02 (in=b);
           if a=1 and b=1 then output;
     by HHID00;
run;

proc sort data=vill_id_&f.03 out=work&f.05;
     by V84;
run;

******************************************************************************
** This step removes all cases about which there is no information about    **
** how their laborers were compensated. This is my fix for the time being.  **
** Note: in doing so, I lose 7 cases (a case here is a helper group)        **
******************************************************************************;

data work&f.06;
     set work&f.05;

     rename X6_86L=HELPHHID;

     if X6_86W ^in (.,9) then output;

run;

************************************************************************************
** The steps below convert the ban lek ti information on the helping household    **
** into the standard HHID##, as a preparatory step to creating network datafiles. **
************************************************************************************;

data work&f.07;
     set work&f.06;

     if LOCATION=9 then do;
                           HELPHHID=HHID00;
                        end;
        else if LOCATION in(0,4) then do;
                                  HELPHHID=substr(HELPHHID,2,9);
                                end;
        else if LOCATION in (2) then do;
                                    HELPHHID=substr(HELPHHID,2,6);
                                end;
        else HELPHHID=".";
run;

*********************************
** Clean HELPHH00 of BAD CODES **
*********************************; 

data work&f.08a (drop=HHID&y.C);
     set work&f.07 (rename=(HHID&y=HHID&y.C));
     HELPHH00=input(HELPHHID,9.0);
     V84N=input(V84,2.0);
     HHID00=input(HHID&y.C,9.);
run;

data work&f.08b (drop=HHID00 V84);
     set in&f.1.indiv00 (keep=HHID00 V84);
     HELPHH00=input(HHID00,9.0);
     V84N=input(V84,2.0);
run;

proc sort data=work&f.08a out=work&f.09a;
     by V84N HELPHH00;
run;

proc sort data=work&f.08b out=work&f.09b nodupkey;
     by V84N HELPHH00;
run;

data work&f.10 aonly bonly;
     merge work&f.09a (in=a)
           work&f.09b (in=b);
     by V84N HELPHH00;
     if a=1 and b=1 then output work&f.10;
     if a=1 and b=0 then output aonly;
     if a=0 and b=1 then output bonly;
run;


** This step removes the HELPHH00 codes from a number of HHs **;
** that had non-existent or incorrect HHs listed as helping  **;

data aonly_fix;
     set aonly;
     if HELPHH00>999999 then do;     
                               HELPHH00=".";
                               HELPHHID=".";
                             end;
run;
 
data work&f.11;
     set work&f.10 (in=a)
         aonly_fix (in=b);
run;

************************************************************************
**input average village wages during high demand from community survey**
************************************************************************;

data work&f.vill_wages_01a (drop=X45MHIGH X45MTYP);  
    set in&f.3.comm&y (keep=VILL00 X45MHIGH X45MTYP);

    if X45MHIGH=9999998 then X45MHIGH=.; ** USING ONLY MALES B/C MALE AND FEMALE WAGES IDENTICAL **;
    if X45MTYP=9999998 then X45MTYP=.;

    if X45MHIGH=. then RICEWAGH=125.77;
       else RICEWAGH=X45MHIGH;
    if X45MTYP=. then RICEWAGN=105.29;
       else RICEWAGN=X45MTYP;
run;

proc sort data=work&f.vill_wages_01a out=work&f.vill_wages_01b nodupkey;
     by VILL&y;
run;

proc sort data=in&f.1.indiv&y  out=work&f.vill_wages_02a nodupkey;
     by HHID&y;
run;

data work&f.make_vill;
     set work&f.vill_wages_02a (keep=HHID&y);
     length VILL&y $ 6;

     VILL&y=substr(HHID&y,1,6);
run;

data work&f.numeric (drop=HHID&y.C);
     set work&f.make_vill (rename=(HHID&y=HHID&y.C));

     HHID&y=input(HHID&y.C,best9.);
run;

proc sort data=work&f.numeric out=work&f.vill_wages_02b;
     by VILL&y HHID&y;
run;

data work&f.vill_wages_03;
     merge work&f.vill_wages_01b (in=a)
           work&f.vill_wages_02b (in=b);
     by VILL&y;
     if b=1 then output;
run;

proc sort data=work&f.vill_wages_03 out=work&f.vill_wages_04;
     by HHID&y;
run;

proc sort data=work&f.11 out=work&f.vill_wages_05;
     by HHID&y;
run;

data work&f.vill_wages_06;
     merge work&f.vill_wages_04 (in=a)
           work&f.vill_wages_05 (in=b);
     by HHID&y;
     if b=1 then output;
run;

proc sort data=work&f.vill_wages_06 out=work&f.11B;
     by HHID&y;
run;

data work&f.11C;
     set work&f.11B;

     if X6_86W=3 then PAIDHH00=2;
        else PAIDHH00=X6_86W;

     if X6_86W=1 then ALLWAGE=round(X6_86N*RICEWAGH,.01);   ** ACTUAL DAYS WORKED*AVG WAGES IN VILL**;
        else ALLWAGE=0;                                             

     NUMWRKS=X6_86N;
  
run;

data work&f.12 (keep=HHID00 PAIDHH00 HELPHH00 LOCATION ALLWAGE NUMWRKS V84N);
     set work&f.11C;
run;

proc sort data=work&f.12 out=work&f.13;              ** MAJOR CHECKPOINT - TOTAL LABOR CHILD FILE **;
     by HHID&y HELPHH&y;
run;

*******************************************************************************************************************************;

***********************************************
** Separate Paid and Unpaid Labor into Files **
***********************************************;

** Note, all code below must be repeated for paid and then unpaid labor **;

**************************************************************************
** PAID: Create Village Aggregate Measures for Help from Beyond Village **
**************************************************************************;

%let p=paid;    *** saves keystrokes ***;

data work&f.14_&p (drop=PAIDHH&y);
     set work&f.13;
     if PAIDHH&y=1;
run;

** SPLIT DATA INTO 3 GROUPS - SAME VILL, OTH VILL, ALL OTHER **;

data work&f.15_&p._othvill (drop=LOCATION);   ** 1 **;
     set work&f.14_&p;
     if LOCATION in (4);
     if HELPHH&y ne .;
run;

data work&f.15_&p._vill (drop=LOCATION);      ** 2 **;
     set work&f.14_&p;
     if LOCATION in (0,1);
     if HELPHH&y ne .;
run;

data work&f.15_&p._other (drop=LOCATION);     ** 3 **;
     set work&f.14_&p;
     if (LOCATION in (2,3,5,6,7,8,9) OR HELPHH&y=.);
run;

****************************************************
** (1) OTHVILL: PROCESS LABOR FROM OTHER VILLAGES **
****************************************************;

** Collapse Labor from the same 84 village **;

data work&f.16_&p._othvill (keep=HHID&y V84N TOT_WRKS TOT_WAGE);
     set work&f.15_&p._othvill;

     retain TOT_WRKS TOT_WAGE;

     by HHID&y V84N;

     if first.V84N then do;
                          TOT_WRKS=0;
                          TOT_WAGE=0;
                        end;
     if NUMWRKS ne . then TOT_WRKS=TOT_WRKS+NUMWRKS;
     if ALLWAGE ne . then TOT_WAGE=TOT_WAGE+ALLWAGE;

    if last.V84N then output;
 
run;

** Merge with all rice-growing households **;

data work&f.add_all_01 (drop=HHID&y.C); 
     set vill_id_&f.02 (rename=(HHID&y=HHID&y.C));
     HHID&y=input(HHID&y.C,best12.);
run;

proc sort data=work&f.add_all_01 out=work&f.add_all_02;
     by HHID&y;
run;

data work&f.17_&p._othvill;
     merge work&f.16_&p._othvill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;

data work&f.18_&p._othvill (drop=V84N TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._othvill;

     retain PID_H01-PID_H10 PWRK_H01-PWRK_H10 PWAG_H01-PWAG_H10 i;

     length PID_H01-PID_H10 8;             * 10 for good measure *;
     length PWRK_H01-PWRK_H10 8;             * 10 for good measure *;
     length PWAG_H01-PWAG_H10 8;             * 10 for good measure *;

     array id(1:10) PID_H01-PID_H10;
     array wk(1:10) PWRK_H01-PWRK_H10;
     array wg(1:10) PWAG_H01-PWAG_H10;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 10;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=V84N;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
 
run;

***********************************************
** (2) VILL: PROCESS LABOR FROM SAME VILLAGE **
***********************************************;

** Placeholder for consistency **;

data work&f.16_&p._vill (keep=HHID&y HELPHH&y TOT_WRKS TOT_WAGE);
     set work&f.15_&p._vill;

     TOT_WRKS=NUMWRKS;
     TOT_WAGE=ALLWAGE;
RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._vill;
     merge work&f.16_&p._vill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;

data work&f.18_&p._vill (drop=HELPHH&y TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._vill;

     retain PID_H11-PID_H60 PWRK_H11-PWRK_H60 PWAG_H11-PWAG_H60 i;    ** Starts at 11 to leave room for village **; 

     length PID_H11-PID_H60 8;                 * 50 for good measure *;
     length PWRK_H11-PWRK_H60 8;             * 50 for good measure *;
     length PWAG_H11-PWAG_H60 8;             * 50 for good measure *;

     array id(1:50) PID_H11-PID_H60;
     array wk(1:50) PWRK_H11-PWRK_H60;
     array wg(1:50) PWAG_H11-PWAG_H60;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 50;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=HELPHH&y;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
run;

****************************************
** (3) OTHER: PROCESS ALL OTHER LABOR **
****************************************;

** This labor is aggregated and included in an attribute file **;

** Placeholder for consistency **;

data work&f.16_&p._other (keep=HHID&y HELPHH&y NUMWRKS ALLWAGE);
     set work&f.15_&p._other;

RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._other;
     merge work&f.16_&p._other (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;
** THIS FILE TO BE MERGED BELOW WITH OTHER ATTRIBUTES **;

data work&f.18_&p._other (drop=HELPHH&y NUMWRKS ALLWAGE);
     set work&f.17_&p._other;

     retain OTH_PWRK OTH_PWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            OTH_PWRK=0;
                            OTH_PWAG=0;
                          end;

     if NUMWRKS ne . then OTH_PWRK=OTH_PWRK+NUMWRKS;
     if ALLWAGE ne . then OTH_PWAG=OTH_PWAG+ALLWAGE;

    if last.HHID&y then output;
run;                                                             


***********************************
** Add other attribute variables ** 
***********************************;

** HH-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_hh (drop=HHID&y.C);
     set in&f.2.hh&y (keep=HHID&y RICE rename=(HHID&y=HHID&y.C));

     if RICE in (.,2,8) then RICE=0;

     IS_HH=1;

     HHID&y=input(HHID&y.C,best12.);
    
run;

** Merge with all rice-growing households **;

data work&f._&p._add_all_hh (drop=V84N LOCATION HELPHH&y);
     merge work&f.14_&p (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
                          
     if b=1 then output;
run;

data work&f._&p._add_tot_hh (drop=NUMWRKS ALLWAGE); **Generate Count Vars for ALL labor **;
     set work&f._&p._add_all_hh;

     retain ALL_PWRK ALL_PWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            ALL_PWRK=0;
                            ALL_PWAG=0;
                          end;

     if NUMWRKS ne . then ALL_PWRK=ALL_PWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_PWAG=ALL_PWAG+ALLWAGE;

    if last.HHID&y then output;
run;                           

** MERGE **;

data work&f.19_&p._all_hh;
     merge work&f._&p._add_rice_hh (in=a)
           work&f._&p._add_tot_hh (in=b)
           work&f.18_&p._other (in=c);
     by HHID&y;
     if c=1 then output;
run;

data work&f.19_&p._all_hh_2 (drop=V84);
     set work&f.19_&p._all_hh;
     V84N=input(V84,2.0);
run;

** VILLAGE-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_vill (drop=V84 HHID&y);
     set work&f.add_all_02;
     RICE=1;
     IS_HH=0;
     OTH_PWRK=0;
     OTH_PWAG=0;
     V84N=input(V84,2.0);
run;

** Prep for merge **;

proc sort data=work&f._&p._add_rice_vill out=work&f._&p._add_rice_vill_2 nodupkey;
     by V84N;
run;

proc sort data=work&f.14_&p out=work&f.14_&p._by_v84;
     by V84N;
run;

**Generate Count Vars for ALL labor **;

data work&f._&p._add_tot_vill (drop=HHID&y HELPHH&y NUMWRKS ALLWAGE); 
     set work&f.14_&p._by_v84;

     retain ALL_PWRK ALL_PWAG;
 
     by V84N;

     if first.V84N then do;
                            ALL_PWRK=0;
                            ALL_PWAG=0;
                          end;

     if NUMWRKS ne . then ALL_PWRK=ALL_PWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_PWAG=ALL_PWAG+ALLWAGE;

     if last.V84N then output;
run;                           

data work&f.19_&p._all_vill;
     merge work&f._&p._add_rice_vill_2 (in=a)
           work&f._&p._add_tot_vill (in=b);
     by V84N;

     HHID&y=V84N;
     if b=1 then output;
run;

***************************************
** Merge HH and VILL Attribute Files **
***************************************;

data work&f.19_&p._attrib;
     set work&f.19_&p._all_vill (drop=LOCATION)
         work&f.19_&p._all_hh_2;
run;

**********************************************
** Merge Same and Other Village Labor Files **
**********************************************;

data work&f.19_&p._adjlist;
     merge work&f.18_&p._othvill (in=b)
           work&f.18_&p._vill (in=a);
     by HHID&y;

     if a=1 and b=1 then output;
run;

***********************************************************
** Add Villages to adjacency matrix - all missing values **
***********************************************************;

proc sort data=vill_id_&f.02 out=add_vill_&p._&f._01 nodupkey;
     by V84;
run;

data add_vill_&p._&f._02 (drop=V84 j);
     set add_vill_&p._&f._01 (drop=HHID&y);

     length PID_H01-PID_H60 8;                 
     length PWRK_H01-PWRK_H60 8;             
     length PWAG_H01-PWAG_H60 8;             

     array id(1:60) PID_H01-PID_H60;
     array wk(1:60) PWRK_H01-PWRK_H60;
     array wg(1:60) PWAG_H01-PWAG_H60;
 
     do j= 1 to 60;
        id(j)=.;
        wk(j)=.;
        wg(j)=.;
     end;
 
     HHID&y=input(V84,best12.);    
     V84N=input(V84,2.);    

run;   

data add_vill_&p._&f._03 (drop=V84);
     set work&f.19_&p._adjlist;
     V84N=input(V84,2.0);
run;
 
data add_vill_&p._&f._04;
     set add_vill_&p._&f._02 
         add_vill_&p._&f._03;
run;


****************************************************************************
** UNPAID: Create Village Aggregate Measures for Help from Beyond Village **
****************************************************************************;

%let p=unpaid;    *** saves keystrokes ***;

data work&f.14_unpaid (drop=PAIDHH&y);
     set work&f.13;
     if PAIDHH&y=2;
run;

** SPLIT DATA INTO 3 GROUPS - SAME VILL, OTH VILL, ALL OTHER **;

data work&f.15_&p._othvill (drop=LOCATION);   ** 1 **;
     set work&f.14_&p;
     if LOCATION in (4);
     if HELPHH&y ne .;
run;

data work&f.15_&p._vill (drop=LOCATION);      ** 2 **;
     set work&f.14_&p;
     if LOCATION in (0,1);
     if HELPHH&y ne .;
run;

data work&f.15_&p._other (drop=LOCATION);     ** 3 **;
     set work&f.14_&p;
     if (LOCATION in (2,3,5,6,7,8,9) OR HELPHH&y=.);
run;

****************************************************
** (1) OTHVILL: PROCESS LABOR FROM OTHER VILLAGES **
****************************************************;

** Collapse Labor from the same 84 village **;

data work&f.16_&p._othvill (keep=HHID&y V84N TOT_WRKS TOT_WAGE);
     set work&f.15_&p._othvill;

     retain TOT_WRKS TOT_WAGE;

     by HHID&y V84N;

     if first.V84N then do;
                          TOT_WRKS=0;
                          TOT_WAGE=0;
                        end;
     if NUMWRKS ne . then TOT_WRKS=TOT_WRKS+NUMWRKS;
     if ALLWAGE ne . then TOT_WAGE=TOT_WAGE+ALLWAGE;

    if last.V84N then output;
 
run;

** Merge with all rice-growing households **;

data work&f.add_all_01 (drop=HHID&y.C); 
     set vill_id_&f.02 (rename=(HHID&y=HHID&y.C));
     HHID&y=input(HHID&y.C,best12.);
run;

proc sort data=work&f.add_all_01 out=work&f.add_all_02;
     by HHID&y;
run;

data work&f.17_&p._othvill;
     merge work&f.16_&p._othvill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;


data work&f.18_&p._othvill (drop=V84N TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._othvill;

     retain UID_H01-UID_H10 UWRK_H01-UWRK_H10 UWAG_H01-UWAG_H10 i;

     length UID_H01-UID_H10 8;             * 10 for good measure *;
     length UWRK_H01-UWRK_H10 8;             * 10 for good measure *;
     length UWAG_H01-UWAG_H10 8;             * 10 for good measure *;

     array id(1:10) UID_H01-UID_H10;
     array wk(1:10) UWRK_H01-UWRK_H10;
     array wg(1:10) UWAG_H01-UWAG_H10;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 10;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=V84N;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
 
run;

***********************************************
** (2) VILL: PROCESS LABOR FROM SAME VILLAGE **
***********************************************;

** Placeholder for consistency **;

data work&f.16_&p._vill (keep=HHID&y HELPHH&y TOT_WRKS TOT_WAGE);
     set work&f.15_&p._vill;

     TOT_WRKS=NUMWRKS;
     TOT_WAGE=ALLWAGE;
RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._vill;
     merge work&f.16_&p._vill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;

data work&f.18_&p._vill (drop=HELPHH&y TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._vill;

     retain UID_H11-UID_H60 UWRK_H11-UWRK_H60 UWAG_H11-UWAG_H60 i;    ** Starts at 11 to leave room for village **; 

     length UID_H11-UID_H60 8;                 * 50 for good measure *;
     length UWRK_H11-UWRK_H60 8;             * 50 for good measure *;
     length UWAG_H11-UWAG_H60 8;             * 50 for good measure *;

     array id(1:50) UID_H11-UID_H60;
     array wk(1:50) UWRK_H11-UWRK_H60;
     array wg(1:50) UWAG_H11-UWAG_H60;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 50;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=HELPHH&y;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
run;

****************************************
** (3) OTHER: PROCESS ALL OTHER LABOR **
****************************************;

** This labor is aggregated and included in an attribute file **;

** Placeholder for consistency **;

data work&f.16_&p._other (keep=HHID&y HELPHH&y NUMWRKS ALLWAGE);
     set work&f.15_&p._other;

RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._other;
     merge work&f.16_&p._other (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;
** THIS FILE TO BE MERGED BELOW WITH OTHER ATTRIBUTES **;

data work&f.18_&p._other (drop=HELPHH&y NUMWRKS ALLWAGE);
     set work&f.17_&p._other;

     retain OTH_UWRK OTH_UWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            OTH_UWRK=0;
                            OTH_UWAG=0;
                          end;

     if NUMWRKS ne . then OTH_UWRK=OTH_UWRK+NUMWRKS;
     if ALLWAGE ne . then OTH_UWAG=OTH_UWAG+ALLWAGE;

    if last.HHID&y then output;
run;                                                             


***********************************
** Add other attribute variables ** 
***********************************;

** HH-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_hh (drop=HHID&y.C);
     set in&f.2.hh&y (keep=HHID&y RICE rename=(HHID&y=HHID&y.C));

     if RICE in (.,2,8) then RICE=0;

     IS_HH=1;

     HHID&y=input(HHID&y.C,best12.);

run;

** Merge with all rice-growing households **;

data work&f._&p._add_all_hh (drop=V84N LOCATION HELPHH&y);
     merge work&f.14_&p (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
                          
     if b=1 then output;
run;

data work&f._&p._add_tot_hh (drop=NUMWRKS ALLWAGE); **Generate Count Vars for ALL labor **;
     set work&f._&p._add_all_hh;

     retain ALL_UWRK ALL_UWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            ALL_UWRK=0;
                            ALL_UWAG=0;
                          end;

     if NUMWRKS ne . then ALL_UWRK=ALL_UWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_UWAG=ALL_UWAG+ALLWAGE;

    if last.HHID&y then output;
run;                           

** MERGE **;

data work&f.19_&p._all_hh;
     merge work&f._&p._add_rice_hh (in=a)
           work&f._&p._add_tot_hh (in=b)
           work&f.18_&p._other (in=c);
     by HHID&y;
     if c=1 then output;
run;

data work&f.19_&p._all_hh_2 (drop=V84);
     set work&f.19_&p._all_hh;
     V84N=input(V84,2.0);
run;

** VILLAGE-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_vill (drop=V84 HHID&y);
     set work&f.add_all_02;
     RICE=1;
     IS_HH=0;
     OTH_UWRK=0;
     OTH_UWAG=0;
     V84N=input(V84,2.0);
run;

** Prep for merge **;

proc sort data=work&f._&p._add_rice_vill out=work&f._&p._add_rice_vill_2 nodupkey;
     by V84N;
run;

proc sort data=work&f.14_&p out=work&f.14_&p._by_v84;
     by V84N;
run;

**Generate Count Vars for ALL labor **;

data work&f._&p._add_tot_vill (drop=HHID&y HELPHH&y NUMWRKS ALLWAGE); 
     set work&f.14_&p._by_v84;

     retain ALL_UWRK ALL_UWAG;
 
     by V84N;

     if first.V84N then do;
                            ALL_UWRK=0;
                            ALL_UWAG=0;
                          end;

     if NUMWRKS ne . then ALL_UWRK=ALL_UWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_UWAG=ALL_UWAG+ALLWAGE;

     if last.V84N then output;
run;                           

data work&f.19_&p._all_vill;
     merge work&f._&p._add_rice_vill_2 (in=a)
           work&f._&p._add_tot_vill (in=b);
     by V84N;

     HHID&y=V84N;
     if b=1 then output;
run;

***************************************
** Merge HH and VILL Attribute Files **
***************************************;

data work&f.19_&p._attrib;
     set work&f.19_&p._all_vill (drop=LOCATION)
         work&f.19_&p._all_hh_2;
run;

**********************************************
** Merge Same and Other Village Labor Files **
**********************************************;

data work&f.19_&p._adjlist;
     merge work&f.18_&p._othvill (in=b)
           work&f.18_&p._vill (in=a);
     by HHID&y;

     if a=1 and b=1 then output;
run;

***********************************************************
** Add Villages to adjacency matrix - all missing values **
***********************************************************;

proc sort data=vill_id_&f.02 out=add_vill_&p._&f._01 nodupkey;
     by V84;
run;

data add_vill_&p._&f._02 (drop=V84 j);
     set add_vill_&p._&f._01 (drop=HHID&y);

     length UID_H01-UID_H60 8;                 
     length UWRK_H01-UWRK_H60 8;             
     length UWAG_H01-UWAG_H60 8;             

     array id(1:60) UID_H01-UID_H60;
     array wk(1:60) UWRK_H01-UWRK_H60;
     array wg(1:60) UWAG_H01-UWAG_H60;
 
     do j= 1 to 60;
        id(j)=.;
        wk(j)=.;
        wg(j)=.;
     end;
 
     HHID&y=input(V84,best12.);    
     V84N=input(V84,2.);    

run;   

data add_vill_&p._&f._03 (drop=V84);
     set work&f.19_&p._adjlist;
     V84N=input(V84,2.0);
run;
 
data add_vill_&p._&f._04;
     set add_vill_&p._&f._02 
         add_vill_&p._&f._03;
run;


*******************************************************************************************************************;

*********************************************************************
** Temporarily Merge Together All 4 Final Files to purge ID errors **
*********************************************************************;

proc sort data=add_vill_paid_&f._04 out=work&f.20_paid_adj;
     by HHID&y V84N;
run;

proc sort data=add_vill_unpaid_&f._04 out=work&f.20_unpaid_adj;
     by HHID&y V84N;
run;

proc sort data=work&f.19_paid_attrib out=work&f.20_paid_att;
     by HHID&y V84N;
run;

proc sort data=work&f.19_unpaid_attrib out=work&f.20_unpaid_att;
     by HHID&y V84N;
run;


data work&f.21_all_files;
     merge work&f.20_paid_adj (in=a)
           work&f.20_unpaid_adj (in=b)
           work&f.20_paid_att (in=c)
           work&f.20_unpaid_att (drop=RICE IS_HH in=d);
     by HHID&y V84N;
     if a=1 and b=1 and c=1 and d=1 then output;
run;  

*******************************************
** Format files to be exported to UCINET **
*******************************************;

** I have decided to revert these matrices back to hh-only - 2011 02 07 **;
** These changes are reflected only in the four data steps below **;
** The village-level information still exists in all prior data steps **;
** In the future, I'd like to add a village-level only analysis **;

** I have also re-combined all files into a single one as of 2011 02 09 **;

data r&y._all (drop=PID_H01-PID_H10 PWRK_H01-PWRK_H10 PWAG_H01-PWAG_H10
                    UID_H01-UID_H10 UWRK_H01-UWRK_H10 UWAG_H01-UWAG_H10);
     set work&f.21_all_files;

     ALL_WRKS=ALL_PWRK+ALL_UWRK;
     ALL_WAGE=ALL_PWAG+ALL_UWAG;

     if HHID&y>100;               * Remove village datalines *;
run;

**************************************************************************************************;

***************************************************************************************
* 8 ** VILLAGE NETWORKS: RICE UNPAID ** Create VALUED adjacency matrices: # WORKERS  **
***************************************************************************************;

** Use transpose in final step to reverse direction of ties, indicating labor movement **;

************************************************
** Create separate village files EACH VILLAGE **
************************************************;

%macro v_split (numvill=);  %* macro splits villages *;

       %* NUMVILL=Number of Unique Villages in file *;

%do i=1 %to &numvill;

    data r00_u&i (drop=V84N);
         set r&y._all;
         if V84N=&i;
    run;

%end;

%mend v_split;

%v_split (numvill=51);

%macro v_adj1 (numvill=);
%do i=1 %to &numvill;
proc iml;
     %include '/home/jhull/public/span/adjval.mod';
     %include '/home/jhull/public/span/pajwrite.mod';
     %let p1=%quote(/home/jhull/nangrong/data_paj/2000/a2-net/v0);
     %let p2=%quote(00a2.net);
     use r00_u&i;
     read all var{
                  UID_H11 UID_H12 UID_H13 UID_H14 UID_H15
                  UID_H16 UID_H17 UID_H18 UID_H19 UID_H20
                  UID_H21 UID_H22 UID_H23 UID_H24 UID_H25
                  UID_H26 UID_H27 UID_H28 UID_H29 UID_H30
                  UID_H31 UID_H32 UID_H33 UID_H34 UID_H35
                  UID_H36 UID_H37 UID_H38 UID_H39 UID_H40
                  UID_H41 UID_H42 UID_H43 UID_H44 UID_H45
                  UID_H46 UID_H47 UID_H48 UID_H49 UID_H50
                  UID_H51 UID_H52 UID_H53 UID_H54 UID_H55
                  UID_H56 UID_H57 UID_H58 UID_H59 UID_H60} into rcv;
     read all var{
                  UWRK_H11 UWRK_H12 UWRK_H13 UWRK_H14 UWRK_H15
                  UWRK_H16 UWRK_H17 UWRK_H18 UWRK_H19 UWRK_H20
                  UWRK_H21 UWRK_H22 UWRK_H23 UWRK_H24 UWRK_H25
                  UWRK_H26 UWRK_H27 UWRK_H28 UWRK_H29 UWRK_H30
                  UWRK_H31 UWRK_H32 UWRK_H33 UWRK_H34 UWRK_H35
                  UWRK_H36 UWRK_H37 UWRK_H38 UWRK_H39 UWRK_H40
                  UWRK_H41 UWRK_H42 UWRK_H43 UWRK_H44 UWRK_H45
                  UWRK_H46 UWRK_H47 UWRK_H48 UWRK_H49 UWRK_H50
                  UWRK_H51 UWRK_H52 UWRK_H53 UWRK_H54 UWRK_H55
                  UWRK_H56 UWRK_H57 UWRK_H58 UWRK_H59 UWRK_H60} into val;
     read all var{HHID00} into snd;
     r00_u=adjval(snd,rcv,val);
     id=r00_u[,1];
     r00_u=r00_u[,2:ncol(r00_u)];
     adj=r00_u;
     file "&p1.&i.&p2";
     call pajwrite(adj`,id,2);
     adjinv=adj`;
     create adj0&i from adjinv;
            append from adjinv;
     idinv=id`;
     create id0&i from idinv;
            append from idinv;
quit;
%end;
%mend v_adj1;

%v_adj1(numvill=9);


%macro v_adj2 (numvill=);
%do i=10 %to &numvill;
proc iml;
     %include '/home/jhull/public/span/adjval.mod';
     %include '/home/jhull/public/span/pajwrite.mod';
     %let p1=%quote(/home/jhull/nangrong/data_paj/2000/a2-net/v);
     %let p2=%quote(00a2.net);
     use r00_u&i;
     read all var{
                  UID_H11 UID_H12 UID_H13 UID_H14 UID_H15
                  UID_H16 UID_H17 UID_H18 UID_H19 UID_H20
                  UID_H21 UID_H22 UID_H23 UID_H24 UID_H25
                  UID_H26 UID_H27 UID_H28 UID_H29 UID_H30
                  UID_H31 UID_H32 UID_H33 UID_H34 UID_H35
                  UID_H36 UID_H37 UID_H38 UID_H39 UID_H40
                  UID_H41 UID_H42 UID_H43 UID_H44 UID_H45
                  UID_H46 UID_H47 UID_H48 UID_H49 UID_H50
                  UID_H51 UID_H52 UID_H53 UID_H54 UID_H55
                  UID_H56 UID_H57 UID_H58 UID_H59 UID_H60} into rcv;
     read all var{
                  UWRK_H11 UWRK_H12 UWRK_H13 UWRK_H14 UWRK_H15
                  UWRK_H16 UWRK_H17 UWRK_H18 UWRK_H19 UWRK_H20
                  UWRK_H21 UWRK_H22 UWRK_H23 UWRK_H24 UWRK_H25
                  UWRK_H26 UWRK_H27 UWRK_H28 UWRK_H29 UWRK_H30
                  UWRK_H31 UWRK_H32 UWRK_H33 UWRK_H34 UWRK_H35
                  UWRK_H36 UWRK_H37 UWRK_H38 UWRK_H39 UWRK_H40
                  UWRK_H41 UWRK_H42 UWRK_H43 UWRK_H44 UWRK_H45
                  UWRK_H46 UWRK_H47 UWRK_H48 UWRK_H49 UWRK_H50
                  UWRK_H51 UWRK_H52 UWRK_H53 UWRK_H54 UWRK_H55
                  UWRK_H56 UWRK_H57 UWRK_H58 UWRK_H59 UWRK_H60} into val;
     read all var{HHID00} into snd;
     r00_u=adjval(snd,rcv,val);
     id=r00_u[,1];
     r00_u=r00_u[,2:ncol(r00_u)];
     adj=r00_u;
     file "&p1.&i.&p2";
     call pajwrite(adj`,id,2);
     adjinv=adj`;
     create adj&i from adjinv;
            append from adjinv;
     idinv=id`;
     create id&i from idinv;
            append from idinv;
quit;
%end;
%mend v_adj2;

%v_adj2(numvill=51);

%macro v_adj3(numvill=);
%do i=1 %to &numvill;
    data _null_ ;
         %let p1=%quote(/home/jhull/nangrong/data_paj/2000/a2-adj/v0);
         %let p2=%quote(00a2.adj);
         set adj0&i;
         file "&p1.&i.&p2"  lrecl=1000;
         put (_ALL_) (+0);
run;
    data _null_ ;
         %let p3=%quote(/home/jhull/nangrong/data_paj/2000/a2-id/v0);
         %let p4=%quote(00a2.id);
         set id0&i;
         file "&p3.&i.&p4"  lrecl=10000;
         put (_ALL_) (+0);
run;

%end;
%mend v_adj3;

%v_adj3(numvill=9);

%macro v_adj4(numvill=);
%do i=10 %to &numvill;  
    data _null_ ;
         %let p1=%quote(/home/jhull/nangrong/data_paj/2000/a2-adj/v);
         %let p2=%quote(00a2.adj);
         set adj&i;
         file "&p1.&i.&p2"  lrecl=1000;
         put (_ALL_) (+0);
run;
    data _null_ ;
         %let p3=%quote(/home/jhull/nangrong/data_paj/2000/a2-id/v);
         %let p4=%quote(00a2.id);
         set id&i;
         file "&p3.&i.&p4"  lrecl=10000;
         put (_ALL_) (+0);
run;
%end;
%mend v_adj4;

%v_adj4(numvill=51);







In [ ]:
********************************************************************
**     Program Name: /home/jhull/nangrong/prog_sas/p02_nets/2000/p02_00a3.sas
**     Programmer: james r. hull
**     Start Date: 2011 03 09
**     Purpose:
**        1.) Create A3 = PAID LABOR NETWORKS TOTAL FLOW IN BAHT
**
**     Input Data:
**        '/home/jhull/nangrong/data_sas/2000/current/indiv00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/hh00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/comm00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/sibs00.xpt'
**
**     Output Data:
**        1.) /home/jhull/nangrong/data_paj/2000/a3/vXX00a3.net
**
**     Notes: 
**        1.) 
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=60;

%let f=00a3;  ** names file (allows portability) **;
%let y=00; 

**********************
**  Data Libraries  **
**********************;

libname in&f.1 xport '/home/jhull/nangrong/data_sas/2000/current/indiv00.xpt';
libname in&f.2 xport '/home/jhull/nangrong/data_sas/2000/current/hh00.xpt';
libname in&f.3 xport '/home/jhull/nangrong/data_sas/2000/current/comm00.xpt';
libname in&f.4 xport '/home/jhull/nangrong/data_sas/2000/current/sibs00.xpt';


******************************
**  Create Working Dataset  **
******************************;

********************************************************************
**  Length 10
**  0 In this 1984 village                2 + Village # + House #
**  2 In this village but blt             2 + Village # + 999
**  Length 7 
**  4 Another village in Nang Rong        3 + Village #
**  8 Unknown village in Nang Rong        3 + 999999
**  5 Another district in Buriram         4 + District # + 0000
**  6 Another province                    5 + District # + 0000
**  7 Another country 
**  8 Missing/Don't know                  9999999999 or 9999999
**  Length 9
**  9 Code 2 or 3 Returning HH member
*******************************************************************;

***************************************************************************
** Stack rice harvest labor data into a child file and label by location **
***************************************************************************;

data work&f.01a;
     set in&f.2.hh00 (keep=HHID00 X6_84C: X6_84W:);
     keep HHID00 X6_86L X6_86N X6_86W LOCATION;

     length X6_86L $ 10;

     array a(1:7) X6_84C1-X6_84C7;
     array b(1:7) X6_84W1-X6_84W7;

     do i=1 to 7;
          X6_86L=a(i);
          X6_86N=1;
          X6_86W=b(i);
          LOCATION=9;
          if a(i) ne " ." then output;  * Keep only those cases with data *;
     end;
run;

data work&f.01b;
     set in&f.2.hh00 (keep=HHID00 X6_85H: X6_85N: X6_85W:);
     keep HHID00 X6_86L X6_86N X6_86W LOCATION;

     length X6_86L $ 10;

     array a(1:13) X6_85H1-X6_85H13;
     array b(1:13) X6_85N1-X6_85N13;
     array c(1:13) X6_85W1-X6_85W13;

     do i=1 to 13;
 
          ** FIX ANOTHER CRAZY ERROR IN ORIGINAL DATA WITH REVERSED CODING FOR IN-VILLAGE LABOR **;
         
          if substr(a(i),5,3) in ('020','021','170','180') then a(i)=cat("2",substr(a(i),5,6),substr(a(i),2,3));

          if a(i)="9999999999" then LOCATION=8;
             else if substr(a(i),8,3)='999' then LOCATION=2;
             else LOCATION=0;

          X6_86L=a(i);
          X6_86N=b(i);
          X6_86W=c(i);

          if a(i) ne "         ." then output;  * Keep only those cases with data *;
     end;
run;

data work&f.01c;
     set in&f.2.hh00 (keep=HHID00 X6_86L: X6_86N: X6_86W:);
     keep HHID00 X6_86L X6_86N X6_86W LOCATION;

     length X6_86L $ 10;

     array a(1:10) X6_86L1-X6_86L10;
     array b(1:10) X6_86N1-X6_86N10;
     array c(1:10) X6_86W1-X6_86W10;

     do i=1 to 10;
          X6_86L=a(i);
          X6_86N=b(i);
          X6_86W=c(i);
          if a(i)='9999999' then LOCATION=8;
             else if substr(a(i),1,1)='5' then LOCATION=6;
             else if substr(a(i),1,1)='4' then LOCATION=5;
             else if substr(a(i),1,2)='39' then LOCATION=8;
             else LOCATION=4;
          if a(i) ne "      ." then output;  * Keep only those cases with data *;
     end;

run;

*********************************************************
** Take Care of Missing Data Issues - Recodes at least **
*********************************************************;

data work&f.02a;
     set work&f.01a;

     if X6_86W=9 then X6_86W=.;
     if X6_86N=99 then X6_86N=1; * Assume at least 1 person worked *;

run;

****************************************************************************
** This code collapses multiple code 2 & 3 workers from a household to    **
** a single observation and sums the values for each into summary counts. **
** For the "type of labor" variable, I use an "any paid --> paid" rule    **
** because paying a code 2 or 3 laborer is a rare behavior and distinct   **
****************************************************************************;

 data work&f.02a2;
     set work&f.02a;

     by HHID00;

     retain SUM_N SUM_TYPE SUM_LOC i;


     if first.HHID00 then do;
                            SUM_N=0;
                            SUM_TYPE=2; * Default is unpaid labor*;
                            SUM_LOC=9;
                            i=1;
                         end;

     SUM_N=SUM_N+X6_86N;
     if X6_86W=1 then SUM_TYPE=1;  * Any paid --> all paid *;
     SUM_LOC=9;
     i=i+1;

     if last.HHID00 then output;

run;

data work&f.02a3 (drop=SUM_N SUM_TYPE SUM_LOC i);
     set work&f.02a2;

     X6_86L="   ";
     X6_86N=SUM_N;
     X6_86W=SUM_TYPE;
     LOCATION=SUM_LOC;

run;

*********************************************************
** Take Care of Missing Data Issues - Recodes at least **
*********************************************************;

data work&f.02b;
     set work&f.01b;

     if X6_86W=9 then X6_86W=.;
     if X6_86N=99 then X6_86N=1; * Assume at least 1 person worked *;
run;

data work&f.02c;
     set work&f.01c;

     if X6_86W=9 then X6_86W=.;
     if X6_86N=99 then X6_86N=1; * Assume at least 1 person worked *;
run;

**************************
** Merge files together **
**************************;

data work&f.03;
     set work&f.02a3
         work&f.02b
         work&f.02c;
run;

***************************************************************************
** Add V84 identifiers to 2000 data file as per Rick's suggestion on web **
***************************************************************************;



proc sort data=work&f.03 out=work&f.04;
     by HHID00 X6_86L LOCATION;
run;

data vill_id_&f.01;
     set in&f.1.indiv00;
     keep HHID00 V84;
run;

proc sort data=vill_id_&f.01 out=vill_id_&f.02 nodupkey;
     by HHID00 v84;
run;

data vill_id_&f.03;
     merge work&f.04 (in=a)
           vill_id_&f.02 (in=b);
           if a=1 and b=1 then output;
     by HHID00;
run;

proc sort data=vill_id_&f.03 out=work&f.05;
     by V84;
run;

******************************************************************************
** This step removes all cases about which there is no information about    **
** how their laborers were compensated. This is my fix for the time being.  **
** Note: in doing so, I lose 7 cases (a case here is a helper group)        **
******************************************************************************;

data work&f.06;
     set work&f.05;

     rename X6_86L=HELPHHID;

     if X6_86W ^in (.,9) then output;

run;

************************************************************************************
** The steps below convert the ban lek ti information on the helping household    **
** into the standard HHID##, as a preparatory step to creating network datafiles. **
************************************************************************************;

data work&f.07;
     set work&f.06;

     if LOCATION=9 then do;
                           HELPHHID=HHID00;
                        end;
        else if LOCATION in(0,4) then do;
                                  HELPHHID=substr(HELPHHID,2,9);
                                end;
        else if LOCATION in (2) then do;
                                    HELPHHID=substr(HELPHHID,2,6);
                                end;
        else HELPHHID=".";
run;

*********************************
** Clean HELPHH00 of BAD CODES **
*********************************; 

data work&f.08a (drop=HHID&y.C);
     set work&f.07 (rename=(HHID&y=HHID&y.C));
     HELPHH00=input(HELPHHID,9.0);
     V84N=input(V84,2.0);
     HHID00=input(HHID&y.C,9.);
run;

data work&f.08b (drop=HHID00 V84);
     set in&f.1.indiv00 (keep=HHID00 V84);
     HELPHH00=input(HHID00,9.0);
     V84N=input(V84,2.0);
run;

proc sort data=work&f.08a out=work&f.09a;
     by V84N HELPHH00;
run;

proc sort data=work&f.08b out=work&f.09b nodupkey;
     by V84N HELPHH00;
run;

data work&f.10 aonly bonly;
     merge work&f.09a (in=a)
           work&f.09b (in=b);
     by V84N HELPHH00;
     if a=1 and b=1 then output work&f.10;
     if a=1 and b=0 then output aonly;
     if a=0 and b=1 then output bonly;
run;


** This step removes the HELPHH00 codes from a number of HHs **;
** that had non-existent or incorrect HHs listed as helping  **;

data aonly_fix;
     set aonly;
     if HELPHH00>999999 then do;     
                               HELPHH00=".";
                               HELPHHID=".";
                             end;
run;
 
data work&f.11;
     set work&f.10 (in=a)
         aonly_fix (in=b);
run;

************************************************************************
**input average village wages during high demand from community survey**
************************************************************************;

data work&f.vill_wages_01a (drop=X45MHIGH X45MTYP);  
    set in&f.3.comm&y (keep=VILL00 X45MHIGH X45MTYP);

    if X45MHIGH=9999998 then X45MHIGH=.; ** USING ONLY MALES B/C MALE AND FEMALE WAGES IDENTICAL **;
    if X45MTYP=9999998 then X45MTYP=.;

    if X45MHIGH=. then RICEWAGH=125.77;
       else RICEWAGH=X45MHIGH;
    if X45MTYP=. then RICEWAGN=105.29;
       else RICEWAGN=X45MTYP;
run;

proc sort data=work&f.vill_wages_01a out=work&f.vill_wages_01b nodupkey;
     by VILL&y;
run;

proc sort data=in&f.1.indiv&y  out=work&f.vill_wages_02a nodupkey;
     by HHID&y;
run;

data work&f.make_vill;
     set work&f.vill_wages_02a (keep=HHID&y);
     length VILL&y $ 6;

     VILL&y=substr(HHID&y,1,6);
run;

data work&f.numeric (drop=HHID&y.C);
     set work&f.make_vill (rename=(HHID&y=HHID&y.C));

     HHID&y=input(HHID&y.C,best9.);
run;

proc sort data=work&f.numeric out=work&f.vill_wages_02b;
     by VILL&y HHID&y;
run;

data work&f.vill_wages_03;
     merge work&f.vill_wages_01b (in=a)
           work&f.vill_wages_02b (in=b);
     by VILL&y;
     if b=1 then output;
run;

proc sort data=work&f.vill_wages_03 out=work&f.vill_wages_04;
     by HHID&y;
run;

proc sort data=work&f.11 out=work&f.vill_wages_05;
     by HHID&y;
run;

data work&f.vill_wages_06;
     merge work&f.vill_wages_04 (in=a)
           work&f.vill_wages_05 (in=b);
     by HHID&y;
     if b=1 then output;
run;

proc sort data=work&f.vill_wages_06 out=work&f.11B;
     by HHID&y;
run;

data work&f.11C;
     set work&f.11B;

     if X6_86W=3 then PAIDHH00=2;
        else PAIDHH00=X6_86W;

     if X6_86W=1 then ALLWAGE=round(X6_86N*RICEWAGH,.01);   ** ACTUAL DAYS WORKED*AVG WAGES IN VILL**;
        else ALLWAGE=0;                                             

     NUMWRKS=X6_86N;
  
run;

data work&f.12 (keep=HHID00 PAIDHH00 HELPHH00 LOCATION ALLWAGE NUMWRKS V84N);
     set work&f.11C;
run;

proc sort data=work&f.12 out=work&f.13;              ** MAJOR CHECKPOINT - TOTAL LABOR CHILD FILE **;
     by HHID&y HELPHH&y;
run;

*******************************************************************************************************************************;

***********************************************
** Separate Paid and Unpaid Labor into Files **
***********************************************;

** Note, all code below must be repeated for paid and then unpaid labor **;

**************************************************************************
** PAID: Create Village Aggregate Measures for Help from Beyond Village **
**************************************************************************;

%let p=paid;    *** saves keystrokes ***;

data work&f.14_&p (drop=PAIDHH&y);
     set work&f.13;
     if PAIDHH&y=1;
run;

** SPLIT DATA INTO 3 GROUPS - SAME VILL, OTH VILL, ALL OTHER **;

data work&f.15_&p._othvill (drop=LOCATION);   ** 1 **;
     set work&f.14_&p;
     if LOCATION in (4);
     if HELPHH&y ne .;
run;

data work&f.15_&p._vill (drop=LOCATION);      ** 2 **;
     set work&f.14_&p;
     if LOCATION in (0,1);
     if HELPHH&y ne .;
run;

data work&f.15_&p._other (drop=LOCATION);     ** 3 **;
     set work&f.14_&p;
     if (LOCATION in (2,3,5,6,7,8,9) OR HELPHH&y=.);
run;

****************************************************
** (1) OTHVILL: PROCESS LABOR FROM OTHER VILLAGES **
****************************************************;

** Collapse Labor from the same 84 village **;

data work&f.16_&p._othvill (keep=HHID&y V84N TOT_WRKS TOT_WAGE);
     set work&f.15_&p._othvill;

     retain TOT_WRKS TOT_WAGE;

     by HHID&y V84N;

     if first.V84N then do;
                          TOT_WRKS=0;
                          TOT_WAGE=0;
                        end;
     if NUMWRKS ne . then TOT_WRKS=TOT_WRKS+NUMWRKS;
     if ALLWAGE ne . then TOT_WAGE=TOT_WAGE+ALLWAGE;

    if last.V84N then output;
 
run;

** Merge with all rice-growing households **;

data work&f.add_all_01 (drop=HHID&y.C); 
     set vill_id_&f.02 (rename=(HHID&y=HHID&y.C));
     HHID&y=input(HHID&y.C,best12.);
run;

proc sort data=work&f.add_all_01 out=work&f.add_all_02;
     by HHID&y;
run;

data work&f.17_&p._othvill;
     merge work&f.16_&p._othvill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;

data work&f.18_&p._othvill (drop=V84N TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._othvill;

     retain PID_H01-PID_H10 PWRK_H01-PWRK_H10 PWAG_H01-PWAG_H10 i;

     length PID_H01-PID_H10 8;             * 10 for good measure *;
     length PWRK_H01-PWRK_H10 8;             * 10 for good measure *;
     length PWAG_H01-PWAG_H10 8;             * 10 for good measure *;

     array id(1:10) PID_H01-PID_H10;
     array wk(1:10) PWRK_H01-PWRK_H10;
     array wg(1:10) PWAG_H01-PWAG_H10;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 10;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=V84N;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
 
run;

***********************************************
** (2) VILL: PROCESS LABOR FROM SAME VILLAGE **
***********************************************;

** Placeholder for consistency **;

data work&f.16_&p._vill (keep=HHID&y HELPHH&y TOT_WRKS TOT_WAGE);
     set work&f.15_&p._vill;

     TOT_WRKS=NUMWRKS;
     TOT_WAGE=ALLWAGE;
RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._vill;
     merge work&f.16_&p._vill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;

data work&f.18_&p._vill (drop=HELPHH&y TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._vill;

     retain PID_H11-PID_H60 PWRK_H11-PWRK_H60 PWAG_H11-PWAG_H60 i;    ** Starts at 11 to leave room for village **; 

     length PID_H11-PID_H60 8;                 * 50 for good measure *;
     length PWRK_H11-PWRK_H60 8;             * 50 for good measure *;
     length PWAG_H11-PWAG_H60 8;             * 50 for good measure *;

     array id(1:50) PID_H11-PID_H60;
     array wk(1:50) PWRK_H11-PWRK_H60;
     array wg(1:50) PWAG_H11-PWAG_H60;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 50;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=HELPHH&y;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
run;

****************************************
** (3) OTHER: PROCESS ALL OTHER LABOR **
****************************************;

** This labor is aggregated and included in an attribute file **;

** Placeholder for consistency **;

data work&f.16_&p._other (keep=HHID&y HELPHH&y NUMWRKS ALLWAGE);
     set work&f.15_&p._other;

RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._other;
     merge work&f.16_&p._other (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;
** THIS FILE TO BE MERGED BELOW WITH OTHER ATTRIBUTES **;

data work&f.18_&p._other (drop=HELPHH&y NUMWRKS ALLWAGE);
     set work&f.17_&p._other;

     retain OTH_PWRK OTH_PWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            OTH_PWRK=0;
                            OTH_PWAG=0;
                          end;

     if NUMWRKS ne . then OTH_PWRK=OTH_PWRK+NUMWRKS;
     if ALLWAGE ne . then OTH_PWAG=OTH_PWAG+ALLWAGE;

    if last.HHID&y then output;
run;                                                             


***********************************
** Add other attribute variables ** 
***********************************;

** HH-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_hh (drop=HHID&y.C);
     set in&f.2.hh&y (keep=HHID&y RICE rename=(HHID&y=HHID&y.C));

     if RICE in (.,2,8) then RICE=0;

     IS_HH=1;

     HHID&y=input(HHID&y.C,best12.);
    
run;

** Merge with all rice-growing households **;

data work&f._&p._add_all_hh (drop=V84N LOCATION HELPHH&y);
     merge work&f.14_&p (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
                          
     if b=1 then output;
run;

data work&f._&p._add_tot_hh (drop=NUMWRKS ALLWAGE); **Generate Count Vars for ALL labor **;
     set work&f._&p._add_all_hh;

     retain ALL_PWRK ALL_PWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            ALL_PWRK=0;
                            ALL_PWAG=0;
                          end;

     if NUMWRKS ne . then ALL_PWRK=ALL_PWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_PWAG=ALL_PWAG+ALLWAGE;

    if last.HHID&y then output;
run;                           

** MERGE **;

data work&f.19_&p._all_hh;
     merge work&f._&p._add_rice_hh (in=a)
           work&f._&p._add_tot_hh (in=b)
           work&f.18_&p._other (in=c);
     by HHID&y;
     if c=1 then output;
run;

data work&f.19_&p._all_hh_2 (drop=V84);
     set work&f.19_&p._all_hh;
     V84N=input(V84,2.0);
run;

** VILLAGE-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_vill (drop=V84 HHID&y);
     set work&f.add_all_02;
     RICE=1;
     IS_HH=0;
     OTH_PWRK=0;
     OTH_PWAG=0;
     V84N=input(V84,2.0);
run;

** Prep for merge **;

proc sort data=work&f._&p._add_rice_vill out=work&f._&p._add_rice_vill_2 nodupkey;
     by V84N;
run;

proc sort data=work&f.14_&p out=work&f.14_&p._by_v84;
     by V84N;
run;

**Generate Count Vars for ALL labor **;

data work&f._&p._add_tot_vill (drop=HHID&y HELPHH&y NUMWRKS ALLWAGE); 
     set work&f.14_&p._by_v84;

     retain ALL_PWRK ALL_PWAG;
 
     by V84N;

     if first.V84N then do;
                            ALL_PWRK=0;
                            ALL_PWAG=0;
                          end;

     if NUMWRKS ne . then ALL_PWRK=ALL_PWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_PWAG=ALL_PWAG+ALLWAGE;

     if last.V84N then output;
run;                           

data work&f.19_&p._all_vill;
     merge work&f._&p._add_rice_vill_2 (in=a)
           work&f._&p._add_tot_vill (in=b);
     by V84N;

     HHID&y=V84N;
     if b=1 then output;
run;

***************************************
** Merge HH and VILL Attribute Files **
***************************************;

data work&f.19_&p._attrib;
     set work&f.19_&p._all_vill (drop=LOCATION)
         work&f.19_&p._all_hh_2;
run;

**********************************************
** Merge Same and Other Village Labor Files **
**********************************************;

data work&f.19_&p._adjlist;
     merge work&f.18_&p._othvill (in=b)
           work&f.18_&p._vill (in=a);
     by HHID&y;

     if a=1 and b=1 then output;
run;

***********************************************************
** Add Villages to adjacency matrix - all missing values **
***********************************************************;

proc sort data=vill_id_&f.02 out=add_vill_&p._&f._01 nodupkey;
     by V84;
run;

data add_vill_&p._&f._02 (drop=V84 j);
     set add_vill_&p._&f._01 (drop=HHID&y);

     length PID_H01-PID_H60 8;                 
     length PWRK_H01-PWRK_H60 8;             
     length PWAG_H01-PWAG_H60 8;             

     array id(1:60) PID_H01-PID_H60;
     array wk(1:60) PWRK_H01-PWRK_H60;
     array wg(1:60) PWAG_H01-PWAG_H60;
 
     do j= 1 to 60;
        id(j)=.;
        wk(j)=.;
        wg(j)=.;
     end;
 
     HHID&y=input(V84,best12.);    
     V84N=input(V84,2.);    

run;   

data add_vill_&p._&f._03 (drop=V84);
     set work&f.19_&p._adjlist;
     V84N=input(V84,2.0);
run;
 
data add_vill_&p._&f._04;
     set add_vill_&p._&f._02 
         add_vill_&p._&f._03;
run;


****************************************************************************
** UNPAID: Create Village Aggregate Measures for Help from Beyond Village **
****************************************************************************;

%let p=unpaid;    *** saves keystrokes ***;

data work&f.14_unpaid (drop=PAIDHH&y);
     set work&f.13;
     if PAIDHH&y=2;
run;

** SPLIT DATA INTO 3 GROUPS - SAME VILL, OTH VILL, ALL OTHER **;

data work&f.15_&p._othvill (drop=LOCATION);   ** 1 **;
     set work&f.14_&p;
     if LOCATION in (4);
     if HELPHH&y ne .;
run;

data work&f.15_&p._vill (drop=LOCATION);      ** 2 **;
     set work&f.14_&p;
     if LOCATION in (0,1);
     if HELPHH&y ne .;
run;

data work&f.15_&p._other (drop=LOCATION);     ** 3 **;
     set work&f.14_&p;
     if (LOCATION in (2,3,5,6,7,8,9) OR HELPHH&y=.);
run;

****************************************************
** (1) OTHVILL: PROCESS LABOR FROM OTHER VILLAGES **
****************************************************;

** Collapse Labor from the same 84 village **;

data work&f.16_&p._othvill (keep=HHID&y V84N TOT_WRKS TOT_WAGE);
     set work&f.15_&p._othvill;

     retain TOT_WRKS TOT_WAGE;

     by HHID&y V84N;

     if first.V84N then do;
                          TOT_WRKS=0;
                          TOT_WAGE=0;
                        end;
     if NUMWRKS ne . then TOT_WRKS=TOT_WRKS+NUMWRKS;
     if ALLWAGE ne . then TOT_WAGE=TOT_WAGE+ALLWAGE;

    if last.V84N then output;
 
run;

** Merge with all rice-growing households **;

data work&f.add_all_01 (drop=HHID&y.C); 
     set vill_id_&f.02 (rename=(HHID&y=HHID&y.C));
     HHID&y=input(HHID&y.C,best12.);
run;

proc sort data=work&f.add_all_01 out=work&f.add_all_02;
     by HHID&y;
run;

data work&f.17_&p._othvill;
     merge work&f.16_&p._othvill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;


data work&f.18_&p._othvill (drop=V84N TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._othvill;

     retain UID_H01-UID_H10 UWRK_H01-UWRK_H10 UWAG_H01-UWAG_H10 i;

     length UID_H01-UID_H10 8;             * 10 for good measure *;
     length UWRK_H01-UWRK_H10 8;             * 10 for good measure *;
     length UWAG_H01-UWAG_H10 8;             * 10 for good measure *;

     array id(1:10) UID_H01-UID_H10;
     array wk(1:10) UWRK_H01-UWRK_H10;
     array wg(1:10) UWAG_H01-UWAG_H10;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 10;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=V84N;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
 
run;

***********************************************
** (2) VILL: PROCESS LABOR FROM SAME VILLAGE **
***********************************************;

** Placeholder for consistency **;

data work&f.16_&p._vill (keep=HHID&y HELPHH&y TOT_WRKS TOT_WAGE);
     set work&f.15_&p._vill;

     TOT_WRKS=NUMWRKS;
     TOT_WAGE=ALLWAGE;
RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._vill;
     merge work&f.16_&p._vill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;

data work&f.18_&p._vill (drop=HELPHH&y TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._vill;

     retain UID_H11-UID_H60 UWRK_H11-UWRK_H60 UWAG_H11-UWAG_H60 i;    ** Starts at 11 to leave room for village **; 

     length UID_H11-UID_H60 8;                 * 50 for good measure *;
     length UWRK_H11-UWRK_H60 8;             * 50 for good measure *;
     length UWAG_H11-UWAG_H60 8;             * 50 for good measure *;

     array id(1:50) UID_H11-UID_H60;
     array wk(1:50) UWRK_H11-UWRK_H60;
     array wg(1:50) UWAG_H11-UWAG_H60;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 50;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=HELPHH&y;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
run;

****************************************
** (3) OTHER: PROCESS ALL OTHER LABOR **
****************************************;

** This labor is aggregated and included in an attribute file **;

** Placeholder for consistency **;

data work&f.16_&p._other (keep=HHID&y HELPHH&y NUMWRKS ALLWAGE);
     set work&f.15_&p._other;

RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._other;
     merge work&f.16_&p._other (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;
** THIS FILE TO BE MERGED BELOW WITH OTHER ATTRIBUTES **;

data work&f.18_&p._other (drop=HELPHH&y NUMWRKS ALLWAGE);
     set work&f.17_&p._other;

     retain OTH_UWRK OTH_UWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            OTH_UWRK=0;
                            OTH_UWAG=0;
                          end;

     if NUMWRKS ne . then OTH_UWRK=OTH_UWRK+NUMWRKS;
     if ALLWAGE ne . then OTH_UWAG=OTH_UWAG+ALLWAGE;

    if last.HHID&y then output;
run;                                                             


***********************************
** Add other attribute variables ** 
***********************************;

** HH-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_hh (drop=HHID&y.C);
     set in&f.2.hh&y (keep=HHID&y RICE rename=(HHID&y=HHID&y.C));

     if RICE in (.,2,8) then RICE=0;

     IS_HH=1;

     HHID&y=input(HHID&y.C,best12.);

run;

** Merge with all rice-growing households **;

data work&f._&p._add_all_hh (drop=V84N LOCATION HELPHH&y);
     merge work&f.14_&p (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
                          
     if b=1 then output;
run;

data work&f._&p._add_tot_hh (drop=NUMWRKS ALLWAGE); **Generate Count Vars for ALL labor **;
     set work&f._&p._add_all_hh;

     retain ALL_UWRK ALL_UWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            ALL_UWRK=0;
                            ALL_UWAG=0;
                          end;

     if NUMWRKS ne . then ALL_UWRK=ALL_UWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_UWAG=ALL_UWAG+ALLWAGE;

    if last.HHID&y then output;
run;                           

** MERGE **;

data work&f.19_&p._all_hh;
     merge work&f._&p._add_rice_hh (in=a)
           work&f._&p._add_tot_hh (in=b)
           work&f.18_&p._other (in=c);
     by HHID&y;
     if c=1 then output;
run;

data work&f.19_&p._all_hh_2 (drop=V84);
     set work&f.19_&p._all_hh;
     V84N=input(V84,2.0);
run;

** VILLAGE-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_vill (drop=V84 HHID&y);
     set work&f.add_all_02;
     RICE=1;
     IS_HH=0;
     OTH_UWRK=0;
     OTH_UWAG=0;
     V84N=input(V84,2.0);
run;

** Prep for merge **;

proc sort data=work&f._&p._add_rice_vill out=work&f._&p._add_rice_vill_2 nodupkey;
     by V84N;
run;

proc sort data=work&f.14_&p out=work&f.14_&p._by_v84;
     by V84N;
run;

**Generate Count Vars for ALL labor **;

data work&f._&p._add_tot_vill (drop=HHID&y HELPHH&y NUMWRKS ALLWAGE); 
     set work&f.14_&p._by_v84;

     retain ALL_UWRK ALL_UWAG;
 
     by V84N;

     if first.V84N then do;
                            ALL_UWRK=0;
                            ALL_UWAG=0;
                          end;

     if NUMWRKS ne . then ALL_UWRK=ALL_UWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_UWAG=ALL_UWAG+ALLWAGE;

     if last.V84N then output;
run;                           

data work&f.19_&p._all_vill;
     merge work&f._&p._add_rice_vill_2 (in=a)
           work&f._&p._add_tot_vill (in=b);
     by V84N;

     HHID&y=V84N;
     if b=1 then output;
run;

***************************************
** Merge HH and VILL Attribute Files **
***************************************;

data work&f.19_&p._attrib;
     set work&f.19_&p._all_vill (drop=LOCATION)
         work&f.19_&p._all_hh_2;
run;

**********************************************
** Merge Same and Other Village Labor Files **
**********************************************;

data work&f.19_&p._adjlist;
     merge work&f.18_&p._othvill (in=b)
           work&f.18_&p._vill (in=a);
     by HHID&y;

     if a=1 and b=1 then output;
run;

***********************************************************
** Add Villages to adjacency matrix - all missing values **
***********************************************************;

proc sort data=vill_id_&f.02 out=add_vill_&p._&f._01 nodupkey;
     by V84;
run;

data add_vill_&p._&f._02 (drop=V84 j);
     set add_vill_&p._&f._01 (drop=HHID&y);

     length UID_H01-UID_H60 8;                 
     length UWRK_H01-UWRK_H60 8;             
     length UWAG_H01-UWAG_H60 8;             

     array id(1:60) UID_H01-UID_H60;
     array wk(1:60) UWRK_H01-UWRK_H60;
     array wg(1:60) UWAG_H01-UWAG_H60;
 
     do j= 1 to 60;
        id(j)=.;
        wk(j)=.;
        wg(j)=.;
     end;
 
     HHID&y=input(V84,best12.);    
     V84N=input(V84,2.);    

run;   

data add_vill_&p._&f._03 (drop=V84);
     set work&f.19_&p._adjlist;
     V84N=input(V84,2.0);
run;
 
data add_vill_&p._&f._04;
     set add_vill_&p._&f._02 
         add_vill_&p._&f._03;
run;


*******************************************************************************************************************;

*********************************************************************
** Temporarily Merge Together All 4 Final Files to purge ID errors **
*********************************************************************;

proc sort data=add_vill_paid_&f._04 out=work&f.20_paid_adj;
     by HHID&y V84N;
run;

proc sort data=add_vill_unpaid_&f._04 out=work&f.20_unpaid_adj;
     by HHID&y V84N;
run;

proc sort data=work&f.19_paid_attrib out=work&f.20_paid_att;
     by HHID&y V84N;
run;

proc sort data=work&f.19_unpaid_attrib out=work&f.20_unpaid_att;
     by HHID&y V84N;
run;


data work&f.21_all_files;
     merge work&f.20_paid_adj (in=a)
           work&f.20_unpaid_adj (in=b)
           work&f.20_paid_att (in=c)
           work&f.20_unpaid_att (drop=RICE IS_HH in=d);
     by HHID&y V84N;
     if a=1 and b=1 and c=1 and d=1 then output;
run;  

*******************************************
** Format files to be exported to UCINET **
*******************************************;

** I have decided to revert these matrices back to hh-only - 2011 02 07 **;
** These changes are reflected only in the four data steps below **;
** The village-level information still exists in all prior data steps **;
** In the future, I'd like to add a village-level only analysis **;

** I have also re-combined all files into a single one as of 2011 02 09 **;

data r&y._all (drop=PID_H01-PID_H10 PWRK_H01-PWRK_H10 PWAG_H01-PWAG_H10
                    UID_H01-UID_H10 UWRK_H01-UWRK_H10 UWAG_H01-UWAG_H10);
     set work&f.21_all_files;

     ALL_WRKS=ALL_PWRK+ALL_UWRK;
     ALL_WAGE=ALL_PWAG+ALL_UWAG;

     if HHID&y>100;               * Remove village datalines *;
run;

**************************************************************************************************;

************************************************
** Create separate village files EACH VILLAGE **
************************************************;

%macro v_split (numvill=);  %* macro splits villages *;

       %* NUMVILL=Number of Unique Villages in file *;

%do i=1 %to &numvill;

    data r00_p&i (drop=V84N);
         set r&y._all;
         if V84N=&i;
    run;

%end;

%mend v_split;

%v_split (numvill=51);

************************************************************************************
* 7 ** VILLAGE NETWORKS: RICE PAID ** Create VALUED adjacency matrices: TOT WAGES **
************************************************************************************;

%macro v_adj1 (numvill=);
%do i=1 %to &numvill;
proc iml;
     %include '/home/jhull/public/span/adjval.mod';
     %include '/home/jhull/public/span/pajwrite.mod';
     %let p1=%quote(/home/jhull/nangrong/data_paj/2000/a3-net/v0);
     %let p2=%quote(00a3.net);
     use r00_p&i;
     read all var{
                  PID_H11 PID_H12 PID_H13 PID_H14 PID_H15
                  PID_H16 PID_H17 PID_H18 PID_H19 PID_H20
                  PID_H21 PID_H22 PID_H23 PID_H24 PID_H25
                  PID_H26 PID_H27 PID_H28 PID_H29 PID_H30
                  PID_H31 PID_H32 PID_H33 PID_H34 PID_H35
                  PID_H36 PID_H37 PID_H38 PID_H39 PID_H40
                  PID_H41 PID_H42 PID_H43 PID_H44 PID_H45
                  PID_H46 PID_H47 PID_H48 PID_H49 PID_H50
                  PID_H51 PID_H52 PID_H53 PID_H54 PID_H55
                  PID_H56 PID_H57 PID_H58 PID_H59 PID_H60} into rcv;
     read all var{
                  PWAG_H11 PWAG_H12 PWAG_H13 PWAG_H14 PWAG_H15
                  PWAG_H16 PWAG_H17 PWAG_H18 PWAG_H19 PWAG_H20
                  PWAG_H21 PWAG_H22 PWAG_H23 PWAG_H24 PWAG_H25
                  PWAG_H26 PWAG_H27 PWAG_H28 PWAG_H29 PWAG_H30
                  PWAG_H31 PWAG_H32 PWAG_H33 PWAG_H34 PWAG_H35
                  PWAG_H36 PWAG_H37 PWAG_H38 PWAG_H39 PWAG_H40
                  PWAG_H41 PWAG_H42 PWAG_H43 PWAG_H44 PWAG_H45
                  PWAG_H46 PWAG_H47 PWAG_H48 PWAG_H49 PWAG_H50
                  PWAG_H51 PWAG_H52 PWAG_H53 PWAG_H54 PWAG_H55
                  PWAG_H56 PWAG_H57 PWAG_H58 PWAG_H59 PWAG_H60} into val;
     read all var{HHID00} into snd;
     r00_w=adjval(snd,rcv,val);
     id=r00_w[,1];
     r00_w=r00_w[,2:ncol(r00_w)];
     adj=r00_w;
     file "&p1.&i.&p2";
     call pajwrite(adj,id,2);
     create adj0&i from adj;
            append from adj;
     idinv=id`;
     create id0&i from idinv;
            append from idinv;
quit;
%end;
%mend v_adj1;

%v_adj1(numvill=9);

%macro v_adj2 (numvill=);
%do i=10 %to &numvill;
proc iml;
     %include '/home/jhull/public/span/adjval.mod';
     %include '/home/jhull/public/span/pajwrite.mod';
     %let p1=%quote(/home/jhull/nangrong/data_paj/2000/a3-net/v);
     %let p2=%quote(00a3.net);
     use r00_p&i;
     read all var{
                  PID_H11 PID_H12 PID_H13 PID_H14 PID_H15
                  PID_H16 PID_H17 PID_H18 PID_H19 PID_H20
                  PID_H21 PID_H22 PID_H23 PID_H24 PID_H25
                  PID_H26 PID_H27 PID_H28 PID_H29 PID_H30
                  PID_H31 PID_H32 PID_H33 PID_H34 PID_H35
                  PID_H36 PID_H37 PID_H38 PID_H39 PID_H40
                  PID_H41 PID_H42 PID_H43 PID_H44 PID_H45
                  PID_H46 PID_H47 PID_H48 PID_H49 PID_H50
                  PID_H51 PID_H52 PID_H53 PID_H54 PID_H55
                  PID_H56 PID_H57 PID_H58 PID_H59 PID_H60} into rcv;
     read all var{
                  PWAG_H11 PWAG_H12 PWAG_H13 PWAG_H14 PWAG_H15
                  PWAG_H16 PWAG_H17 PWAG_H18 PWAG_H19 PWAG_H20
                  PWAG_H21 PWAG_H22 PWAG_H23 PWAG_H24 PWAG_H25
                  PWAG_H26 PWAG_H27 PWAG_H28 PWAG_H29 PWAG_H30
                  PWAG_H31 PWAG_H32 PWAG_H33 PWAG_H34 PWAG_H35
                  PWAG_H36 PWAG_H37 PWAG_H38 PWAG_H39 PWAG_H40
                  PWAG_H41 PWAG_H42 PWAG_H43 PWAG_H44 PWAG_H45
                  PWAG_H46 PWAG_H47 PWAG_H48 PWAG_H49 PWAG_H50
                  PWAG_H51 PWAG_H52 PWAG_H53 PWAG_H54 PWAG_H55
                  PWAG_H56 PWAG_H57 PWAG_H58 PWAG_H59 PWAG_H60} into val;
     read all var{HHID00} into snd;
     r00_w=adjval(snd,rcv,val);
     id=r00_w[,1];
     r00_w=r00_w[,2:ncol(r00_w)];
     adj=r00_w;
     file "&p1.&i.&p2";
     call pajwrite(adj,id,2);
     create adj&i from adj;
            append from adj;
     idinv=id`;
     create id&i from idinv;
            append from idinv;
quit;
%end;
%mend v_adj2;

%v_adj2(numvill=51);

%macro v_adj3(numvill=);
%do i=1 %to &numvill;
    data _null_ ;
         %let p1=%quote(/home/jhull/nangrong/data_paj/2000/a3-adj/v0);
         %let p2=%quote(00a3.adj);
         set adj0&i;
         file "&p1.&i.&p2"  lrecl=1000;
         put (_ALL_) (+0);
run;
    data _null_ ;
         %let p3=%quote(/home/jhull/nangrong/data_paj/2000/a3-id/v0);
         %let p4=%quote(00a3.id);
         set id0&i;
         file "&p3.&i.&p4"  lrecl=10000;
         put (_ALL_) (+0);
run;

%end;
%mend v_adj3;

%v_adj3(numvill=9);

%macro v_adj4(numvill=);
%do i=10 %to &numvill;  
    data _null_ ;
         %let p1=%quote(/home/jhull/nangrong/data_paj/2000/a3-adj/v);
         %let p2=%quote(00a3.adj);
         set adj&i;
         file "&p1.&i.&p2"  lrecl=1000;
         put (_ALL_) (+0);
run;
    data _null_ ;
         %let p3=%quote(/home/jhull/nangrong/data_paj/2000/a3-id/v);
         %let p4=%quote(00a3.id);
         set id&i;
         file "&p3.&i.&p4"  lrecl=10000;
         put (_ALL_) (+0);
run;
%end;
%mend v_adj4;

%v_adj4(numvill=51);

In [ ]:
********************************************************************
**     Program Name: /home/jhull/nangrong/prog_sas/p02_nets/2000/p02_00a4.sas
**     Programmer: james r. hull
**     Start Date: 2011 03 09
**     Purpose:
**        1.) Create A4 = UNPAID LABOR NETWORKS TOTAL FLOW IN BAHT
**
**     Input Data:
**        '/home/jhull/nangrong/data_sas/2000/current/indiv00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/hh00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/comm00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/sibs00.xpt'
**
**     Output Data:
**        1.) /home/jhull/nangrong/data_paj/2000/a4/vXX00a4.net
**
**     Notes: 
**        1.) 
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=60;

%let f=00a4;  ** names file (allows portability) **;
%let y=00; 

**********************
**  Data Libraries  **
**********************;

libname in&f.1 xport '/home/jhull/nangrong/data_sas/2000/current/indiv00.xpt';
libname in&f.2 xport '/home/jhull/nangrong/data_sas/2000/current/hh00.xpt';
libname in&f.3 xport '/home/jhull/nangrong/data_sas/2000/current/comm00.xpt';
libname in&f.4 xport '/home/jhull/nangrong/data_sas/2000/current/sibs00.xpt';


******************************
**  Create Working Dataset  **
******************************;

********************************************************************
**  Length 10
**  0 In this 1984 village                2 + Village # + House #
**  2 In this village but blt             2 + Village # + 999
**  Length 7 
**  4 Another village in Nang Rong        3 + Village #
**  8 Unknown village in Nang Rong        3 + 999999
**  5 Another district in Buriram         4 + District # + 0000
**  6 Another province                    5 + District # + 0000
**  7 Another country 
**  8 Missing/Don't know                  9999999999 or 9999999
**  Length 9
**  9 Code 2 or 3 Returning HH member
*******************************************************************;

***************************************************************************
** Stack rice harvest labor data into a child file and label by location **
***************************************************************************;

data work&f.01a;
     set in&f.2.hh00 (keep=HHID00 X6_84C: X6_84W:);
     keep HHID00 X6_86L X6_86N X6_86W LOCATION;

     length X6_86L $ 10;

     array a(1:7) X6_84C1-X6_84C7;
     array b(1:7) X6_84W1-X6_84W7;

     do i=1 to 7;
          X6_86L=a(i);
          X6_86N=1;
          X6_86W=b(i);
          LOCATION=9;
          if a(i) ne " ." then output;  * Keep only those cases with data *;
     end;
run;

data work&f.01b;
     set in&f.2.hh00 (keep=HHID00 X6_85H: X6_85N: X6_85W:);
     keep HHID00 X6_86L X6_86N X6_86W LOCATION;

     length X6_86L $ 10;

     array a(1:13) X6_85H1-X6_85H13;
     array b(1:13) X6_85N1-X6_85N13;
     array c(1:13) X6_85W1-X6_85W13;

     do i=1 to 13;
 
          ** FIX ANOTHER CRAZY ERROR IN ORIGINAL DATA WITH REVERSED CODING FOR IN-VILLAGE LABOR **;
         
          if substr(a(i),5,3) in ('020','021','170','180') then a(i)=cat("2",substr(a(i),5,6),substr(a(i),2,3));

          if a(i)="9999999999" then LOCATION=8;
             else if substr(a(i),8,3)='999' then LOCATION=2;
             else LOCATION=0;

          X6_86L=a(i);
          X6_86N=b(i);
          X6_86W=c(i);

          if a(i) ne "         ." then output;  * Keep only those cases with data *;
     end;
run;

data work&f.01c;
     set in&f.2.hh00 (keep=HHID00 X6_86L: X6_86N: X6_86W:);
     keep HHID00 X6_86L X6_86N X6_86W LOCATION;

     length X6_86L $ 10;

     array a(1:10) X6_86L1-X6_86L10;
     array b(1:10) X6_86N1-X6_86N10;
     array c(1:10) X6_86W1-X6_86W10;

     do i=1 to 10;
          X6_86L=a(i);
          X6_86N=b(i);
          X6_86W=c(i);
          if a(i)='9999999' then LOCATION=8;
             else if substr(a(i),1,1)='5' then LOCATION=6;
             else if substr(a(i),1,1)='4' then LOCATION=5;
             else if substr(a(i),1,2)='39' then LOCATION=8;
             else LOCATION=4;
          if a(i) ne "      ." then output;  * Keep only those cases with data *;
     end;

run;

*********************************************************
** Take Care of Missing Data Issues - Recodes at least **
*********************************************************;

data work&f.02a;
     set work&f.01a;

     if X6_86W=9 then X6_86W=.;
     if X6_86N=99 then X6_86N=1; * Assume at least 1 person worked *;

run;

****************************************************************************
** This code collapses multiple code 2 & 3 workers from a household to    **
** a single observation and sums the values for each into summary counts. **
** For the "type of labor" variable, I use an "any paid --> paid" rule    **
** because paying a code 2 or 3 laborer is a rare behavior and distinct   **
****************************************************************************;

 data work&f.02a2;
     set work&f.02a;

     by HHID00;

     retain SUM_N SUM_TYPE SUM_LOC i;


     if first.HHID00 then do;
                            SUM_N=0;
                            SUM_TYPE=2; * Default is unpaid labor*;
                            SUM_LOC=9;
                            i=1;
                         end;

     SUM_N=SUM_N+X6_86N;
     if X6_86W=1 then SUM_TYPE=1;  * Any paid --> all paid *;
     SUM_LOC=9;
     i=i+1;

     if last.HHID00 then output;

run;

data work&f.02a3 (drop=SUM_N SUM_TYPE SUM_LOC i);
     set work&f.02a2;

     X6_86L="   ";
     X6_86N=SUM_N;
     X6_86W=SUM_TYPE;
     LOCATION=SUM_LOC;

run;

*********************************************************
** Take Care of Missing Data Issues - Recodes at least **
*********************************************************;

data work&f.02b;
     set work&f.01b;

     if X6_86W=9 then X6_86W=.;
     if X6_86N=99 then X6_86N=1; * Assume at least 1 person worked *;
run;

data work&f.02c;
     set work&f.01c;

     if X6_86W=9 then X6_86W=.;
     if X6_86N=99 then X6_86N=1; * Assume at least 1 person worked *;
run;

**************************
** Merge files together **
**************************;

data work&f.03;
     set work&f.02a3
         work&f.02b
         work&f.02c;
run;

***************************************************************************
** Add V84 identifiers to 2000 data file as per Rick's suggestion on web **
***************************************************************************;



proc sort data=work&f.03 out=work&f.04;
     by HHID00 X6_86L LOCATION;
run;

data vill_id_&f.01;
     set in&f.1.indiv00;
     keep HHID00 V84;
run;

proc sort data=vill_id_&f.01 out=vill_id_&f.02 nodupkey;
     by HHID00 v84;
run;

data vill_id_&f.03;
     merge work&f.04 (in=a)
           vill_id_&f.02 (in=b);
           if a=1 and b=1 then output;
     by HHID00;
run;

proc sort data=vill_id_&f.03 out=work&f.05;
     by V84;
run;

******************************************************************************
** This step removes all cases about which there is no information about    **
** how their laborers were compensated. This is my fix for the time being.  **
** Note: in doing so, I lose 7 cases (a case here is a helper group)        **
******************************************************************************;

data work&f.06;
     set work&f.05;

     rename X6_86L=HELPHHID;

     if X6_86W ^in (.,9) then output;

run;

************************************************************************************
** The steps below convert the ban lek ti information on the helping household    **
** into the standard HHID##, as a preparatory step to creating network datafiles. **
************************************************************************************;

data work&f.07;
     set work&f.06;

     if LOCATION=9 then do;
                           HELPHHID=HHID00;
                        end;
        else if LOCATION in(0,4) then do;
                                  HELPHHID=substr(HELPHHID,2,9);
                                end;
        else if LOCATION in (2) then do;
                                    HELPHHID=substr(HELPHHID,2,6);
                                end;
        else HELPHHID=".";
run;

*********************************
** Clean HELPHH00 of BAD CODES **
*********************************; 

data work&f.08a (drop=HHID&y.C);
     set work&f.07 (rename=(HHID&y=HHID&y.C));
     HELPHH00=input(HELPHHID,9.0);
     V84N=input(V84,2.0);
     HHID00=input(HHID&y.C,9.);
run;

data work&f.08b (drop=HHID00 V84);
     set in&f.1.indiv00 (keep=HHID00 V84);
     HELPHH00=input(HHID00,9.0);
     V84N=input(V84,2.0);
run;

proc sort data=work&f.08a out=work&f.09a;
     by V84N HELPHH00;
run;

proc sort data=work&f.08b out=work&f.09b nodupkey;
     by V84N HELPHH00;
run;

data work&f.10 aonly bonly;
     merge work&f.09a (in=a)
           work&f.09b (in=b);
     by V84N HELPHH00;
     if a=1 and b=1 then output work&f.10;
     if a=1 and b=0 then output aonly;
     if a=0 and b=1 then output bonly;
run;


** This step removes the HELPHH00 codes from a number of HHs **;
** that had non-existent or incorrect HHs listed as helping  **;

data aonly_fix;
     set aonly;
     if HELPHH00>999999 then do;     
                               HELPHH00=".";
                               HELPHHID=".";
                             end;
run;
 
data work&f.11;
     set work&f.10 (in=a)
         aonly_fix (in=b);
run;

************************************************************************
**input average village wages during high demand from community survey**
************************************************************************;

data work&f.vill_wages_01a (drop=X45MHIGH X45MTYP);  
    set in&f.3.comm&y (keep=VILL00 X45MHIGH X45MTYP);

    if X45MHIGH=9999998 then X45MHIGH=.; ** USING ONLY MALES B/C MALE AND FEMALE WAGES IDENTICAL **;
    if X45MTYP=9999998 then X45MTYP=.;

    if X45MHIGH=. then RICEWAGH=125.77;
       else RICEWAGH=X45MHIGH;
    if X45MTYP=. then RICEWAGN=105.29;
       else RICEWAGN=X45MTYP;
run;

proc sort data=work&f.vill_wages_01a out=work&f.vill_wages_01b nodupkey;
     by VILL&y;
run;

proc sort data=in&f.1.indiv&y  out=work&f.vill_wages_02a nodupkey;
     by HHID&y;
run;

data work&f.make_vill;
     set work&f.vill_wages_02a (keep=HHID&y);
     length VILL&y $ 6;

     VILL&y=substr(HHID&y,1,6);
run;

data work&f.numeric (drop=HHID&y.C);
     set work&f.make_vill (rename=(HHID&y=HHID&y.C));

     HHID&y=input(HHID&y.C,best9.);
run;

proc sort data=work&f.numeric out=work&f.vill_wages_02b;
     by VILL&y HHID&y;
run;

data work&f.vill_wages_03;
     merge work&f.vill_wages_01b (in=a)
           work&f.vill_wages_02b (in=b);
     by VILL&y;
     if b=1 then output;
run;

proc sort data=work&f.vill_wages_03 out=work&f.vill_wages_04;
     by HHID&y;
run;

proc sort data=work&f.11 out=work&f.vill_wages_05;
     by HHID&y;
run;

data work&f.vill_wages_06;
     merge work&f.vill_wages_04 (in=a)
           work&f.vill_wages_05 (in=b);
     by HHID&y;
     if b=1 then output;
run;

proc sort data=work&f.vill_wages_06 out=work&f.11B;
     by HHID&y;
run;

data work&f.11C;
     set work&f.11B;

     if X6_86W=3 then PAIDHH00=2;
        else PAIDHH00=X6_86W;

     if X6_86W=1 then ALLWAGE=round(X6_86N*RICEWAGH,.01);   ** ACTUAL DAYS WORKED*AVG WAGES IN VILL**;
        else ALLWAGE=0;                                             

     NUMWRKS=X6_86N;
  
run;

data work&f.12 (keep=HHID00 PAIDHH00 HELPHH00 LOCATION ALLWAGE NUMWRKS V84N);
     set work&f.11C;
run;

proc sort data=work&f.12 out=work&f.13;              ** MAJOR CHECKPOINT - TOTAL LABOR CHILD FILE **;
     by HHID&y HELPHH&y;
run;

*******************************************************************************************************************************;

***********************************************
** Separate Paid and Unpaid Labor into Files **
***********************************************;

** Note, all code below must be repeated for paid and then unpaid labor **;

**************************************************************************
** PAID: Create Village Aggregate Measures for Help from Beyond Village **
**************************************************************************;

%let p=paid;    *** saves keystrokes ***;

data work&f.14_&p (drop=PAIDHH&y);
     set work&f.13;
     if PAIDHH&y=1;
run;

** SPLIT DATA INTO 3 GROUPS - SAME VILL, OTH VILL, ALL OTHER **;

data work&f.15_&p._othvill (drop=LOCATION);   ** 1 **;
     set work&f.14_&p;
     if LOCATION in (4);
     if HELPHH&y ne .;
run;

data work&f.15_&p._vill (drop=LOCATION);      ** 2 **;
     set work&f.14_&p;
     if LOCATION in (0,1);
     if HELPHH&y ne .;
run;

data work&f.15_&p._other (drop=LOCATION);     ** 3 **;
     set work&f.14_&p;
     if (LOCATION in (2,3,5,6,7,8,9) OR HELPHH&y=.);
run;

****************************************************
** (1) OTHVILL: PROCESS LABOR FROM OTHER VILLAGES **
****************************************************;

** Collapse Labor from the same 84 village **;

data work&f.16_&p._othvill (keep=HHID&y V84N TOT_WRKS TOT_WAGE);
     set work&f.15_&p._othvill;

     retain TOT_WRKS TOT_WAGE;

     by HHID&y V84N;

     if first.V84N then do;
                          TOT_WRKS=0;
                          TOT_WAGE=0;
                        end;
     if NUMWRKS ne . then TOT_WRKS=TOT_WRKS+NUMWRKS;
     if ALLWAGE ne . then TOT_WAGE=TOT_WAGE+ALLWAGE;

    if last.V84N then output;
 
run;

** Merge with all rice-growing households **;

data work&f.add_all_01 (drop=HHID&y.C); 
     set vill_id_&f.02 (rename=(HHID&y=HHID&y.C));
     HHID&y=input(HHID&y.C,best12.);
run;

proc sort data=work&f.add_all_01 out=work&f.add_all_02;
     by HHID&y;
run;

data work&f.17_&p._othvill;
     merge work&f.16_&p._othvill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;

data work&f.18_&p._othvill (drop=V84N TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._othvill;

     retain PID_H01-PID_H10 PWRK_H01-PWRK_H10 PWAG_H01-PWAG_H10 i;

     length PID_H01-PID_H10 8;             * 10 for good measure *;
     length PWRK_H01-PWRK_H10 8;             * 10 for good measure *;
     length PWAG_H01-PWAG_H10 8;             * 10 for good measure *;

     array id(1:10) PID_H01-PID_H10;
     array wk(1:10) PWRK_H01-PWRK_H10;
     array wg(1:10) PWAG_H01-PWAG_H10;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 10;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=V84N;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
 
run;

***********************************************
** (2) VILL: PROCESS LABOR FROM SAME VILLAGE **
***********************************************;

** Placeholder for consistency **;

data work&f.16_&p._vill (keep=HHID&y HELPHH&y TOT_WRKS TOT_WAGE);
     set work&f.15_&p._vill;

     TOT_WRKS=NUMWRKS;
     TOT_WAGE=ALLWAGE;
RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._vill;
     merge work&f.16_&p._vill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;

data work&f.18_&p._vill (drop=HELPHH&y TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._vill;

     retain PID_H11-PID_H60 PWRK_H11-PWRK_H60 PWAG_H11-PWAG_H60 i;    ** Starts at 11 to leave room for village **; 

     length PID_H11-PID_H60 8;                 * 50 for good measure *;
     length PWRK_H11-PWRK_H60 8;             * 50 for good measure *;
     length PWAG_H11-PWAG_H60 8;             * 50 for good measure *;

     array id(1:50) PID_H11-PID_H60;
     array wk(1:50) PWRK_H11-PWRK_H60;
     array wg(1:50) PWAG_H11-PWAG_H60;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 50;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=HELPHH&y;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
run;

****************************************
** (3) OTHER: PROCESS ALL OTHER LABOR **
****************************************;

** This labor is aggregated and included in an attribute file **;

** Placeholder for consistency **;

data work&f.16_&p._other (keep=HHID&y HELPHH&y NUMWRKS ALLWAGE);
     set work&f.15_&p._other;

RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._other;
     merge work&f.16_&p._other (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;
** THIS FILE TO BE MERGED BELOW WITH OTHER ATTRIBUTES **;

data work&f.18_&p._other (drop=HELPHH&y NUMWRKS ALLWAGE);
     set work&f.17_&p._other;

     retain OTH_PWRK OTH_PWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            OTH_PWRK=0;
                            OTH_PWAG=0;
                          end;

     if NUMWRKS ne . then OTH_PWRK=OTH_PWRK+NUMWRKS;
     if ALLWAGE ne . then OTH_PWAG=OTH_PWAG+ALLWAGE;

    if last.HHID&y then output;
run;                                                             


***********************************
** Add other attribute variables ** 
***********************************;

** HH-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_hh (drop=HHID&y.C);
     set in&f.2.hh&y (keep=HHID&y RICE rename=(HHID&y=HHID&y.C));

     if RICE in (.,2,8) then RICE=0;

     IS_HH=1;

     HHID&y=input(HHID&y.C,best12.);
    
run;

** Merge with all rice-growing households **;

data work&f._&p._add_all_hh (drop=V84N LOCATION HELPHH&y);
     merge work&f.14_&p (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
                          
     if b=1 then output;
run;

data work&f._&p._add_tot_hh (drop=NUMWRKS ALLWAGE); **Generate Count Vars for ALL labor **;
     set work&f._&p._add_all_hh;

     retain ALL_PWRK ALL_PWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            ALL_PWRK=0;
                            ALL_PWAG=0;
                          end;

     if NUMWRKS ne . then ALL_PWRK=ALL_PWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_PWAG=ALL_PWAG+ALLWAGE;

    if last.HHID&y then output;
run;                           

** MERGE **;

data work&f.19_&p._all_hh;
     merge work&f._&p._add_rice_hh (in=a)
           work&f._&p._add_tot_hh (in=b)
           work&f.18_&p._other (in=c);
     by HHID&y;
     if c=1 then output;
run;

data work&f.19_&p._all_hh_2 (drop=V84);
     set work&f.19_&p._all_hh;
     V84N=input(V84,2.0);
run;

** VILLAGE-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_vill (drop=V84 HHID&y);
     set work&f.add_all_02;
     RICE=1;
     IS_HH=0;
     OTH_PWRK=0;
     OTH_PWAG=0;
     V84N=input(V84,2.0);
run;

** Prep for merge **;

proc sort data=work&f._&p._add_rice_vill out=work&f._&p._add_rice_vill_2 nodupkey;
     by V84N;
run;

proc sort data=work&f.14_&p out=work&f.14_&p._by_v84;
     by V84N;
run;

**Generate Count Vars for ALL labor **;

data work&f._&p._add_tot_vill (drop=HHID&y HELPHH&y NUMWRKS ALLWAGE); 
     set work&f.14_&p._by_v84;

     retain ALL_PWRK ALL_PWAG;
 
     by V84N;

     if first.V84N then do;
                            ALL_PWRK=0;
                            ALL_PWAG=0;
                          end;

     if NUMWRKS ne . then ALL_PWRK=ALL_PWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_PWAG=ALL_PWAG+ALLWAGE;

     if last.V84N then output;
run;                           

data work&f.19_&p._all_vill;
     merge work&f._&p._add_rice_vill_2 (in=a)
           work&f._&p._add_tot_vill (in=b);
     by V84N;

     HHID&y=V84N;
     if b=1 then output;
run;

***************************************
** Merge HH and VILL Attribute Files **
***************************************;

data work&f.19_&p._attrib;
     set work&f.19_&p._all_vill (drop=LOCATION)
         work&f.19_&p._all_hh_2;
run;

**********************************************
** Merge Same and Other Village Labor Files **
**********************************************;

data work&f.19_&p._adjlist;
     merge work&f.18_&p._othvill (in=b)
           work&f.18_&p._vill (in=a);
     by HHID&y;

     if a=1 and b=1 then output;
run;

***********************************************************
** Add Villages to adjacency matrix - all missing values **
***********************************************************;

proc sort data=vill_id_&f.02 out=add_vill_&p._&f._01 nodupkey;
     by V84;
run;

data add_vill_&p._&f._02 (drop=V84 j);
     set add_vill_&p._&f._01 (drop=HHID&y);

     length PID_H01-PID_H60 8;                 
     length PWRK_H01-PWRK_H60 8;             
     length PWAG_H01-PWAG_H60 8;             

     array id(1:60) PID_H01-PID_H60;
     array wk(1:60) PWRK_H01-PWRK_H60;
     array wg(1:60) PWAG_H01-PWAG_H60;
 
     do j= 1 to 60;
        id(j)=.;
        wk(j)=.;
        wg(j)=.;
     end;
 
     HHID&y=input(V84,best12.);    
     V84N=input(V84,2.);    

run;   

data add_vill_&p._&f._03 (drop=V84);
     set work&f.19_&p._adjlist;
     V84N=input(V84,2.0);
run;
 
data add_vill_&p._&f._04;
     set add_vill_&p._&f._02 
         add_vill_&p._&f._03;
run;


****************************************************************************
** UNPAID: Create Village Aggregate Measures for Help from Beyond Village **
****************************************************************************;

%let p=unpaid;    *** saves keystrokes ***;

data work&f.14_unpaid (drop=PAIDHH&y);
     set work&f.13;
     if PAIDHH&y=2;
run;

** SPLIT DATA INTO 3 GROUPS - SAME VILL, OTH VILL, ALL OTHER **;

data work&f.15_&p._othvill (drop=LOCATION);   ** 1 **;
     set work&f.14_&p;
     if LOCATION in (4);
     if HELPHH&y ne .;
run;

data work&f.15_&p._vill (drop=LOCATION);      ** 2 **;
     set work&f.14_&p;
     if LOCATION in (0,1);
     if HELPHH&y ne .;
run;

data work&f.15_&p._other (drop=LOCATION);     ** 3 **;
     set work&f.14_&p;
     if (LOCATION in (2,3,5,6,7,8,9) OR HELPHH&y=.);
run;

****************************************************
** (1) OTHVILL: PROCESS LABOR FROM OTHER VILLAGES **
****************************************************;

** Collapse Labor from the same 84 village **;

data work&f.16_&p._othvill (keep=HHID&y V84N TOT_WRKS TOT_WAGE);
     set work&f.15_&p._othvill;

     retain TOT_WRKS TOT_WAGE;

     by HHID&y V84N;

     if first.V84N then do;
                          TOT_WRKS=0;
                          TOT_WAGE=0;
                        end;
     if NUMWRKS ne . then TOT_WRKS=TOT_WRKS+NUMWRKS;
     if ALLWAGE ne . then TOT_WAGE=TOT_WAGE+ALLWAGE;

    if last.V84N then output;
 
run;

** Merge with all rice-growing households **;

data work&f.add_all_01 (drop=HHID&y.C); 
     set vill_id_&f.02 (rename=(HHID&y=HHID&y.C));
     HHID&y=input(HHID&y.C,best12.);
run;

proc sort data=work&f.add_all_01 out=work&f.add_all_02;
     by HHID&y;
run;

data work&f.17_&p._othvill;
     merge work&f.16_&p._othvill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;


data work&f.18_&p._othvill (drop=V84N TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._othvill;

     retain UID_H01-UID_H10 UWRK_H01-UWRK_H10 UWAG_H01-UWAG_H10 i;

     length UID_H01-UID_H10 8;             * 10 for good measure *;
     length UWRK_H01-UWRK_H10 8;             * 10 for good measure *;
     length UWAG_H01-UWAG_H10 8;             * 10 for good measure *;

     array id(1:10) UID_H01-UID_H10;
     array wk(1:10) UWRK_H01-UWRK_H10;
     array wg(1:10) UWAG_H01-UWAG_H10;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 10;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=V84N;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
 
run;

***********************************************
** (2) VILL: PROCESS LABOR FROM SAME VILLAGE **
***********************************************;

** Placeholder for consistency **;

data work&f.16_&p._vill (keep=HHID&y HELPHH&y TOT_WRKS TOT_WAGE);
     set work&f.15_&p._vill;

     TOT_WRKS=NUMWRKS;
     TOT_WAGE=ALLWAGE;
RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._vill;
     merge work&f.16_&p._vill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;

data work&f.18_&p._vill (drop=HELPHH&y TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._vill;

     retain UID_H11-UID_H60 UWRK_H11-UWRK_H60 UWAG_H11-UWAG_H60 i;    ** Starts at 11 to leave room for village **; 

     length UID_H11-UID_H60 8;                 * 50 for good measure *;
     length UWRK_H11-UWRK_H60 8;             * 50 for good measure *;
     length UWAG_H11-UWAG_H60 8;             * 50 for good measure *;

     array id(1:50) UID_H11-UID_H60;
     array wk(1:50) UWRK_H11-UWRK_H60;
     array wg(1:50) UWAG_H11-UWAG_H60;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 50;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=HELPHH&y;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
run;

****************************************
** (3) OTHER: PROCESS ALL OTHER LABOR **
****************************************;

** This labor is aggregated and included in an attribute file **;

** Placeholder for consistency **;

data work&f.16_&p._other (keep=HHID&y HELPHH&y NUMWRKS ALLWAGE);
     set work&f.15_&p._other;

RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._other;
     merge work&f.16_&p._other (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;
** THIS FILE TO BE MERGED BELOW WITH OTHER ATTRIBUTES **;

data work&f.18_&p._other (drop=HELPHH&y NUMWRKS ALLWAGE);
     set work&f.17_&p._other;

     retain OTH_UWRK OTH_UWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            OTH_UWRK=0;
                            OTH_UWAG=0;
                          end;

     if NUMWRKS ne . then OTH_UWRK=OTH_UWRK+NUMWRKS;
     if ALLWAGE ne . then OTH_UWAG=OTH_UWAG+ALLWAGE;

    if last.HHID&y then output;
run;                                                             


***********************************
** Add other attribute variables ** 
***********************************;

** HH-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_hh (drop=HHID&y.C);
     set in&f.2.hh&y (keep=HHID&y RICE rename=(HHID&y=HHID&y.C));

     if RICE in (.,2,8) then RICE=0;

     IS_HH=1;

     HHID&y=input(HHID&y.C,best12.);

run;

** Merge with all rice-growing households **;

data work&f._&p._add_all_hh (drop=V84N LOCATION HELPHH&y);
     merge work&f.14_&p (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
                          
     if b=1 then output;
run;

data work&f._&p._add_tot_hh (drop=NUMWRKS ALLWAGE); **Generate Count Vars for ALL labor **;
     set work&f._&p._add_all_hh;

     retain ALL_UWRK ALL_UWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            ALL_UWRK=0;
                            ALL_UWAG=0;
                          end;

     if NUMWRKS ne . then ALL_UWRK=ALL_UWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_UWAG=ALL_UWAG+ALLWAGE;

    if last.HHID&y then output;
run;                           

** MERGE **;

data work&f.19_&p._all_hh;
     merge work&f._&p._add_rice_hh (in=a)
           work&f._&p._add_tot_hh (in=b)
           work&f.18_&p._other (in=c);
     by HHID&y;
     if c=1 then output;
run;

data work&f.19_&p._all_hh_2 (drop=V84);
     set work&f.19_&p._all_hh;
     V84N=input(V84,2.0);
run;

** VILLAGE-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_vill (drop=V84 HHID&y);
     set work&f.add_all_02;
     RICE=1;
     IS_HH=0;
     OTH_UWRK=0;
     OTH_UWAG=0;
     V84N=input(V84,2.0);
run;

** Prep for merge **;

proc sort data=work&f._&p._add_rice_vill out=work&f._&p._add_rice_vill_2 nodupkey;
     by V84N;
run;

proc sort data=work&f.14_&p out=work&f.14_&p._by_v84;
     by V84N;
run;

**Generate Count Vars for ALL labor **;

data work&f._&p._add_tot_vill (drop=HHID&y HELPHH&y NUMWRKS ALLWAGE); 
     set work&f.14_&p._by_v84;

     retain ALL_UWRK ALL_UWAG;
 
     by V84N;

     if first.V84N then do;
                            ALL_UWRK=0;
                            ALL_UWAG=0;
                          end;

     if NUMWRKS ne . then ALL_UWRK=ALL_UWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_UWAG=ALL_UWAG+ALLWAGE;

     if last.V84N then output;
run;                           

data work&f.19_&p._all_vill;
     merge work&f._&p._add_rice_vill_2 (in=a)
           work&f._&p._add_tot_vill (in=b);
     by V84N;

     HHID&y=V84N;
     if b=1 then output;
run;

***************************************
** Merge HH and VILL Attribute Files **
***************************************;

data work&f.19_&p._attrib;
     set work&f.19_&p._all_vill (drop=LOCATION)
         work&f.19_&p._all_hh_2;
run;

**********************************************
** Merge Same and Other Village Labor Files **
**********************************************;

data work&f.19_&p._adjlist;
     merge work&f.18_&p._othvill (in=b)
           work&f.18_&p._vill (in=a);
     by HHID&y;

     if a=1 and b=1 then output;
run;

***********************************************************
** Add Villages to adjacency matrix - all missing values **
***********************************************************;

proc sort data=vill_id_&f.02 out=add_vill_&p._&f._01 nodupkey;
     by V84;
run;

data add_vill_&p._&f._02 (drop=V84 j);
     set add_vill_&p._&f._01 (drop=HHID&y);

     length UID_H01-UID_H60 8;                 
     length UWRK_H01-UWRK_H60 8;             
     length UWAG_H01-UWAG_H60 8;             

     array id(1:60) UID_H01-UID_H60;
     array wk(1:60) UWRK_H01-UWRK_H60;
     array wg(1:60) UWAG_H01-UWAG_H60;
 
     do j= 1 to 60;
        id(j)=.;
        wk(j)=.;
        wg(j)=.;
     end;
 
     HHID&y=input(V84,best12.);    
     V84N=input(V84,2.);    

run;   

data add_vill_&p._&f._03 (drop=V84);
     set work&f.19_&p._adjlist;
     V84N=input(V84,2.0);
run;
 
data add_vill_&p._&f._04;
     set add_vill_&p._&f._02 
         add_vill_&p._&f._03;
run;


*******************************************************************************************************************;

*********************************************************************
** Temporarily Merge Together All 4 Final Files to purge ID errors **
*********************************************************************;

proc sort data=add_vill_paid_&f._04 out=work&f.20_paid_adj;
     by HHID&y V84N;
run;

proc sort data=add_vill_unpaid_&f._04 out=work&f.20_unpaid_adj;
     by HHID&y V84N;
run;

proc sort data=work&f.19_paid_attrib out=work&f.20_paid_att;
     by HHID&y V84N;
run;

proc sort data=work&f.19_unpaid_attrib out=work&f.20_unpaid_att;
     by HHID&y V84N;
run;


data work&f.21_all_files;
     merge work&f.20_paid_adj (in=a)
           work&f.20_unpaid_adj (in=b)
           work&f.20_paid_att (in=c)
           work&f.20_unpaid_att (drop=RICE IS_HH in=d);
     by HHID&y V84N;
     if a=1 and b=1 and c=1 and d=1 then output;
run;  

*******************************************
** Format files to be exported to UCINET **
*******************************************;

** I have decided to revert these matrices back to hh-only - 2011 02 07 **;
** These changes are reflected only in the four data steps below **;
** The village-level information still exists in all prior data steps **;
** In the future, I'd like to add a village-level only analysis **;

** I have also re-combined all files into a single one as of 2011 02 09 **;

data r&y._all (drop=PID_H01-PID_H10 PWRK_H01-PWRK_H10 PWAG_H01-PWAG_H10
                    UID_H01-UID_H10 UWRK_H01-UWRK_H10 UWAG_H01-UWAG_H10);
     set work&f.21_all_files;

     ALL_WRKS=ALL_PWRK+ALL_UWRK;
     ALL_WAGE=ALL_PWAG+ALL_UWAG;

     if HHID&y>100;               * Remove village datalines *;
run;

**************************************************************************************************;

************************************************
** Create separate village files EACH VILLAGE **
************************************************;

%macro v_split (numvill=);  %* macro splits villages *;

       %* NUMVILL=Number of Unique Villages in file *;

%do i=1 %to &numvill;

    data r00_u&i (drop=V84N);
         set r&y._all;
         if V84N=&i;
    run;

%end;

%mend v_split;

%v_split (numvill=51);

***************************************************************************************
* 9 ** VILLAGE NETWORKS: RICE UNPAID ** Create VALUED adjacency matrices: TOT WAGES  **
***************************************************************************************;

%macro v_adj1 (numvill=);
%do i=1 %to &numvill;
proc iml;
     %include '/home/jhull/public/span/adjval.mod';
     %include '/home/jhull/public/span/pajwrite.mod';
     %let p1=%quote(/home/jhull/nangrong/data_paj/2000/a4-net/v0);
     %let p2=%quote(00a4.net);
     use r00_u&i;
     read all var{
                  UID_H11 UID_H12 UID_H13 UID_H14 UID_H15
                  UID_H16 UID_H17 UID_H18 UID_H19 UID_H20
                  UID_H21 UID_H22 UID_H23 UID_H24 UID_H25
                  UID_H26 UID_H27 UID_H28 UID_H29 UID_H30
                  UID_H31 UID_H32 UID_H33 UID_H34 UID_H35
                  UID_H36 UID_H37 UID_H38 UID_H39 UID_H40
                  UID_H41 UID_H42 UID_H43 UID_H44 UID_H45
                  UID_H46 UID_H47 UID_H48 UID_H49 UID_H50
                  UID_H51 UID_H52 UID_H53 UID_H54 UID_H55
                  UID_H56 UID_H57 UID_H58 UID_H59 UID_H60} into rcv;
     read all var{
                  UWAG_H11 UWAG_H12 UWAG_H13 UWAG_H14 UWAG_H15
                  UWAG_H16 UWAG_H17 UWAG_H18 UWAG_H19 UWAG_H20
                  UWAG_H21 UWAG_H22 UWAG_H23 UWAG_H24 UWAG_H25
                  UWAG_H26 UWAG_H27 UWAG_H28 UWAG_H29 UWAG_H30
                  UWAG_H31 UWAG_H32 UWAG_H33 UWAG_H34 UWAG_H35
                  UWAG_H36 UWAG_H37 UWAG_H38 UWAG_H39 UWAG_H40
                  UWAG_H41 UWAG_H42 UWAG_H43 UWAG_H44 UWAG_H45
                  UWAG_H46 UWAG_H47 UWAG_H48 UWAG_H49 UWAG_H50
                  UWAG_H51 UWAG_H52 UWAG_H53 UWAG_H54 UWAG_H55
                  UWAG_H56 UWAG_H57 UWAG_H58 UWAG_H59 UWAG_H60} into val;
     read all var{HHID00} into snd;
     r00_n=adjval(snd,rcv,val);
     id=r00_n[,1];
     r00_n=r00_n[,2:ncol(r00_n)];
     adj=r00_n;
     file "&p1.&i.&p2";
     call pajwrite(adj,id,2);
     create adj0&i from adj;
            append from adj;
     idinv=id`;
     create id0&i from idinv;
            append from idinv;
quit;
%end;
%mend v_adj1;

%v_adj1(numvill=9);

%macro v_adj2 (numvill=);
%do i=10 %to &numvill;
proc iml;
     %include '/home/jhull/public/span/adjval.mod';
     %include '/home/jhull/public/span/pajwrite.mod';
     %let p1=%quote(/home/jhull/nangrong/data_paj/2000/a4-net/v);
     %let p2=%quote(00a4.net);
     use r00_u&i;
     read all var{
                  UID_H11 UID_H12 UID_H13 UID_H14 UID_H15
                  UID_H16 UID_H17 UID_H18 UID_H19 UID_H20
                  UID_H21 UID_H22 UID_H23 UID_H24 UID_H25
                  UID_H26 UID_H27 UID_H28 UID_H29 UID_H30
                  UID_H31 UID_H32 UID_H33 UID_H34 UID_H35
                  UID_H36 UID_H37 UID_H38 UID_H39 UID_H40
                  UID_H41 UID_H42 UID_H43 UID_H44 UID_H45
                  UID_H46 UID_H47 UID_H48 UID_H49 UID_H50
                  UID_H51 UID_H52 UID_H53 UID_H54 UID_H55
                  UID_H56 UID_H57 UID_H58 UID_H59 UID_H60} into rcv;
     read all var{
                  UWAG_H11 UWAG_H12 UWAG_H13 UWAG_H14 UWAG_H15
                  UWAG_H16 UWAG_H17 UWAG_H18 UWAG_H19 UWAG_H20
                  UWAG_H21 UWAG_H22 UWAG_H23 UWAG_H24 UWAG_H25
                  UWAG_H26 UWAG_H27 UWAG_H28 UWAG_H29 UWAG_H30
                  UWAG_H31 UWAG_H32 UWAG_H33 UWAG_H34 UWAG_H35
                  UWAG_H36 UWAG_H37 UWAG_H38 UWAG_H39 UWAG_H40
                  UWAG_H41 UWAG_H42 UWAG_H43 UWAG_H44 UWAG_H45
                  UWAG_H46 UWAG_H47 UWAG_H48 UWAG_H49 UWAG_H50
                  UWAG_H51 UWAG_H52 UWAG_H53 UWAG_H54 UWAG_H55
                  UWAG_H56 UWAG_H57 UWAG_H58 UWAG_H59 UWAG_H60} into val;
     read all var{HHID00} into snd;
     r00_n=adjval(snd,rcv,val);
     id=r00_n[,1];
     r00_n=r00_n[,2:ncol(r00_n)];
     adj=r00_n;
     file "&p1.&i.&p2";
     call pajwrite(adj,id,2);
     create adj&i from adj;
            append from adj;
     idinv=id`;
     create id&i from idinv;
            append from idinv;
quit;
%end;
%mend v_adj2;

%v_adj2(numvill=51);

%macro v_adj3(numvill=);
%do i=1 %to &numvill;
    data _null_ ;
         %let p1=%quote(/home/jhull/nangrong/data_paj/2000/a4-adj/v0);
         %let p2=%quote(00a4.adj);
         set adj0&i;
         file "&p1.&i.&p2"  lrecl=1000;
         put (_ALL_) (+0);
run;
    data _null_ ;
         %let p3=%quote(/home/jhull/nangrong/data_paj/2000/a4-id/v0);
         %let p4=%quote(00a4.id);
         set id0&i;
         file "&p3.&i.&p4"  lrecl=10000;
         put (_ALL_) (+0);
run;

%end;
%mend v_adj3;

%v_adj3(numvill=9);

%macro v_adj4(numvill=);
%do i=10 %to &numvill;  
    data _null_ ;
         %let p1=%quote(/home/jhull/nangrong/data_paj/2000/a4-adj/v);
         %let p2=%quote(00a4.adj);
         set adj&i;
         file "&p1.&i.&p2"  lrecl=1000;
         put (_ALL_) (+0);
run;
    data _null_ ;
         %let p3=%quote(/home/jhull/nangrong/data_paj/2000/a4-id/v);
         %let p4=%quote(00a4.id);
         set id&i;
         file "&p3.&i.&p4"  lrecl=10000;
         put (_ALL_) (+0);
run;
%end;
%mend v_adj4;

%v_adj4(numvill=51);

In [ ]:
********************************************************************
**     Program Name: /home/jhull/nangrong/prog_sas/p02_nets/2000/p02_00b1.sas
**     Programmer: james r. hull
**     Start Date: 2011 03 09
**     Purpose:
**        1.) Create B1 = MALE SIBLING NETWORKS
**
**     Input Data:
**        '/home/jhull/nangrong/data_sas/2000/current/indiv00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/hh00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/comm00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/sibs00.xpt'
**
**     Output Data:
**        1.) /home/jhull/nangrong/data_paj/2000/b1/vXX00b1.net
**
**     Notes: 
**        1.) 
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=60;

%let f=00b1;  ** names file (allows portability) **;
%let y=00; 

**********************
**  Data Libraries  **
**********************;

libname in&f.1 xport '/home/jhull/nangrong/data_sas/2000/current/indiv00.xpt';
libname in&f.2 xport '/home/jhull/nangrong/data_sas/2000/current/hh00.xpt';
libname in&f.3 xport '/home/jhull/nangrong/data_sas/2000/current/comm00.xpt';
libname in&f.4 xport '/home/jhull/nangrong/data_sas/2000/current/sibs00.xpt';


*****************************************************************************************************;

*************************************************
**  Create Sibling and Parent Network Matrices **
*************************************************;

**************
** SIBLINGS **
**************;

**********************************************************************************
**                                                                              **
** In this (1984) village                     2 + Village #  + House #          **
** In this (1984) village, house # is unknown 2 + Village #  + 999              **
** Another village in Nang Rong               3 + 000            + Village #    **
** In Nang Rong, but village # is unknown     3 + 000            + 999999       **
** Outside Nang Rong, but within Buriram      4 + 0 + District # + 000000       **
** Another province                           5 + 0 + Province # + 000000       **
** Another country                            6 + 0 + Country #  + 000000       **
** N/A                                        [ ]                               **
** Missing/Don?t know                         9 + 999            + 999999       **
**                                                                              **
**********************************************************************************;

data work&f.31 (keep=HHID00 SEX AGE PLACE);
     set in&f.4.sibs00 (keep=HHID00 X4_5A: X4_5S: X4_5R:);

     length place $10.;

     array a(1:16) X4_5A1-X4_5A16;
     array s(1:16) X4_5S1-X4_5S16;
     array p(1:16) X4_5R1-X4_5R16;

     do i=1 to 16;
        SEX=s(i);
        AGE=a(i);
        PLACE=p(i);
        if PLACE ^in ("9999999999","          ")  then output;
     end;
run;

data work&f.32 (drop=HHID00C VILLAGE BLT00 LOCATION SEX) work&f.33 (drop=HHID00C BLT00 VILLBLT);
     set work&f.31;

     HHID00C=put(HHID00,$9.);


     if AGE in (99,.) then AGE=.;
     if SEX in (9,.) then SEX=.;

     if SEX=2 then MALE=0;
        else if SEX=1 then MALE=1;
        else MALE=.;

     if substr(PLACE,1,1)="2" then LOCATION=1;
     if substr(PLACE,1,1)="3" then LOCATION=2;
     if substr(PLACE,1,1)="4" then LOCATION=3;
     if substr(PLACE,1,1)="5" then LOCATION=4;
     if substr(PLACE,1,1)="6" then LOCATION=5;

     if LOCATION=1 then VILLAGE=substr(PLACE,2,6);

     if LOCATION=2 then do;
                           if substr(PLACE,5,6) ne "999999" then VILLAGE=substr(PLACE,5,6);
                        end;


     if LOCATION=1 and substr(PLACE,8,3) ne "999" then BLT00=substr(PLACE,8,3);


     if VILLAGE ne "      " and BLT00 ne "   " then VILLBLT=trim(VILLAGE)||BLT00;

     ** All Identifiable In-Village Sibling Ties **;

     if LOCATION in (1,2) and VILLAGE ne "      " and BLT00 ne "   " then output work&f.32;

     ** All Identifiable Sibling Ties in Nang Rong (Excludes other Provinces, etc.) **;

     if LOCATION in (1,2) and VILLAGE ne "      " then output work&f.33;

run;

******************************************************************************************
** Re-Sort by VILLAGE and BLT then merge to standard list to remove errors and mistakes **
******************************************************************************************;

data vill_id_&f.01;
     set in&f.1.indiv00;
     keep HHID00 V84;
run;

proc sort data=vill_id_&f.01 out=vill_id_&f.02 nodupkey;
     by HHID00 v84;
run;

data work&f.32b;
     merge work&f.32 (in=a)
           vill_id_&f.02 (in=b);
     by HHID00;
     if a=1 then output;
run;


data work&f.34 (drop=VILL00 HOUSE00);
     set in&f.2.hh00 (keep=HHID00 VILL00 HOUSE00);

     SIBHH00=HHID00;

     VILLBLT=VILL00||HOUSE00;
run;

data work&f.34b (drop=HHID00);
     merge work&f.34 (in=a)
           vill_id_&f.02 (in=b);
     by HHID00;
     if a=1 then output;
run;


proc sort data=work&f.32b out=work&f.35;
     by V84 VILLBLT;
run;

proc sort data=work&f.34b out=work&f.36;
     by V84 VILLBLT;
run;

data work&f.37 (drop=PLACE VILLBLT);
     merge work&f.35 (in=a)
           work&f.36 (in=b);
     by V84 VILLBLT;
     if a=1 and b=1 then output;
run;

proc sort data=work&f.37 out=work&f.38;
     by HHID00;
run;


*********************************************
** Reformat character variables to numeric **
*********************************************;

data work&f.40 (drop=SIBHH00 HHID00 V84);
     set work&f.38;

     SIBHH00N=input(strip(SIBHH00),9.);
     HHID00N=input(strip(HHID00),9.);
     V84N=input(V84,2.);
run;

data work&f.41 (drop=SIBHH00N HHID00N V84N);
     set work&f.40;

     SIBHH00=SIBHH00N;
     HHID00=HHID00N;
     V84=V84N;
run;                                     ** Major Checkpoint - Village Sibling "Child" File **;

****************************************************************************
** MALES: Separate Males from Females to Create Separate Sibling Networks **
****************************************************************************;

%let g=male;

data work&f.41_&g;
     set work&f.41;
     if MALE=1;
run;

**************************************************************
** MALES: Collapse non-ego HHs containing multiple siblings **
**************************************************************;

proc sort data=work&f.41_&g out=work&f.42_&g;
     by HHID00 SIBHH00;
run;

data work&f.43_&g;
     set work&f.42_&g;

     HHSIB00=trim(HHID00)||strip(SIBHH00);
run;

data work&f.44_&g (drop=MALE AGE HHSIB00 i);
     set work&f.43_&g;

     by HHSIB00;

     retain SUM_SEX SUM_AGE SUM_SIB MIS_SIB i;

     if first.HHSIB00 then do;
                             SUM_SEX=0;
                             SUM_AGE=0;
                             SUM_SIB=0;
                             MIS_SIB=0;
                             i=1;
                           end;

     SUM_SEX=SUM_SEX+MALE;
     if AGE ne . then SUM_AGE=SUM_AGE+AGE;
     if AGE ne . then MIS_SIB=MIS_SIB+1;
     SUM_SIB=SUM_SIB+1;
     i=i+1;

    if last.HHSIB00 then output;

run;

data work&f.45_&g(drop=SUM_SEX SUM_AGE MIS_SIB);
     set work&f.44_&g;

     RAT_M=round(SUM_SEX/SUM_SIB,.01);
     if MIS_SIB=0 then AVG_A=round(SUM_AGE/SUM_SIB,.01);
        else AVG_A=round(SUM_AGE/MIS_SIB,.01);
run;

******************************************
** MALES: Unstack back to a mother file **
******************************************;

data work&f.46_&g (keep= HHID00 V84 MALE01-MALE16 AGE01-AGE16 NUM01-NUM16 SIBHH01-SIBHH16);
     set work&f.45_&g;
     by HHID00;

     retain MALE01-MALE16 AGE01-AGE16 NUM01-NUM16 SIBHH01-SIBHH16 i;


     array s(1:16) MALE01-MALE16;
     array a(1:16) AGE01-AGE16;
     array t(1:16) NUM01-NUM16;
     array p(1:16) SIBHH01-SIBHH16;

     if first.HHID00 then do;
                            do j=1 to 16;
                                        s(j)=.;
                                        a(j)=.;
                                        t(j)=.;
                                        p(j)=.;
                            end;
                            i=1;
                          end;

     s(i)=RAT_M;
     a(i)=AVG_A;
     t(i)=SUM_SIB;
     p(i)=SIBHH00;

     i=i+1;

     if last.HHID00 then output;
run;

*************************************************
** MALES: merge in households with no siblings **
*************************************************;

data vill_id_&f.03_&g (drop=V84 HHID00);
     set vill_id_&f.02;
     V84N=input(V84,2.0);
     HHID00N=input(HHID00,9.0);
run;

data vill_id_&f.04_&g (drop=V84N HHID00N);
     set vill_id_&f.03_&g;
     V84=V84N;
     HHID00=HHID00N;
run;

data work&f.47_&g;
     merge work&f.46_&g (in=a)
           vill_id_&f.04_&g (in=b);
     by HHID00;
     if b=1 then output;
run;


***********************************************************
** MALES: Create separate village files for EACH VILLAGE **
***********************************************************;

proc sort data=work&f.47_&g out=work&f.48_&g;
     by V84 HHID00;
run;

%macro v_split (numvill=);  %* macro splits villages *;

       %* NUMVILL=Number of Unique Villages in file *;

%do i=1 %to &numvill;

    data v00_ms&i (drop=V84);
         set work&f.48_&g;
         if V84=&i;
    run;

%end;

%mend v_split;

%v_split (numvill=51);


*******************************************************************************
** MALES: Create 51 VALUED adjacency matrices, one for each village -sibling **
*******************************************************************************;

%macro v_adj1 (numvill=);

%do i=1 %to &numvill;

proc iml;
     %include '/home/jhull/public/span/adjval.mod';
     %include '/home/jhull/public/span/pajwrite.mod';
     %include '/home/jhull/public/span/uciwrite.mod';
     %let p1=%quote(/home/jhull/nangrong/data_paj/2000/b1-net/v0);
     %let p2=%quote(00b1.net);
     use v00_ms&i;
     read all var{SIBHH01 SIBHH02 SIBHH03 SIBHH04 SIBHH05
                  SIBHH06 SIBHH07 SIBHH08 SIBHH09 SIBHH10
                  SIBHH11 SIBHH12 SIBHH13 SIBHH14 SIBHH15
                  SIBHH16} into rcv;
     read all var{NUM01 NUM02 NUM03 NUM04 NUM05
                  NUM06 NUM07 NUM08 NUM09 NUM10
                  NUM11 NUM12 NUM13 NUM14 NUM15
                  NUM16} into val;

     read all var{HHID00} into snd;
     r00_ms=adjval(snd,rcv,val);
     id=r00_ms[,1];
     r00_ms=r00_ms[,2:ncol(r00_ms)];
     adj=r00_ms;
     file "&p1.&i.&p2";
     call pajwrite(adj,id,2);
     create adj0&i from adj;
            append from adj;
     idinv=id`;
     create id0&i from idinv;
            append from idinv;
quit;
%end;
%mend v_adj1;

%v_adj1(numvill=9);


%macro v_adj2 (numvill=);
%do i=10 %to &numvill;
proc iml;
     %include '/home/jhull/public/span/adjval.mod';
     %include '/home/jhull/public/span/pajwrite.mod';
     %include '/home/jhull/public/span/uciwrite.mod';
     %let p1=%quote(/home/jhull/nangrong/data_paj/2000/b1-net/v);
     %let p2=%quote(00b1.net);
     use v00_ms&i;
     read all var{SIBHH01 SIBHH02 SIBHH03 SIBHH04 SIBHH05
                  SIBHH06 SIBHH07 SIBHH08 SIBHH09 SIBHH10
                  SIBHH11 SIBHH12 SIBHH13 SIBHH14 SIBHH15
                  SIBHH16} into rcv;
     read all var{NUM01 NUM02 NUM03 NUM04 NUM05
                  NUM06 NUM07 NUM08 NUM09 NUM10
                  NUM11 NUM12 NUM13 NUM14 NUM15
                  NUM16} into val;
     read all var{HHID00} into snd;
     r00_ms=adjval(snd,rcv,val);
     id=r00_ms[,1];
     r00_ms=r00_ms[,2:ncol(r00_ms)];
     adj=r00_ms;
     file "&p1.&i.&p2";
     call pajwrite(adj,id,2);
     create adj&i from adj;
            append from adj;
     idinv=id`;
     create id&i from idinv;
            append from idinv;
quit;
%end;
%mend v_adj2;

%v_adj2(numvill=51);

%macro v_adj3(numvill=);
%do i=1 %to &numvill;
    data _null_ ;
         %let p1=%quote(/home/jhull/nangrong/data_paj/2000/b1-adj/v0);
         %let p2=%quote(00b1.adj);
         set adj0&i;
         file "&p1.&i.&p2"  lrecl=1000;
         put (_ALL_) (+0);
run;
    data _null_ ;
         %let p3=%quote(/home/jhull/nangrong/data_paj/2000/b1-id/v0);
         %let p4=%quote(00b1.id);
         set id0&i;
         file "&p3.&i.&p4"  lrecl=10000;
         put (_ALL_) (+0);
run;

%end;
%mend v_adj3;

%v_adj3(numvill=9);

%macro v_adj4(numvill=);
%do i=10 %to &numvill;  
    data _null_ ;
         %let p1=%quote(/home/jhull/nangrong/data_paj/2000/b1-adj/v);
         %let p2=%quote(00b1.adj);
         set adj&i;
         file "&p1.&i.&p2"  lrecl=1000;
         put (_ALL_) (+0);
run;
    data _null_ ;
         %let p3=%quote(/home/jhull/nangrong/data_paj/2000/b1-id/v);
         %let p4=%quote(00b1.id);
         set id&i;
         file "&p3.&i.&p4"  lrecl=10000;
         put (_ALL_) (+0);
run;
%end;
%mend v_adj4;

%v_adj4(numvill=51);

In [ ]:
********************************************************************
**     Program Name: /home/jhull/nangrong/prog_sas/p02_nets/2000/p02_00b2.sas
**     Programmer: james r. hull
**     Start Date: 2011 03 09
**     Purpose:
**        1.) Create B2 = FEMALE SIBLING NETWORKS
**
**     Input Data:
**        '/home/jhull/nangrong/data_sas/2000/current/indiv00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/hh00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/comm00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/sibs00.xpt'
**
**     Output Data:
**        1.) /home/jhull/nangrong/data_paj/2000/b2/vXX00b2.net
**
**     Notes: 
**        1.) 
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=60;

%let f=00b2;  ** names file (allows portability) **;
%let y=00; 

**********************
**  Data Libraries  **
**********************;

libname in&f.1 xport '/home/jhull/nangrong/data_sas/2000/current/indiv00.xpt';
libname in&f.2 xport '/home/jhull/nangrong/data_sas/2000/current/hh00.xpt';
libname in&f.3 xport '/home/jhull/nangrong/data_sas/2000/current/comm00.xpt';
libname in&f.4 xport '/home/jhull/nangrong/data_sas/2000/current/sibs00.xpt';


*****************************************************************************************************;

*************************************************
**  Create Sibling and Parent Network Matrices **
*************************************************;

**************
** SIBLINGS **
**************;

**********************************************************************************
**                                                                              **
** In this (1984) village                     2 + Village #  + House #          **
** In this (1984) village, house # is unknown 2 + Village #  + 999              **
** Another village in Nang Rong               3 + 000            + Village #    **
** In Nang Rong, but village # is unknown     3 + 000            + 999999       **
** Outside Nang Rong, but within Buriram      4 + 0 + District # + 000000       **
** Another province                           5 + 0 + Province # + 000000       **
** Another country                            6 + 0 + Country #  + 000000       **
** N/A                                        [ ]                               **
** Missing/Don?t know                         9 + 999            + 999999       **
**                                                                              **
**********************************************************************************;

data work&f.31 (keep=HHID00 SEX AGE PLACE);
     set in&f.4.sibs00 (keep=HHID00 X4_5A: X4_5S: X4_5R:);

     length place $10.;

     array a(1:16) X4_5A1-X4_5A16;
     array s(1:16) X4_5S1-X4_5S16;
     array p(1:16) X4_5R1-X4_5R16;

     do i=1 to 16;
        SEX=s(i);
        AGE=a(i);
        PLACE=p(i);
        if PLACE ^in ("9999999999","          ")  then output;
     end;
run;

data work&f.32 (drop=HHID00C VILLAGE BLT00 LOCATION SEX) work&f.33 (drop=HHID00C BLT00 VILLBLT);
     set work&f.31;

     HHID00C=put(HHID00,$9.);


     if AGE in (99,.) then AGE=.;
     if SEX in (9,.) then SEX=.;

     if SEX=2 then MALE=0;
        else if SEX=1 then MALE=1;
        else MALE=.;

     if substr(PLACE,1,1)="2" then LOCATION=1;
     if substr(PLACE,1,1)="3" then LOCATION=2;
     if substr(PLACE,1,1)="4" then LOCATION=3;
     if substr(PLACE,1,1)="5" then LOCATION=4;
     if substr(PLACE,1,1)="6" then LOCATION=5;

     if LOCATION=1 then VILLAGE=substr(PLACE,2,6);

     if LOCATION=2 then do;
                           if substr(PLACE,5,6) ne "999999" then VILLAGE=substr(PLACE,5,6);
                        end;


     if LOCATION=1 and substr(PLACE,8,3) ne "999" then BLT00=substr(PLACE,8,3);


     if VILLAGE ne "      " and BLT00 ne "   " then VILLBLT=trim(VILLAGE)||BLT00;

     ** All Identifiable In-Village Sibling Ties **;

     if LOCATION in (1,2) and VILLAGE ne "      " and BLT00 ne "   " then output work&f.32;

     ** All Identifiable Sibling Ties in Nang Rong (Excludes other Provinces, etc.) **;

     if LOCATION in (1,2) and VILLAGE ne "      " then output work&f.33;

run;

******************************************************************************************
** Re-Sort by VILLAGE and BLT then merge to standard list to remove errors and mistakes **
******************************************************************************************;

data vill_id_&f.01;
     set in&f.1.indiv00;
     keep HHID00 V84;
run;

proc sort data=vill_id_&f.01 out=vill_id_&f.02 nodupkey;
     by HHID00 v84;
run;

data work&f.32b;
     merge work&f.32 (in=a)
           vill_id_&f.02 (in=b);
     by HHID00;
     if a=1 then output;
run;


data work&f.34 (drop=VILL00 HOUSE00);
     set in&f.2.hh00 (keep=HHID00 VILL00 HOUSE00);

     SIBHH00=HHID00;

     VILLBLT=VILL00||HOUSE00;
run;

data work&f.34b (drop=HHID00);
     merge work&f.34 (in=a)
           vill_id_&f.02 (in=b);
     by HHID00;
     if a=1 then output;
run;


proc sort data=work&f.32b out=work&f.35;
     by V84 VILLBLT;
run;

proc sort data=work&f.34b out=work&f.36;
     by V84 VILLBLT;
run;

data work&f.37 (drop=PLACE VILLBLT);
     merge work&f.35 (in=a)
           work&f.36 (in=b);
     by V84 VILLBLT;
     if a=1 and b=1 then output;
run;

proc sort data=work&f.37 out=work&f.38;
     by HHID00;
run;


*********************************************
** Reformat character variables to numeric **
*********************************************;

data work&f.40 (drop=SIBHH00 HHID00 V84);
     set work&f.38;

     SIBHH00N=input(strip(SIBHH00),9.);
     HHID00N=input(strip(HHID00),9.);
     V84N=input(V84,2.);
run;

data work&f.41 (drop=SIBHH00N HHID00N V84N);
     set work&f.40;

     SIBHH00=SIBHH00N;
     HHID00=HHID00N;
     V84=V84N;
run;                                     ** Major Checkpoint - Village Sibling "Child" File **;


******************************************************************************
** FEMALES: Separate Males from Females to Create Separate Sibling Networks **
******************************************************************************;

%let g=fem;

data work&f.41_&g;
     set work&f.41;
     if MALE=0;
run;

****************************************************************
** FEMALES: Collapse non-ego HHs containing multiple siblings **
****************************************************************;

proc sort data=work&f.41_&g out=work&f.42_&g;
     by HHID00 SIBHH00;
run;

data work&f.43_&g;
     set work&f.42_&g;

     HHSIB00=trim(HHID00)||strip(SIBHH00);
run;

data work&f.44_&g (drop=MALE AGE HHSIB00 i);
     set work&f.43_&g;

     by HHSIB00;

     retain SUM_SEX SUM_AGE SUM_SIB MIS_SIB i;

     if first.HHSIB00 then do;
                             SUM_SEX=0;
                             SUM_AGE=0;
                             SUM_SIB=0;
                             MIS_SIB=0;
                             i=1;
                           end;

     SUM_SEX=SUM_SEX+MALE;
     if AGE ne . then SUM_AGE=SUM_AGE+AGE;
     if AGE ne . then MIS_SIB=MIS_SIB+1;
     SUM_SIB=SUM_SIB+1;
     i=i+1;

    if last.HHSIB00 then output;

run;

data work&f.45_&g(drop=SUM_SEX SUM_AGE MIS_SIB);
     set work&f.44_&g;

     RAT_M=round(SUM_SEX/SUM_SIB,.01);
     if MIS_SIB=0 then AVG_A=round(SUM_AGE/SUM_SIB,.01);
        else AVG_A=round(SUM_AGE/MIS_SIB,.01);
run;

********************************************
** FEMALES: Unstack back to a mother file **
********************************************;

data work&f.46_&g (keep= HHID00 V84 MALE01-MALE16 AGE01-AGE16 NUM01-NUM16 SIBHH01-SIBHH16);
     set work&f.45_&g;
     by HHID00;

     retain MALE01-MALE16 AGE01-AGE16 NUM01-NUM16 SIBHH01-SIBHH16 i;


     array s(1:16) MALE01-MALE16;
     array a(1:16) AGE01-AGE16;
     array t(1:16) NUM01-NUM16;
     array p(1:16) SIBHH01-SIBHH16;

     if first.HHID00 then do;
                            do j=1 to 16;
                                        s(j)=.;
                                        a(j)=.;
                                        t(j)=.;
                                        p(j)=.;
                            end;
                            i=1;
                          end;

     s(i)=RAT_M;
     a(i)=AVG_A;
     t(i)=SUM_SIB;
     p(i)=SIBHH00;

     i=i+1;

     if last.HHID00 then output;
run;

***************************************************
** FEMALES: merge in households with no siblings **
***************************************************;

data vill_id_&f.03_&g (drop=V84 HHID00);
     set vill_id_&f.02;
     V84N=input(V84,2.0);
     HHID00N=input(HHID00,9.0);
run;

data vill_id_&f.04_&g (drop=V84N HHID00N);
     set vill_id_&f.03_&g;
     V84=V84N;
     HHID00=HHID00N;
run;

data work&f.47_&g;
     merge work&f.46_&g (in=a)
           vill_id_&f.04_&g (in=b);
     by HHID00;
     if b=1 then output;
run;


*************************************************************
** FEMALES: Create separate village files for EACH VILLAGE **
*************************************************************;

proc sort data=work&f.47_&g out=work&f.48_&g;
     by V84 HHID00;
run;

%macro v_split (numvill=);  %* macro splits villages *;

       %* NUMVILL=Number of Unique Villages in file *;

%do i=1 %to &numvill;

    data v00_fs&i (drop=V84);
         set work&f.48_&g;
         if V84=&i;
    run;

%end;

%mend v_split;

%v_split (numvill=51);


*********************************************************************************
** FEMALES: Create 51 VALUED adjacency matrices, one for each village -sibling **
*********************************************************************************;

%macro v_adj1 (numvill=);

%do i=1 %to &numvill;

proc iml;
     %include '/home/jhull/public/span/adjval.mod';
     %include '/home/jhull/public/span/pajwrite.mod';
     %include '/home/jhull/public/span/uciwrite.mod';
     %let p1=%quote(/home/jhull/nangrong/data_paj/2000/b2-net/v0);
     %let p2=%quote(00b2.net);
     use v00_fs&i;
     read all var{SIBHH01 SIBHH02 SIBHH03 SIBHH04 SIBHH05
                  SIBHH06 SIBHH07 SIBHH08 SIBHH09 SIBHH10
                  SIBHH11 SIBHH12 SIBHH13 SIBHH14 SIBHH15
                  SIBHH16} into rcv;
     read all var{NUM01 NUM02 NUM03 NUM04 NUM05
                  NUM06 NUM07 NUM08 NUM09 NUM10
                  NUM11 NUM12 NUM13 NUM14 NUM15
                  NUM16} into val;
     read all var{HHID00} into snd;
     r00_fs=adjval(snd,rcv,val);
     id=r00_fs[,1];
     r00_fs=r00_fs[,2:ncol(r00_fs)];
     adj=r00_fs;
     file "&p1.&i.&p2";
     call pajwrite(adj,id,2);
     create adj0&i from adj;
            append from adj;
     idinv=id`;
     create id0&i from idinv;
            append from idinv;
quit;
%end;
%mend v_adj1;

%v_adj1(numvill=9);

%macro v_adj2 (numvill=);
%do i=10 %to &numvill;
proc iml;
     %include '/home/jhull/public/span/adjval.mod';
     %include '/home/jhull/public/span/pajwrite.mod';
     %include '/home/jhull/public/span/uciwrite.mod';
     %let p1=%quote(/home/jhull/nangrong/data_paj/2000/b2-net/v);
     %let p2=%quote(00b2.net);
     use v00_fs&i;
     read all var{SIBHH01 SIBHH02 SIBHH03 SIBHH04 SIBHH05
                  SIBHH06 SIBHH07 SIBHH08 SIBHH09 SIBHH10
                  SIBHH11 SIBHH12 SIBHH13 SIBHH14 SIBHH15
                  SIBHH16} into rcv;
     read all var{NUM01 NUM02 NUM03 NUM04 NUM05
                  NUM06 NUM07 NUM08 NUM09 NUM10
                  NUM11 NUM12 NUM13 NUM14 NUM15
                  NUM16} into val;
     read all var{HHID00} into snd;
     r00_fs=adjval(snd,rcv,val);
     id=r00_fs[,1];
     r00_fs=r00_fs[,2:ncol(r00_fs)];
     adj=r00_fs;
     file "&p1.&i.&p2";
     call pajwrite(adj,id,2);
     create adj&i from adj;
            append from adj;
     idinv=id`;
     create id&i from idinv;
            append from idinv;
quit;

%end;

%mend v_adj2;

%v_adj2(numvill=51);

%macro v_adj3(numvill=);
%do i=1 %to &numvill;
    data _null_ ;
         %let p1=%quote(/home/jhull/nangrong/data_paj/2000/b2-adj/v0);
         %let p2=%quote(00b2.adj);
         set adj0&i;
         file "&p1.&i.&p2"  lrecl=1000;
         put (_ALL_) (+0);
run;
    data _null_ ;
         %let p3=%quote(/home/jhull/nangrong/data_paj/2000/b2-id/v0);
         %let p4=%quote(00b2.id);
         set id0&i;
         file "&p3.&i.&p4"  lrecl=10000;
         put (_ALL_) (+0);
run;

%end;
%mend v_adj3;

%v_adj3(numvill=9);

%macro v_adj4(numvill=);
%do i=10 %to &numvill;  
    data _null_ ;
         %let p1=%quote(/home/jhull/nangrong/data_paj/2000/b2-adj/v);
         %let p2=%quote(00b2.adj);
         set adj&i;
         file "&p1.&i.&p2"  lrecl=1000;
         put (_ALL_) (+0);
run;
    data _null_ ;
         %let p3=%quote(/home/jhull/nangrong/data_paj/2000/b2-id/v);
         %let p4=%quote(00b2.id);
         set id&i;
         file "&p3.&i.&p4"  lrecl=10000;
         put (_ALL_) (+0);
run;
%end;
%mend v_adj4;

%v_adj4(numvill=51);

In [ ]:
***********************************************************************************
**     Program Name: /home/jhull/nangrong/prog_sas/p02_nets/2000_att/p02_00at.sas
**     Programmer: james r. hull
**     Start Date: 2011 03 09
**     Purpose:
**        1.) Create AT = HOUSEHOLD ATTRIBUTES BY VILLAGE
**
**     Input Data:
**        '/home/jhull/nangrong/data_sas/2000/current/indiv00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/hh00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/comm00.xpt'
**        '/home/jhull/nangrong/data_sas/2000/current/sibs00.xpt'
**
**     Output Data:
**        1.) /home/jhull/nangrong/data_paj/2000_att/vXX00at.net
**
**     Notes: 
**        1.) ADD LIST: HHTYPE94 
**                      Household Has Migrants
**                      Interviewer Code
**                      Month of Interview - Month of Harvest (94)
**                      Number of Interview Attempts
**                      Relationship of Respondent to HH Head
**                      # Present at start of interview
**
***********************************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=60;

%let f=00at;  ** names file (allows portability) **;
%let y=00; 

**********************
**  Data Libraries  **
**********************;

libname in&f.1 xport '/home/jhull/nangrong/data_sas/2000/current/indiv00.xpt';
libname in&f.2 xport '/home/jhull/nangrong/data_sas/2000/current/hh00.xpt';
libname in&f.3 xport '/home/jhull/nangrong/data_sas/2000/current/comm00.xpt';
libname in&f.4 xport '/home/jhull/nangrong/data_sas/2000/current/sibs00.xpt';


******************************
**  Create Working Dataset  **
******************************;

********************************************************************
**  Length 10
**  0 In this 1984 village                2 + Village # + House #
**  2 In this village but blt             2 + Village # + 999
**  Length 7 
**  4 Another village in Nang Rong        3 + Village #
**  8 Unknown village in Nang Rong        3 + 999999
**  5 Another district in Buriram         4 + District # + 0000
**  6 Another province                    5 + District # + 0000
**  7 Another country 
**  8 Missing/Don't know                  9999999999 or 9999999
**  Length 9
**  9 Code 2 or 3 Returning HH member
*******************************************************************;

***************************************************************************
** Stack rice harvest labor data into a child file and label by location **
***************************************************************************;

data work&f.01a;
     set in&f.2.hh00 (keep=HHID00 X6_84C: X6_84W:);
     keep HHID00 X6_86L X6_86N X6_86W LOCATION;

     length X6_86L $ 10;

     array a(1:7) X6_84C1-X6_84C7;
     array b(1:7) X6_84W1-X6_84W7;

     do i=1 to 7;
          X6_86L=a(i);
          X6_86N=1;
          X6_86W=b(i);
          LOCATION=9;
          if a(i) ne " ." then output;  * Keep only those cases with data *;
     end;
run;

data work&f.01b;
     set in&f.2.hh00 (keep=HHID00 X6_85H: X6_85N: X6_85W:);
     keep HHID00 X6_86L X6_86N X6_86W LOCATION;

     length X6_86L $ 10;

     array a(1:13) X6_85H1-X6_85H13;
     array b(1:13) X6_85N1-X6_85N13;
     array c(1:13) X6_85W1-X6_85W13;

     do i=1 to 13;
 
          ** FIX ANOTHER CRAZY ERROR IN ORIGINAL DATA WITH REVERSED CODING FOR IN-VILLAGE LABOR **;
         
          if substr(a(i),5,3) in ('020','021','170','180') then a(i)=cat("2",substr(a(i),5,6),substr(a(i),2,3));

          if a(i)="9999999999" then LOCATION=8;
             else if substr(a(i),8,3)='999' then LOCATION=2;
             else LOCATION=0;

          X6_86L=a(i);
          X6_86N=b(i);
          X6_86W=c(i);

          if a(i) ne "         ." then output;  * Keep only those cases with data *;
     end;
run;

data work&f.01c;
     set in&f.2.hh00 (keep=HHID00 X6_86L: X6_86N: X6_86W:);
     keep HHID00 X6_86L X6_86N X6_86W LOCATION;

     length X6_86L $ 10;

     array a(1:10) X6_86L1-X6_86L10;
     array b(1:10) X6_86N1-X6_86N10;
     array c(1:10) X6_86W1-X6_86W10;

     do i=1 to 10;
          X6_86L=a(i);
          X6_86N=b(i);
          X6_86W=c(i);
          if a(i)='9999999' then LOCATION=8;
             else if substr(a(i),1,1)='5' then LOCATION=6;
             else if substr(a(i),1,1)='4' then LOCATION=5;
             else if substr(a(i),1,2)='39' then LOCATION=8;
             else LOCATION=4;
          if a(i) ne "      ." then output;  * Keep only those cases with data *;
     end;

run;

*********************************************************
** Take Care of Missing Data Issues - Recodes at least **
*********************************************************;

data work&f.02a;
     set work&f.01a;

     if X6_86W=9 then X6_86W=.;
     if X6_86N=99 then X6_86N=1; * Assume at least 1 person worked *;

run;

****************************************************************************
** This code collapses multiple code 2 & 3 workers from a household to    **
** a single observation and sums the values for each into summary counts. **
** For the "type of labor" variable, I use an "any paid --> paid" rule    **
** because paying a code 2 or 3 laborer is a rare behavior and distinct   **
****************************************************************************;

 data work&f.02a2;
     set work&f.02a;

     by HHID00;

     retain SUM_N SUM_TYPE SUM_LOC i;


     if first.HHID00 then do;
                            SUM_N=0;
                            SUM_TYPE=2; * Default is unpaid labor*;
                            SUM_LOC=9;
                            i=1;
                         end;

     SUM_N=SUM_N+X6_86N;
     if X6_86W=1 then SUM_TYPE=1;  * Any paid --> all paid *;
     SUM_LOC=9;
     i=i+1;

     if last.HHID00 then output;

run;

data work&f.02a3 (drop=SUM_N SUM_TYPE SUM_LOC i);
     set work&f.02a2;

     X6_86L="   ";
     X6_86N=SUM_N;
     X6_86W=SUM_TYPE;
     LOCATION=SUM_LOC;

run;

*********************************************************
** Take Care of Missing Data Issues - Recodes at least **
*********************************************************;

data work&f.02b;
     set work&f.01b;

     if X6_86W=9 then X6_86W=.;
     if X6_86N=99 then X6_86N=1; * Assume at least 1 person worked *;
run;

data work&f.02c;
     set work&f.01c;

     if X6_86W=9 then X6_86W=.;
     if X6_86N=99 then X6_86N=1; * Assume at least 1 person worked *;
run;

**************************
** Merge files together **
**************************;

data work&f.03;
     set work&f.02a3
         work&f.02b
         work&f.02c;
run;

***************************************************************************
** Add V84 identifiers to 2000 data file as per Rick's suggestion on web **
***************************************************************************;



proc sort data=work&f.03 out=work&f.04;
     by HHID00 X6_86L LOCATION;
run;

data vill_id_&f.01;
     set in&f.1.indiv00;
     keep HHID00 V84;
run;

proc sort data=vill_id_&f.01 out=vill_id_&f.02 nodupkey;
     by HHID00 v84;
run;

data vill_id_&f.03;
     merge work&f.04 (in=a)
           vill_id_&f.02 (in=b);
           if a=1 and b=1 then output;
     by HHID00;
run;

proc sort data=vill_id_&f.03 out=work&f.05;
     by V84;
run;

******************************************************************************
** This step removes all cases about which there is no information about    **
** how their laborers were compensated. This is my fix for the time being.  **
** Note: in doing so, I lose 7 cases (a case here is a helper group)        **
******************************************************************************;

data work&f.06;
     set work&f.05;

     rename X6_86L=HELPHHID;

     if X6_86W ^in (.,9) then output;

run;

************************************************************************************
** The steps below convert the ban lek ti information on the helping household    **
** into the standard HHID##, as a preparatory step to creating network datafiles. **
************************************************************************************;

data work&f.07;
     set work&f.06;

     if LOCATION=9 then do;
                           HELPHHID=HHID00;
                        end;
        else if LOCATION in(0,4) then do;
                                  HELPHHID=substr(HELPHHID,2,9);
                                end;
        else if LOCATION in (2) then do;
                                    HELPHHID=substr(HELPHHID,2,6);
                                end;
        else HELPHHID=".";
run;

*********************************
** Clean HELPHH00 of BAD CODES **
*********************************; 

data work&f.08a (drop=HHID&y.C);
     set work&f.07 (rename=(HHID&y=HHID&y.C));
     HELPHH00=input(HELPHHID,9.0);
     V84N=input(V84,2.0);
     HHID00=input(HHID&y.C,9.);
run;

data work&f.08b (drop=HHID00 V84);
     set in&f.1.indiv00 (keep=HHID00 V84);
     HELPHH00=input(HHID00,9.0);
     V84N=input(V84,2.0);
run;

proc sort data=work&f.08a out=work&f.09a;
     by V84N HELPHH00;
run;

proc sort data=work&f.08b out=work&f.09b nodupkey;
     by V84N HELPHH00;
run;

data work&f.10 aonly bonly;
     merge work&f.09a (in=a)
           work&f.09b (in=b);
     by V84N HELPHH00;
     if a=1 and b=1 then output work&f.10;
     if a=1 and b=0 then output aonly;
     if a=0 and b=1 then output bonly;
run;


** This step removes the HELPHH00 codes from a number of HHs **;
** that had non-existent or incorrect HHs listed as helping  **;

data aonly_fix;
     set aonly;
     if HELPHH00>999999 then do;     
                               HELPHH00=".";
                               HELPHHID=".";
                             end;
run;
 
data work&f.11;
     set work&f.10 (in=a)
         aonly_fix (in=b);
run;

************************************************************************
**input average village wages during high demand from community survey**
************************************************************************;

data work&f.vill_wages_01a (drop=X45MHIGH X45MTYP);  
    set in&f.3.comm&y (keep=VILL00 X45MHIGH X45MTYP);

    if X45MHIGH=9999998 then X45MHIGH=.; ** USING ONLY MALES B/C MALE AND FEMALE WAGES IDENTICAL **;
    if X45MTYP=9999998 then X45MTYP=.;

    if X45MHIGH=. then RICEWAGH=125.77;
       else RICEWAGH=X45MHIGH;
    if X45MTYP=. then RICEWAGN=105.29;
       else RICEWAGN=X45MTYP;
run;

proc sort data=work&f.vill_wages_01a out=work&f.vill_wages_01b nodupkey;
     by VILL&y;
run;

proc sort data=in&f.1.indiv&y  out=work&f.vill_wages_02a nodupkey;
     by HHID&y;
run;

data work&f.make_vill;
     set work&f.vill_wages_02a (keep=HHID&y);
     length VILL&y $ 6;

     VILL&y=substr(HHID&y,1,6);
run;

data work&f.numeric (drop=HHID&y.C);
     set work&f.make_vill (rename=(HHID&y=HHID&y.C));

     HHID&y=input(HHID&y.C,best9.);
run;

proc sort data=work&f.numeric out=work&f.vill_wages_02b;
     by VILL&y HHID&y;
run;

data work&f.vill_wages_03;
     merge work&f.vill_wages_01b (in=a)
           work&f.vill_wages_02b (in=b);
     by VILL&y;
     if b=1 then output;
run;

proc sort data=work&f.vill_wages_03 out=work&f.vill_wages_04;
     by HHID&y;
run;

proc sort data=work&f.11 out=work&f.vill_wages_05;
     by HHID&y;
run;

data work&f.vill_wages_06;
     merge work&f.vill_wages_04 (in=a)
           work&f.vill_wages_05 (in=b);
     by HHID&y;
     if b=1 then output;
run;

proc sort data=work&f.vill_wages_06 out=work&f.11B;
     by HHID&y;
run;

data work&f.11C;
     set work&f.11B;

     if X6_86W=3 then PAIDHH00=2;
        else PAIDHH00=X6_86W;

     if X6_86W=1 then ALLWAGE=round(X6_86N*RICEWAGH,.01);   ** ACTUAL DAYS WORKED*AVG WAGES IN VILL**;
        else ALLWAGE=0;                                             

     NUMWRKS=X6_86N;
  
run;

data work&f.12 (keep=HHID00 PAIDHH00 HELPHH00 LOCATION ALLWAGE NUMWRKS V84N);
     set work&f.11C;
run;

proc sort data=work&f.12 out=work&f.13;              ** MAJOR CHECKPOINT - TOTAL LABOR CHILD FILE **;
     by HHID&y HELPHH&y;
run;

*******************************************************************************************************************************;

***********************************************
** Separate Paid and Unpaid Labor into Files **
***********************************************;

** Note, all code below must be repeated for paid and then unpaid labor **;

**************************************************************************
** PAID: Create Village Aggregate Measures for Help from Beyond Village **
**************************************************************************;

%let p=paid;    *** saves keystrokes ***;

data work&f.14_&p (drop=PAIDHH&y);
     set work&f.13;
     if PAIDHH&y=1;
run;

** SPLIT DATA INTO 3 GROUPS - SAME VILL, OTH VILL, ALL OTHER **;

data work&f.15_&p._othvill (drop=LOCATION);   ** 1 **;
     set work&f.14_&p;
     if LOCATION in (4);
     if HELPHH&y ne .;
run;

data work&f.15_&p._vill (drop=LOCATION);      ** 2 **;
     set work&f.14_&p;
     if LOCATION in (0,1);
     if HELPHH&y ne .;
run;

data work&f.15_&p._other (drop=LOCATION);     ** 3 **;
     set work&f.14_&p;
     if (LOCATION in (2,3,5,6,7,8,9) OR HELPHH&y=.);
run;

****************************************************
** (1) OTHVILL: PROCESS LABOR FROM OTHER VILLAGES **
****************************************************;

** Collapse Labor from the same 84 village **;

data work&f.16_&p._othvill (keep=HHID&y V84N TOT_WRKS TOT_WAGE);
     set work&f.15_&p._othvill;

     retain TOT_WRKS TOT_WAGE;

     by HHID&y V84N;

     if first.V84N then do;
                          TOT_WRKS=0;
                          TOT_WAGE=0;
                        end;
     if NUMWRKS ne . then TOT_WRKS=TOT_WRKS+NUMWRKS;
     if ALLWAGE ne . then TOT_WAGE=TOT_WAGE+ALLWAGE;

    if last.V84N then output;
 
run;

** Merge with all rice-growing households **;

data work&f.add_all_01 (drop=HHID&y.C); 
     set vill_id_&f.02 (rename=(HHID&y=HHID&y.C));
     HHID&y=input(HHID&y.C,best12.);
run;

proc sort data=work&f.add_all_01 out=work&f.add_all_02;
     by HHID&y;
run;

data work&f.17_&p._othvill;
     merge work&f.16_&p._othvill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;

data work&f.18_&p._othvill (drop=V84N TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._othvill;

     retain PID_H01-PID_H10 PWRK_H01-PWRK_H10 PWAG_H01-PWAG_H10 i;

     length PID_H01-PID_H10 8;             * 10 for good measure *;
     length PWRK_H01-PWRK_H10 8;             * 10 for good measure *;
     length PWAG_H01-PWAG_H10 8;             * 10 for good measure *;

     array id(1:10) PID_H01-PID_H10;
     array wk(1:10) PWRK_H01-PWRK_H10;
     array wg(1:10) PWAG_H01-PWAG_H10;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 10;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=V84N;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
 
run;

***********************************************
** (2) VILL: PROCESS LABOR FROM SAME VILLAGE **
***********************************************;

** Placeholder for consistency **;

data work&f.16_&p._vill (keep=HHID&y HELPHH&y TOT_WRKS TOT_WAGE);
     set work&f.15_&p._vill;

     TOT_WRKS=NUMWRKS;
     TOT_WAGE=ALLWAGE;
RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._vill;
     merge work&f.16_&p._vill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;

data work&f.18_&p._vill (drop=HELPHH&y TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._vill;

     retain PID_H11-PID_H60 PWRK_H11-PWRK_H60 PWAG_H11-PWAG_H60 i;    ** Starts at 11 to leave room for village **; 

     length PID_H11-PID_H60 8;                 * 50 for good measure *;
     length PWRK_H11-PWRK_H60 8;             * 50 for good measure *;
     length PWAG_H11-PWAG_H60 8;             * 50 for good measure *;

     array id(1:50) PID_H11-PID_H60;
     array wk(1:50) PWRK_H11-PWRK_H60;
     array wg(1:50) PWAG_H11-PWAG_H60;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 50;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=HELPHH&y;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
run;

****************************************
** (3) OTHER: PROCESS ALL OTHER LABOR **
****************************************;

** This labor is aggregated and included in an attribute file **;

** Placeholder for consistency **;

data work&f.16_&p._other (keep=HHID&y HELPHH&y NUMWRKS ALLWAGE);
     set work&f.15_&p._other;

RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._other;
     merge work&f.16_&p._other (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;
** THIS FILE TO BE MERGED BELOW WITH OTHER ATTRIBUTES **;

data work&f.18_&p._other (drop=HELPHH&y NUMWRKS ALLWAGE);
     set work&f.17_&p._other;

     retain OTH_PWRK OTH_PWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            OTH_PWRK=0;
                            OTH_PWAG=0;
                          end;

     if NUMWRKS ne . then OTH_PWRK=OTH_PWRK+NUMWRKS;
     if ALLWAGE ne . then OTH_PWAG=OTH_PWAG+ALLWAGE;

    if last.HHID&y then output;
run;                                                             


***********************************
** Add other attribute variables ** 
***********************************;

** HH-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_hh (drop=HHID&y.C);
     set in&f.2.hh&y (keep=HHID&y RICE rename=(HHID&y=HHID&y.C));

     if RICE in (.,2,8) then RICE=0;

     IS_HH=1;

     HHID&y=input(HHID&y.C,best12.);
    
run;

** Merge with all rice-growing households **;

data work&f._&p._add_all_hh (drop=V84N LOCATION HELPHH&y);
     merge work&f.14_&p (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
                          
     if b=1 then output;
run;

data work&f._&p._add_tot_hh (drop=NUMWRKS ALLWAGE); **Generate Count Vars for ALL labor **;
     set work&f._&p._add_all_hh;

     retain ALL_PWRK ALL_PWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            ALL_PWRK=0;
                            ALL_PWAG=0;
                          end;

     if NUMWRKS ne . then ALL_PWRK=ALL_PWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_PWAG=ALL_PWAG+ALLWAGE;

    if last.HHID&y then output;
run;                           

** MERGE **;

data work&f.19_&p._all_hh;
     merge work&f._&p._add_rice_hh (in=a)
           work&f._&p._add_tot_hh (in=b)
           work&f.18_&p._other (in=c);
     by HHID&y;
     if c=1 then output;
run;

data work&f.19_&p._all_hh_2 (drop=V84);
     set work&f.19_&p._all_hh;
     V84N=input(V84,2.0);
run;

** VILLAGE-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_vill (drop=V84 HHID&y);
     set work&f.add_all_02;
     RICE=1;
     IS_HH=0;
     OTH_PWRK=0;
     OTH_PWAG=0;
     V84N=input(V84,2.0);
run;

** Prep for merge **;

proc sort data=work&f._&p._add_rice_vill out=work&f._&p._add_rice_vill_2 nodupkey;
     by V84N;
run;

proc sort data=work&f.14_&p out=work&f.14_&p._by_v84;
     by V84N;
run;

**Generate Count Vars for ALL labor **;

data work&f._&p._add_tot_vill (drop=HHID&y HELPHH&y NUMWRKS ALLWAGE); 
     set work&f.14_&p._by_v84;

     retain ALL_PWRK ALL_PWAG;
 
     by V84N;

     if first.V84N then do;
                            ALL_PWRK=0;
                            ALL_PWAG=0;
                          end;

     if NUMWRKS ne . then ALL_PWRK=ALL_PWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_PWAG=ALL_PWAG+ALLWAGE;

     if last.V84N then output;
run;                           

data work&f.19_&p._all_vill;
     merge work&f._&p._add_rice_vill_2 (in=a)
           work&f._&p._add_tot_vill (in=b);
     by V84N;

     HHID&y=V84N;
     if b=1 then output;
run;

***************************************
** Merge HH and VILL Attribute Files **
***************************************;

data work&f.19_&p._attrib;
     set work&f.19_&p._all_vill (drop=LOCATION)
         work&f.19_&p._all_hh_2;
run;

**********************************************
** Merge Same and Other Village Labor Files **
**********************************************;

data work&f.19_&p._adjlist;
     merge work&f.18_&p._othvill (in=b)
           work&f.18_&p._vill (in=a);
     by HHID&y;

     if a=1 and b=1 then output;
run;

***********************************************************
** Add Villages to adjacency matrix - all missing values **
***********************************************************;

proc sort data=vill_id_&f.02 out=add_vill_&p._&f._01 nodupkey;
     by V84;
run;

data add_vill_&p._&f._02 (drop=V84 j);
     set add_vill_&p._&f._01 (drop=HHID&y);

     length PID_H01-PID_H60 8;                 
     length PWRK_H01-PWRK_H60 8;             
     length PWAG_H01-PWAG_H60 8;             

     array id(1:60) PID_H01-PID_H60;
     array wk(1:60) PWRK_H01-PWRK_H60;
     array wg(1:60) PWAG_H01-PWAG_H60;
 
     do j= 1 to 60;
        id(j)=.;
        wk(j)=.;
        wg(j)=.;
     end;
 
     HHID&y=input(V84,best12.);    
     V84N=input(V84,2.);    

run;   

data add_vill_&p._&f._03 (drop=V84);
     set work&f.19_&p._adjlist;
     V84N=input(V84,2.0);
run;
 
data add_vill_&p._&f._04;
     set add_vill_&p._&f._02 
         add_vill_&p._&f._03;
run;


****************************************************************************
** UNPAID: Create Village Aggregate Measures for Help from Beyond Village **
****************************************************************************;

%let p=unpaid;    *** saves keystrokes ***;

data work&f.14_unpaid (drop=PAIDHH&y);
     set work&f.13;
     if PAIDHH&y=2;
run;

** SPLIT DATA INTO 3 GROUPS - SAME VILL, OTH VILL, ALL OTHER **;

data work&f.15_&p._othvill (drop=LOCATION);   ** 1 **;
     set work&f.14_&p;
     if LOCATION in (4);
     if HELPHH&y ne .;
run;

data work&f.15_&p._vill (drop=LOCATION);      ** 2 **;
     set work&f.14_&p;
     if LOCATION in (0,1);
     if HELPHH&y ne .;
run;

data work&f.15_&p._other (drop=LOCATION);     ** 3 **;
     set work&f.14_&p;
     if (LOCATION in (2,3,5,6,7,8,9) OR HELPHH&y=.);
run;

****************************************************
** (1) OTHVILL: PROCESS LABOR FROM OTHER VILLAGES **
****************************************************;

** Collapse Labor from the same 84 village **;

data work&f.16_&p._othvill (keep=HHID&y V84N TOT_WRKS TOT_WAGE);
     set work&f.15_&p._othvill;

     retain TOT_WRKS TOT_WAGE;

     by HHID&y V84N;

     if first.V84N then do;
                          TOT_WRKS=0;
                          TOT_WAGE=0;
                        end;
     if NUMWRKS ne . then TOT_WRKS=TOT_WRKS+NUMWRKS;
     if ALLWAGE ne . then TOT_WAGE=TOT_WAGE+ALLWAGE;

    if last.V84N then output;
 
run;

** Merge with all rice-growing households **;

data work&f.add_all_01 (drop=HHID&y.C); 
     set vill_id_&f.02 (rename=(HHID&y=HHID&y.C));
     HHID&y=input(HHID&y.C,best12.);
run;

proc sort data=work&f.add_all_01 out=work&f.add_all_02;
     by HHID&y;
run;

data work&f.17_&p._othvill;
     merge work&f.16_&p._othvill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;


data work&f.18_&p._othvill (drop=V84N TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._othvill;

     retain UID_H01-UID_H10 UWRK_H01-UWRK_H10 UWAG_H01-UWAG_H10 i;

     length UID_H01-UID_H10 8;             * 10 for good measure *;
     length UWRK_H01-UWRK_H10 8;             * 10 for good measure *;
     length UWAG_H01-UWAG_H10 8;             * 10 for good measure *;

     array id(1:10) UID_H01-UID_H10;
     array wk(1:10) UWRK_H01-UWRK_H10;
     array wg(1:10) UWAG_H01-UWAG_H10;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 10;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=V84N;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
 
run;

***********************************************
** (2) VILL: PROCESS LABOR FROM SAME VILLAGE **
***********************************************;

** Placeholder for consistency **;

data work&f.16_&p._vill (keep=HHID&y HELPHH&y TOT_WRKS TOT_WAGE);
     set work&f.15_&p._vill;

     TOT_WRKS=NUMWRKS;
     TOT_WAGE=ALLWAGE;
RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._vill;
     merge work&f.16_&p._vill (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;

data work&f.18_&p._vill (drop=HELPHH&y TOT_WRKS TOT_WAGE i j);
     set work&f.17_&p._vill;

     retain UID_H11-UID_H60 UWRK_H11-UWRK_H60 UWAG_H11-UWAG_H60 i;    ** Starts at 11 to leave room for village **; 

     length UID_H11-UID_H60 8;                 * 50 for good measure *;
     length UWRK_H11-UWRK_H60 8;             * 50 for good measure *;
     length UWAG_H11-UWAG_H60 8;             * 50 for good measure *;

     array id(1:50) UID_H11-UID_H60;
     array wk(1:50) UWRK_H11-UWRK_H60;
     array wg(1:50) UWAG_H11-UWAG_H60;
 
     by HHID&y;

     if first.HHID&y then do;
                            do j= 1 to 50;
                                id(j)=.;
                                wk(j)=.;
                                wg(j)=.;
                             end;
                             i=1;
                          end;

     id(i)=HELPHH&y;
     wk(i)=TOT_WRKS;
     wg(i)=TOT_WAGE;
     i=i+1;

    if last.HHID&y then output;
run;

****************************************
** (3) OTHER: PROCESS ALL OTHER LABOR **
****************************************;

** This labor is aggregated and included in an attribute file **;

** Placeholder for consistency **;

data work&f.16_&p._other (keep=HHID&y HELPHH&y NUMWRKS ALLWAGE);
     set work&f.15_&p._other;

RUN;

** Merge with all rice-growing households **;

data work&f.17_&p._other;
     merge work&f.16_&p._other (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
     if b=1 then output;
run;

** Unstack data from child file to mother (household) file **;
** THIS FILE TO BE MERGED BELOW WITH OTHER ATTRIBUTES **;

data work&f.18_&p._other (drop=HELPHH&y NUMWRKS ALLWAGE);
     set work&f.17_&p._other;

     retain OTH_UWRK OTH_UWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            OTH_UWRK=0;
                            OTH_UWAG=0;
                          end;

     if NUMWRKS ne . then OTH_UWRK=OTH_UWRK+NUMWRKS;
     if ALLWAGE ne . then OTH_UWAG=OTH_UWAG+ALLWAGE;

    if last.HHID&y then output;
run;                                                             


***********************************
** Add other attribute variables ** 
***********************************;

** HH-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_hh (drop=HHID&y.C);
     set in&f.2.hh&y (keep=HHID&y RICE rename=(HHID&y=HHID&y.C));

     if RICE in (.,2,8) then RICE=0;

     IS_HH=1;

     HHID&y=input(HHID&y.C,best12.);

run;

** Merge with all rice-growing households **;

data work&f._&p._add_all_hh (drop=V84N LOCATION HELPHH&y);
     merge work&f.14_&p (in=a)
           work&f.add_all_02 (in=b);
     by HHID&y;
                          
     if b=1 then output;
run;

data work&f._&p._add_tot_hh (drop=NUMWRKS ALLWAGE); **Generate Count Vars for ALL labor **;
     set work&f._&p._add_all_hh;

     retain ALL_UWRK ALL_UWAG;
 
     by HHID&y;

     if first.HHID&y then do;
                            ALL_UWRK=0;
                            ALL_UWAG=0;
                          end;

     if NUMWRKS ne . then ALL_UWRK=ALL_UWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_UWAG=ALL_UWAG+ALLWAGE;

    if last.HHID&y then output;
run;                           

** MERGE **;

data work&f.19_&p._all_hh;
     merge work&f._&p._add_rice_hh (in=a)
           work&f._&p._add_tot_hh (in=b)
           work&f.18_&p._other (in=c);
     by HHID&y;
     if c=1 then output;
run;

data work&f.19_&p._all_hh_2 (drop=V84);
     set work&f.19_&p._all_hh;
     V84N=input(V84,2.0);
run;

** VILLAGE-LEVEL ATTRIBUTES **;

data work&f._&p._add_rice_vill (drop=V84 HHID&y);
     set work&f.add_all_02;
     RICE=1;
     IS_HH=0;
     OTH_UWRK=0;
     OTH_UWAG=0;
     V84N=input(V84,2.0);
run;

** Prep for merge **;

proc sort data=work&f._&p._add_rice_vill out=work&f._&p._add_rice_vill_2 nodupkey;
     by V84N;
run;

proc sort data=work&f.14_&p out=work&f.14_&p._by_v84;
     by V84N;
run;

**Generate Count Vars for ALL labor **;

data work&f._&p._add_tot_vill (drop=HHID&y HELPHH&y NUMWRKS ALLWAGE); 
     set work&f.14_&p._by_v84;

     retain ALL_UWRK ALL_UWAG;
 
     by V84N;

     if first.V84N then do;
                            ALL_UWRK=0;
                            ALL_UWAG=0;
                          end;

     if NUMWRKS ne . then ALL_UWRK=ALL_UWRK+NUMWRKS;
     if ALLWAGE ne . then ALL_UWAG=ALL_UWAG+ALLWAGE;

     if last.V84N then output;
run;                           

data work&f.19_&p._all_vill;
     merge work&f._&p._add_rice_vill_2 (in=a)
           work&f._&p._add_tot_vill (in=b);
     by V84N;

     HHID&y=V84N;
     if b=1 then output;
run;

***************************************
** Merge HH and VILL Attribute Files **
***************************************;

data work&f.19_&p._attrib;
     set work&f.19_&p._all_vill (drop=LOCATION)
         work&f.19_&p._all_hh_2;
run;

**********************************************
** Merge Same and Other Village Labor Files **
**********************************************;

data work&f.19_&p._adjlist;
     merge work&f.18_&p._othvill (in=b)
           work&f.18_&p._vill (in=a);
     by HHID&y;

     if a=1 and b=1 then output;
run;

***********************************************************
** Add Villages to adjacency matrix - all missing values **
***********************************************************;

proc sort data=vill_id_&f.02 out=add_vill_&p._&f._01 nodupkey;
     by V84;
run;

data add_vill_&p._&f._02 (drop=V84 j);
     set add_vill_&p._&f._01 (drop=HHID&y);

     length UID_H01-UID_H60 8;                 
     length UWRK_H01-UWRK_H60 8;             
     length UWAG_H01-UWAG_H60 8;             

     array id(1:60) UID_H01-UID_H60;
     array wk(1:60) UWRK_H01-UWRK_H60;
     array wg(1:60) UWAG_H01-UWAG_H60;
 
     do j= 1 to 60;
        id(j)=.;
        wk(j)=.;
        wg(j)=.;
     end;
 
     HHID&y=input(V84,best12.);    
     V84N=input(V84,2.);    

run;   

data add_vill_&p._&f._03 (drop=V84);
     set work&f.19_&p._adjlist;
     V84N=input(V84,2.0);
run;
 
data add_vill_&p._&f._04;
     set add_vill_&p._&f._02 
         add_vill_&p._&f._03;
run;


*******************************************************************************************************************;

*********************************************************************
** Temporarily Merge Together All 4 Final Files to purge ID errors **
*********************************************************************;

proc sort data=add_vill_paid_&f._04 out=work&f.20_paid_adj;
     by HHID&y V84N;
run;

proc sort data=add_vill_unpaid_&f._04 out=work&f.20_unpaid_adj;
     by HHID&y V84N;
run;

proc sort data=work&f.19_paid_attrib out=work&f.20_paid_att;
     by HHID&y V84N;
run;

proc sort data=work&f.19_unpaid_attrib out=work&f.20_unpaid_att;
     by HHID&y V84N;
run;


data work&f.21_all_files;
     merge work&f.20_paid_adj (in=a)
           work&f.20_unpaid_adj (in=b)
           work&f.20_paid_att (in=c)
           work&f.20_unpaid_att (drop=RICE IS_HH in=d);
     by HHID&y V84N;
     if a=1 and b=1 and c=1 and d=1 then output;
run;  

*******************************************
** Format files to be exported to UCINET **
*******************************************;

** I have decided to revert these matrices back to hh-only - 2011 02 07 **;
** These changes are reflected only in the four data steps below **;
** The village-level information still exists in all prior data steps **;
** In the future, I'd like to add a village-level only analysis **;

** I have also re-combined all files into a single one as of 2011 02 09 **;

data r&y._all (drop=PID_H01-PID_H10 PWRK_H01-PWRK_H10 PWAG_H01-PWAG_H10
                    UID_H01-UID_H10 UWRK_H01-UWRK_H10 UWAG_H01-UWAG_H10);
     set work&f.21_all_files;

     ALL_WRKS=ALL_PWRK+ALL_UWRK;
     ALL_WAGE=ALL_PWAG+ALL_UWAG;

     if HHID&y>100;               * Remove village datalines *;
run;

**************************************************************************************************;

****************************************************************************
* 10 ** VILLAGE NETWORKS: RICE ALL ** Create Rice Network ATTRIBUTE FILES **
****************************************************************************;

%macro v_split2 (numvill=);  %* macro splits villages *;

   %* NUMVILL=Number of Unique Villages in file *;

%do i=1 %to &numvill;
    data r00_a&i (drop=V84N);
         set r&y._all;
         if V84N=&i;
    run;
%end;
%mend v_split2;
%v_split2 (numvill=51);


%macro attrib1(numvill=);
%do i=1 %to &numvill;
    data _null_ ;
         %let p1=%quote(/home/jhull/nangrong/data_paj/2000_att/v0);
         %let p2=%quote(00at.txt);
         set r00_a&i;
         file "&p1.&i.&p2";
         put RICE IS_HH ALL_PWRK ALL_PWAG ALL_UWRK ALL_UWAG ALL_WRKS ALL_WAGE;
run;
%end;
%mend attrib1;

%attrib1(numvill=9);

%macro attrib2(numvill=);
%do i=10 %to &numvill;
    data _null_ ;
         %let p1=%quote(/home/jhull/nangrong/data_paj/2000_att/v);
         %let p2=%quote(00at.txt);
         set r00_a&i;
         file "&p1.&i.&p2";
         put RICE IS_HH ALL_PWRK ALL_PWAG ALL_UWRK ALL_UWAG ALL_WRKS ALL_WAGE;
run;
%end;
%mend attrib2;

%attrib2(numvill=51);




R:

In [ ]:
###################################################################
# PROGRAM NAME: /home/jhull/nangrong?prog_r/p02_nets/p02_0001.R
# PROGRAMMER: james r. hull
# DATE: 2011 03 13
# PURPOSE: Use STATNET Package to Analyze Nang Rong Social Networks
#          This Script Puts Together Rice Labor and Sibling Networks
#          Only into a single multiplex sociogram, colored and etc.
###################################################################

###################################################################
# Set Environmental Variables and Conditions

# set the random seed for strict reproducibility
set.seed(13)

####################################################################
# Make Graphs - Two For-Loops, one for 01-09, another for 10-51

for(i in seq(1,9,1)) {

    # Define Path Names for Adjacency Matrices
    f.adj.a1<-as.name(paste("/home/jhull/nangrong/data_paj/2000/a1-adj/v0",i,"00a1.adj",sep=""))   #Raw Adjaceny File
    f.adj.a2<-as.name(paste("/home/jhull/nangrong/data_paj/2000/a2-adj/v0",i,"00a2.adj",sep=""))   #Raw Adjaceny File
    f.adj.b1<-as.name(paste("/home/jhull/nangrong/data_paj/2000/b1-adj/v0",i,"00b1.adj",sep=""))   #Raw Adjaceny File
    f.adj.b2<-as.name(paste("/home/jhull/nangrong/data_paj/2000/b2-adj/v0",i,"00b2.adj",sep=""))   #Raw Adjaceny File

    # Define Path Names for ID Sets
    f.id.a1<-as.name(paste("/home/jhull/nangrong/data_paj/2000/a1-id/v0",i,"00a1.id",sep=""))   #Raw ID Number File
    f.id.a2<-as.name(paste("/home/jhull/nangrong/data_paj/2000/a2-id/v0",i,"00a2.id",sep=""))   #Raw ID Number File
    f.id.b1<-as.name(paste("/home/jhull/nangrong/data_paj/2000/b1-id/v0",i,"00b1.id",sep=""))   #Raw ID Number File
    f.id.b2<-as.name(paste("/home/jhull/nangrong/data_paj/2000/b2-id/v0",i,"00b2.id",sep=""))   #Raw ID Number File


    f.att.all<-as.name(paste("/home/jhull/nangrong/data_paj/2000_att/v0",i,"00at.txt",sep=""))  #Text Attribute File

    f.png.loc<-as.name(paste("/home/jhull/nangrong/png_r/p02_nets/2000/a1-b2/v0",i,"00a1.png",sep=""))  #Saved PNG Graphic File
    f.title<-as.name(paste("2000: Village ",i,": Multiple Networks",sep=""))                #Title for Saved PNG

    # Load Adjacency Matrices
    adj.a1 <- as.matrix(read.table(paste(f.adj.a1)))
    adj.a2 <- as.matrix(read.table(paste(f.adj.a2)))
    adj.b1 <- as.matrix(read.table(paste(f.adj.b1)))
    adj.b2 <- as.matrix(read.table(paste(f.adj.b2)))

    # Binarize Valued Matrices (Just do it for all of them)

    adj.a1.01 <- matrix(0,nrow(adj.a1),ncol(adj.a1))
    adj.a2.01 <- matrix(0,nrow(adj.a2),ncol(adj.a2))
    adj.b1.01 <- matrix(0,nrow(adj.b1),ncol(adj.b1))
    adj.b2.01 <- matrix(0,nrow(adj.b2),ncol(adj.b2))

    adj.a1.01 [adj.a1!=0] <- 1
    adj.a2.01 [adj.a2!=0] <- 1
    adj.b1.01 [adj.b1!=0] <- 1
    adj.b2.01 [adj.b2!=0] <- 1

   # Load ID sets and Apply to Matrices
  
    id.a1 <- read.table(paste(f.id.a1))
    id.a2 <- read.table(paste(f.id.a2))
    id.b1 <- read.table(paste(f.id.b1))
    id.b2 <- read.table(paste(f.id.b2))

    colnames(adj.a1.01) <- id.a1
    rownames(adj.a1.01) <- id.a1

    colnames(adj.a2.01) <- id.a2
    rownames(adj.a2.01) <- id.a2

    colnames(adj.b1.01) <- id.b1
    rownames(adj.b1.01) <- id.b1

    colnames(adj.b2.01) <- id.b2
    rownames(adj.b2.01) <- id.b2

    # Create Edge Colors Matrix (2 dimensional)

    colors.01 <- ifelse((adj.a1.01!=0),"a1","0")
    colors.01 <- ifelse((adj.a2.01!=0),"a2",colors.01)
    colors.01 <- ifelse((adj.b1.01!=0),"b1",colors.01)
    colors.01 <- ifelse((adj.b2.01!=0),"b2",colors.01)

    multi.01 <- adj.a1.01+adj.a2.01+adj.b1.01+adj.b2.01

    colors.02 <- ifelse((multi.01>1),(as.character(multi.01)),colors.01)

    #Assign Colors to Single Relations and Shades of Grey to Multiple Relations

    color.array <- rbind(c("a1","green"),    #Paid Labor = Green
                         c("a2","yellow"),    #Unaid Labor = Light Yellow
                         c("b1","blue"),     #Male Sibling = Blue
                         c("b2","blue"),     #Female Sibling = (Also) Blue  
                         c("2","gray72"),     #Multiplex (Any Two) = Lightest Grey
                         c("3","gray47"),     #Multiplex (Any Three) = Mid-Range Grey
                         c("4","gray23")      #Multiplex (Any Four) = Mid-Range Grey   
                        )
    colors.03 <- lookup(colors.02, color.array)

    # Create 3-dimensional matrix (Z-dimension is the specific relation)
    # Paid and Unpaid Labor
    values.01  <- abind(adj.a1.01,adj.a2.01,along=3,new.names=c("paid","unpaid"))
    # Male and Female Siblings
    values.02  <- abind(adj.b1.01,adj.b2.01,along=3,new.names=c("male sib","female sib"))
    # (1) Paid Labor (2) Unpaid Labor (3) Male Siblings (4) Female Siblings
    values.03  <- abind(values.01,values.02,along=3)
 
    # Load Attribute File
    att <- read.table(file=paste(f.att.all), header=F,col.names=c("RICE","IS_HH","ALL_PWRK","ALL_PWAG",
                      "ALL_UWRK","ALL_UWAG","ALL_WRKS","ALL_WAGE"))

    # Recode Attributes for Current Graphical Display
    att$RICE2 [att$RICE==0] <- "blue"
    att$RICE2 [att$RICE==1] <- "red"
    att$ALL_WRK2 [att$ALL_WRKS==0] <- 0.5
    att$ALL_WRK2 [att$ALL_WRKS > 0 & att$ALL_WRKS <= 10] <- 0.75
    att$ALL_WRK2 [att$ALL_WRKS > 10 & att$ALL_WRKS <= 20] <- 1.0
    att$ALL_WRK2 [att$ALL_WRKS > 20 & att$ALL_WRKS <= 30] <- 1.25
    att$ALL_WRK2 [att$ALL_WRKS > 30 & att$ALL_WRKS <= 40] <- 1.5
    att$ALL_WRK2 [att$ALL_WRKS > 40 & att$ALL_WRKS <= 50] <- 1.75
    att$ALL_WRK2 [att$ALL_WRKS > 50] <- 2.0

    my.vertex.col <- as.vector(att$RICE2)  
    my.vertex.cex <- as.vector(att$ALL_WRK2)  

    # Open Graphics File
    png(file=paste(f.png.loc), bg="white",height=960, width=960)

    #Generate Complete Multiplex Graph with all vertices and edges properly colored/sized
    gplot(values.03[,,1] | values.03[,,2] | values.03[,,3] | values.03[,,4], 
          displaylabels=FALSE,
          edge.col=colors.03,
          vertex.col = my.vertex.col,
          vertex.cex=my.vertex.cex,
          vertex.sides=20,
          edge.lwd=2,
          arrowhead.cex=0.75      
          )

    #Add Title To Graphic
    title(main=paste(f.title), col.main="black", font.main=20)

    #Add Legend #1 To Graphic
    legend(x="bottom", 
           inset=0.03, 
           c("Rice Growing Household", "Non-Rice-Growing Household"),
           pch=20,
           cex=1, 
           pt.cex=3, 
           col=c("red","blue"), 
           bty="n",
           horiz=TRUE
          )

    #Add Legend #1 To Graphic
    legend(x="bottom",
           c("Paid Labor Tie","Unpaid Labor Tie","Sibling Tie","Multiplex Tie (2)","Multiplex Tie (3)","Multiplex Tie (4)"),
           inset=0.015,
           cex=1, 
           col=c("green","yellow","blue","gray72","gray47","gray23"), 
           lwd=3, 
           bty="n",
           horiz=TRUE
          )

    #Add Legend #3 To Graphic
    legend(x="bottom", 
           c("Vertices Sized by the Total Number of Rice Laborers Used"),
           cex=1,
           bty="n")

    # Clean Up to Prepare for Next Iteration
      rm("f.adj.a1",
          "f.adj.a2",
          "f.adj.b1",
          "f.adj.b2",
          "adj.a1.01",
          "adj.a2.01",
          "adj.b1.01",
          "adj.b2.01",
          "f.id.a1",
          "f.id.a2",
          "f.id.b1",
          "f.id.b2",
          "f.att.all",
          "f.png.loc",
          "f.title",
          "adj.a1",
          "adj.a2",
          "adj.b1",
          "adj.b2",
          "id.a1",
          "id.a2",
          "id.b1",
          "id.b2",
          "multi.01",
          "colors.01",
          "colors.02",
          "colors.03",
          "values.01",
          "values.02",
          "values.03",
          "my.vertex.col",
          "my.vertex.cex"
        )

    # Flush Graphic Output to File
    dev.off()

} # End Of: Make Graphs


for(i in seq(10,51,1)) {

    # Define Path Names for Adjacency Matrices
    f.adj.a1<-as.name(paste("/home/jhull/nangrong/data_paj/2000/a1-adj/v",i,"00a1.adj",sep=""))   #Raw Adjaceny File
    f.adj.a2<-as.name(paste("/home/jhull/nangrong/data_paj/2000/a2-adj/v",i,"00a2.adj",sep=""))   #Raw Adjaceny File
    f.adj.b1<-as.name(paste("/home/jhull/nangrong/data_paj/2000/b1-adj/v",i,"00b1.adj",sep=""))   #Raw Adjaceny File
    f.adj.b2<-as.name(paste("/home/jhull/nangrong/data_paj/2000/b2-adj/v",i,"00b2.adj",sep=""))   #Raw Adjaceny File

    # Define Path Names for ID Sets
    f.id.a1<-as.name(paste("/home/jhull/nangrong/data_paj/2000/a1-id/v",i,"00a1.id",sep=""))   #Raw ID Number File
    f.id.a2<-as.name(paste("/home/jhull/nangrong/data_paj/2000/a2-id/v",i,"00a2.id",sep=""))   #Raw ID Number File
    f.id.b1<-as.name(paste("/home/jhull/nangrong/data_paj/2000/b1-id/v",i,"00b1.id",sep=""))   #Raw ID Number File
    f.id.b2<-as.name(paste("/home/jhull/nangrong/data_paj/2000/b2-id/v",i,"00b2.id",sep=""))   #Raw ID Number File


    f.att.all<-as.name(paste("/home/jhull/nangrong/data_paj/2000_att/v",i,"00at.txt",sep=""))  #Text Attribute File

    f.png.loc<-as.name(paste("/home/jhull/nangrong/png_r/p02_nets/2000/a1-b2/v",i,"00a1.png",sep=""))  #Saved PNG Graphic File
    f.title<-as.name(paste("2000: Village ",i,": Multiple Networks",sep=""))                #Title for Saved PNG

    # Load Adjacency Matrices
    adj.a1 <- as.matrix(read.table(paste(f.adj.a1)))
    adj.a2 <- as.matrix(read.table(paste(f.adj.a2)))
    adj.b1 <- as.matrix(read.table(paste(f.adj.b1)))
    adj.b2 <- as.matrix(read.table(paste(f.adj.b2)))

    # Binarize Valued Matrices (Just do it for all of them)

    adj.a1.01 <- matrix(0,nrow(adj.a1),ncol(adj.a1))
    adj.a2.01 <- matrix(0,nrow(adj.a2),ncol(adj.a2))
    adj.b1.01 <- matrix(0,nrow(adj.b1),ncol(adj.b1))
    adj.b2.01 <- matrix(0,nrow(adj.b2),ncol(adj.b2))

    adj.a1.01 [adj.a1!=0] <- 1
    adj.a2.01 [adj.a2!=0] <- 1
    adj.b1.01 [adj.b1!=0] <- 1
    adj.b2.01 [adj.b2!=0] <- 1

   # Load ID sets and Apply to Matrices
  
    id.a1 <- read.table(paste(f.id.a1))
    id.a2 <- read.table(paste(f.id.a2))
    id.b1 <- read.table(paste(f.id.b1))
    id.b2 <- read.table(paste(f.id.b2))

    colnames(adj.a1.01) <- id.a1
    rownames(adj.a1.01) <- id.a1

    colnames(adj.a2.01) <- id.a2
    rownames(adj.a2.01) <- id.a2

    colnames(adj.b1.01) <- id.b1
    rownames(adj.b1.01) <- id.b1

    colnames(adj.b2.01) <- id.b2
    rownames(adj.b2.01) <- id.b2

    # Create Edge Colors Matrix (2 dimensional)

    colors.01 <- ifelse((adj.a1.01!=0),"a1","0")
    colors.01 <- ifelse((adj.a2.01!=0),"a2",colors.01)
    colors.01 <- ifelse((adj.b1.01!=0),"b1",colors.01)
    colors.01 <- ifelse((adj.b2.01!=0),"b2",colors.01)

    multi.01 <- adj.a1.01+adj.a2.01+adj.b1.01+adj.b2.01

    colors.02 <- ifelse((multi.01>1),(as.character(multi.01)),colors.01)

    #Assign Colors to Single Relations and Shades of Grey to Multiple Relations

    color.array <- rbind(c("a1","green"),    #Paid Labor = Green
                         c("a2","yellow"),    #Unaid Labor = Light Yellow
                         c("b1","blue"),     #Male Sibling = Blue
                         c("b2","blue"),     #Female Sibling = (Also) Blue  
                         c("2","gray72"),     #Multiplex (Any Two) = Lightest Grey
                         c("3","gray47"),     #Multiplex (Any Three) = Mid-Range Grey
                         c("4","gray23")      #Multiplex (Any Four) = Mid-Range Grey   
                        )
    colors.03 <- lookup(colors.02, color.array)

    # Create 3-dimensional matrix (Z-dimension is the specific relation)
    # Paid and Unpaid Labor
    values.01  <- abind(adj.a1.01,adj.a2.01,along=3,new.names=c("paid","unpaid"))
    # Male and Female Siblings
    values.02  <- abind(adj.b1.01,adj.b2.01,along=3,new.names=c("male sib","female sib"))
    # (1) Paid Labor (2) Unpaid Labor (3) Male Siblings (4) Female Siblings
    values.03  <- abind(values.01,values.02,along=3)
 
    # Load Attribute File
    att <- read.table(file=paste(f.att.all), header=F,col.names=c("RICE","IS_HH","ALL_PWRK","ALL_PWAG",
                      "ALL_UWRK","ALL_UWAG","ALL_WRKS","ALL_WAGE"))

    # Recode Attributes for Current Graphical Display
    att$RICE2 [att$RICE==0] <- "blue"
    att$RICE2 [att$RICE==1] <- "red"
    att$ALL_WRK2 [att$ALL_WRKS==0] <- 0.5
    att$ALL_WRK2 [att$ALL_WRKS > 0 & att$ALL_WRKS <= 10] <- 0.75
    att$ALL_WRK2 [att$ALL_WRKS > 10 & att$ALL_WRKS <= 20] <- 1.0
    att$ALL_WRK2 [att$ALL_WRKS > 20 & att$ALL_WRKS <= 30] <- 1.25
    att$ALL_WRK2 [att$ALL_WRKS > 30 & att$ALL_WRKS <= 40] <- 1.5
    att$ALL_WRK2 [att$ALL_WRKS > 40 & att$ALL_WRKS <= 50] <- 1.75
    att$ALL_WRK2 [att$ALL_WRKS > 50] <- 2.0

    my.vertex.col <- as.vector(att$RICE2)  
    my.vertex.cex <- as.vector(att$ALL_WRK2)  

    # Open Graphics File
    png(file=paste(f.png.loc), bg="white",height=960, width=960)

    #Generate Complete Multiplex Graph with all vertices and edges properly colored/sized
    gplot(values.03[,,1] | values.03[,,2] | values.03[,,3] | values.03[,,4], 
          displaylabels=FALSE,
          edge.col=colors.03,
          vertex.col = my.vertex.col,
          vertex.cex=my.vertex.cex,
          vertex.sides=20,
          edge.lwd=2,
          arrowhead.cex=0.75      
          )

    #Add Title To Graphic
    title(main=paste(f.title), col.main="black", font.main=20)

    #Add Legend #1 To Graphic
    legend(x="bottom", 
           inset=0.03, 
           c("Rice Growing Household", "Non-Rice-Growing Household"),
           pch=20,
           cex=1, 
           pt.cex=3, 
           col=c("red","blue"), 
           bty="n",
           horiz=TRUE
          )

    #Add Legend #1 To Graphic
    legend(x="bottom",
           c("Paid Labor Tie","Unpaid Labor Tie","Sibling Tie","Multiplex Tie (2)","Multiplex Tie (3)","Multiplex Tie (4)"),
           inset=0.015,
           cex=1, 
           col=c("green","yellow","blue","gray72","gray47","gray23"), 
           lwd=3, 
           bty="n",
           horiz=TRUE
          )

    #Add Legend #3 To Graphic
    legend(x="bottom", 
           c("Vertices Sized by the Total Number of Rice Laborers Used"),
           cex=1,
           bty="n")

    # Clean Up to Prepare for Next Iteration
      rm("f.adj.a1",
          "f.adj.a2",
          "f.adj.b1",
          "f.adj.b2",
          "adj.a1.01",
          "adj.a2.01",
          "adj.b1.01",
          "adj.b2.01",
          "f.id.a1",
          "f.id.a2",
          "f.id.b1",
          "f.id.b2",
          "f.att.all",
          "f.png.loc",
          "f.title",
          "adj.a1",
          "adj.a2",
          "adj.b1",
          "adj.b2",
          "id.a1",
          "id.a2",
          "id.b1",
          "id.b2",
          "multi.01",
          "colors.01",
          "colors.02",
          "colors.03",
          "values.01",
          "values.02",
          "values.03",
          "my.vertex.col",
          "my.vertex.cex"
        )

    # Flush Graphic Output to File
    dev.off()

} # End Of: Make Graphs

###################################################################
# Clean Up and Close
delete.all <- function() 
      rm(list=ls(pos=.GlobalEnv), pos=.GlobalEnv) 
delete.all()